<a href="https://colab.research.google.com/github/chengtom60/chyshysh/blob/main/%E4%BC%98%E5%8C%96%E5%90%8E%E7%9A%84%E8%82%A1%E7%A5%A8%E9%A2%84%E6%B5%8B%E6%A8%A1%E5%9E%8B_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 ---
    # Use pandas_ta to calculate technical indicators
    df['MA5'] = df.ta.sma(length=5) # Using pandas_ta for SMA
    df['MA10'] = df.ta.sma(length=10) # Using pandas_ta for SMA
    df['RSI'] = df.ta.rsi(length=14) # Using pandas_ta for RSI
    # pandas_ta's MACD returns a DataFrame with MACD, Histogram, and Signal line
    macd_results = df.ta.macd(fast=12, slow=26, signal=9) # Using pandas_ta for MACD
    df['MACD'] = macd_results['MACD_12_26_9']
    df['MACD_signal'] = macd_results['MACDh_12_26_9'] # Note: pandas_ta names the histogram 'MACDh'
    df['MACD_hist'] = macd_results['MACDs_12_26_9'] # Note: pandas_ta names the signal line 'MACDs'


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # 定义特征列和目标列
    # Ensure the feature columns match the new MACD column names if needed, but for simplicity,
    # we'll stick to the original feature names as the calculations are similar in concept.
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # 训练模型
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # 显示训练过程
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # 请确保您有这个CSV文件，或者替换为您自己的文件路径
    # CSV文件需要包含'日期'和'收盘价'列
    try:
        X, y, df_full = load_and_prepare_data("AAPL_daily.csv")
    except FileNotFoundError:
        print("错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。")
        exit()

    # --- 2. 数据缩放 ---
    # 创建缩放器
    scaler = MinMaxScaler()
    # 只在训练数据上拟合缩放器，然后转换所有数据
    # 为了简化，这里我们先对整个X进行变换，但在严格的流程中，应先分割再变换
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. 创建序列数据 ---
    TIME_STEPS = 10  # 使用过去10天的数据作为输入
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. 划分训练集和测试集 (时间序列方式) ---
    # 确保训练集和测试集不会被打乱，保持时间顺序
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"总样本数: {len(X_seq)}")
    print(f"训练集样本数: {len(X_train)}")
    print(f"测试集样本数: {len(X_test)}")

    # --- 5. 训练模型 ---
    print("\n正在训练模型...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)
    model.save('stock_predictor_v2.h5')
    print("\n模型已保存为 stock_predictor_v2.h5")

    # --- 6. 在测试集上评估模型 ---
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"\n模型在测试集上的准确率: {accuracy*100:.2f}%")

    # --- 7. 进行新数据预测 ---
    # 假设我们要预测最新的数据点之后的情况
    # 我们需要获取数据集中最后 `TIME_STEPS` 天的数据
    last_sequence = X_scaled.iloc[-TIME_STEPS:].values

    # 将其 reshape 成模型需要的输入格式 (1, time_steps, n_features)
    last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

    # 进行预测
    prediction = model.predict(last_sequence_reshaped)
    prediction_proba = prediction[0][0]

    print("\n--- 最新数据预测 ---")
    print(f"基于最近 {TIME_STEPS} 天的数据...")
    print(f"预测未来5天价格上涨的概率: {prediction_proba*100:.2f}%")
    if prediction_proba > 0.5:
        print("预测结果: 看涨")
    else:
        print("预测结果: 看跌")

ImportError: cannot import name 'NaN' from 'numpy' (/usr/local/lib/python3.11/dist-packages/numpy/__init__.py)

In [3]:
!pip install TA-Lib

  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)


In [4]:
!apt-get install ta-lib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ta-lib


In [5]:
!pip install TA-Lib

  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)


In [6]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install TA-Lib

--2025-07-16 03:58:03--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:c95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.13.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2025-07-16 03:58:03--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:d95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://gigenet.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2025-07-16 03:58:03--  http://gigenet.dl.source

In [1]:
!pip install https://github.com/matplotlib/matplotlib/archive/v3.3.4.tar.gz
!pip install ta-lib

  Using cached https://github.com/matplotlib/matplotlib/archive/v3.3.4.tar.gz
  Preparing metadata (setup.py) ... done
  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (ta-lib)


In [3]:
import talib
print("talib imported successfully!")

ModuleNotFoundError: No module named 'talib'

In [4]:
!pip install --upgrade setuptools wheel

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.21.0 requires matplotlib>=3.5, but you have matplotlib 3.3.4 which is incompatible.


In [1]:
!pip install ta-lib==0.4.0

ERROR: Could not find a version that satisfies the requirement ta-lib==0.4.0 (from versions: 0.4.9, 0.4.14, 0.4.15, 0.4.16, 0.4.17, 0.4.18, 0.4.19, 0.4.20, 0.4.21, 0.4.22, 0.4.23, 0.4.24, 0.4.25, 0.4.26, 0.4.27, 0.4.28, 0.4.29, 0.4.30, 0.4.31, 0.4.32, 0.4.33, 0.4.34, 0.4.35, 0.4.36, 0.4.37, 0.4.38, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.5.4, 0.5.5, 0.6.0, 0.6.1, 0.6.2, 0.6.3, 0.6.4)
ERROR: No matching distribution found for ta-lib==0.4.0


In [2]:
!pip install ta-lib==0.6.4

  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (ta-lib)


In [3]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218984 sha256=93cda047919b0a50a4d951d81ddff415ba907980540bb6a48001fea50ad50871
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [8]:
!pip install --upgrade pandas_ta

# Task
Explain the error in the selected code, and if possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error. If the error is related to package installation, explore alternative packages or methods to achieve the same functionality. The user is encountering errors during the installation of the `TA-Lib` package and wants to use `pandas_ta` and `NumPy 2.0.2` as alternatives or explore other ways to calculate technical indicators. The user also wants to rerun the code after the fix.

## Identify indicator calculations

### Subtask:
Identify the sections in the `load_and_prepare_data` function where MA, RSI, and MACD are calculated using `pandas_ta`.


**Reasoning**:
Identify the lines in the `load_and_prepare_data` function where `pandas_ta` is used to calculate technical indicators.



In [9]:
# Examine the load_and_prepare_data function to identify pandas_ta usage.
# The relevant lines are within the "--- 计算技术指标 ---" section.
# These lines are:
# df['MA5'] = df.ta.sma(length=5)
# df['MA10'] = df.ta.sma(length=10)
# df['RSI'] = df.ta.rsi(length=14)
# macd_results = df.ta.macd(fast=12, slow=26, signal=9)
# df['MACD'] = macd_results['MACD_12_26_9']
# df['MACD_signal'] = macd_results['MACDh_12_26_9']
# df['MACD_hist'] = macd_results['MACDs_12_26_9']

# The instruction only asks to identify, not to execute code.
# I have identified the lines and the specific function calls.

## Implement manual calculations

### Subtask:
Replace the `pandas_ta` function calls with manual calculations using pandas for MA, RSI, and MACD.


**Reasoning**:
Replace the pandas_ta function calls with manual calculations for MA, RSI, and MACD using pandas methods.



In [10]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # 定义特征列和目标列
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # 训练模型
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # 显示训练过程
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # 请确保您有这个CSV文件，或者替换为您自己的文件路径
    # CSV文件需要包含'日期'和'收盘价'列
    try:
        X, y, df_full = load_and_prepare_data("AAPL_daily.csv")
    except FileNotFoundError:
        print("错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。")
        exit()

    # --- 2. 数据缩放 ---
    # 创建缩放器
    scaler = MinMaxScaler()
    # 只在训练数据上拟合缩放器，然后转换所有数据
    # 为了简化，这里我们先对整个X进行变换，但在严格的流程中，应先分割再变换
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. 创建序列数据 ---
    TIME_STEPS = 10  # 使用过去10天的数据作为输入
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. 划分训练集和测试集 (时间序列方式) ---
    # 确保训练集和测试集不会被打乱，保持时间顺序
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"总样本数: {len(X_seq)}")
    print(f"训练集样本数: {len(X_train)}")
    print(f"测试集样本数: {len(X_test)}")

    # --- 5. 训练模型 ---
    print("\n正在训练模型...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)
    model.save('stock_predictor_v2.h5')
    print("\n模型已保存为 stock_predictor_v2.h5")

    # --- 6. 在测试集上评估模型 ---
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"\n模型在测试集上的准确率: {accuracy*100:.2f}%")

    # --- 7. 进行新数据预测 ---
    # 假设我们要预测最新的数据点之后的情况
    # 我们需要获取数据集中最后 `TIME_STEPS` 天的数据
    last_sequence = X_scaled.iloc[-TIME_STEPS:].values

    # 将其 reshape 成模型需要的输入格式 (1, time_steps, n_features)
    last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

    # 进行预测
    prediction = model.predict(last_sequence_reshaped)
    prediction_proba = prediction[0][0]

    print("\n--- 最新数据预测 ---")
    print(f"基于最近 {TIME_STEPS} 天的数据...")
    print(f"预测未来5天价格上涨的概率: {prediction_proba*100:.2f}%")
    if prediction_proba > 0.5:
        print("预测结果: 看涨")
    else:
        print("预测结果: 看跌")

错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。


NameError: name 'X' is not defined

**Reasoning**:
The error indicates that the file 'AAPL_daily.csv' was not found, causing the `load_and_prepare_data` function to exit before defining X and y. I will add a check to see if the file exists and provide a more informative error message, then finish the task as the primary goal of replacing `pandas_ta` has been achieved. The user also wanted to rerun the code after the fix, so I will re-execute the main block.



In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)
    except FileNotFoundError as e:
        print(e)
        # Exit the script if the file is not found
        exit()

    # --- 2. Data Scaling ---
    # Create scaler
    scaler = MinMaxScaler()
    # Fit the scaler only on the training data, then transform all data
    # For simplicity, we transform the entire X here first, but in a strict process,
    # it should be split first then transformed.
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. Create Sequence Data ---
    TIME_STEPS = 10  # Use data from the past 10 days as input
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. Split Training and Test Sets (Time Series Method) ---
    # Ensure the training and test sets are not shuffled, maintaining time order
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"Total samples: {len(X_seq)}")
    print(f"Training set samples: {len(X_train)}")
    print(f"Test set samples: {len(X_test)}")

    # --- 5. Train Model ---
    print("\nTraining model...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)

    # Only attempt to save and evaluate if the model was built successfully
    if model is not None:
        model.save('stock_predictor_v2.h5')
        print("\nModel saved as stock_predictor_v2.h5")

        # --- 6. Evaluate Model on Test Set ---
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

        # --- 7. Make Prediction on New Data ---
        # Assuming we want to predict after the latest data point
        # We need to get the last `TIME_STEPS` days of data from the dataset
        # Ensure there are enough samples for the last sequence
        if len(X_scaled) >= TIME_STEPS:
            last_sequence = X_scaled.iloc[-TIME_STEPS:].values

            # Reshape it to the input format required by the model (1, time_steps, n_features)
            last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

            # Make prediction
            prediction = model.predict(last_sequence_reshaped)
            prediction_proba = prediction[0][0]

            print("\n--- Latest Data Prediction ---")
            print(f"Based on the last {TIME_STEPS} days of data...")
            print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
            if prediction_proba > 0.5:
                print("Prediction Result: Bullish")
            else:
                print("Prediction Result: Bearish")
        else:
            print(f"\n--- Latest Data Prediction ---")
            print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")


错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。


NameError: name 'X' is not defined

## Verify column names

### Subtask:
Verify that the manually calculated technical indicator columns in the DataFrame have the expected names (`MA5`, `MA10`, `RSI`, `MACD`) before the feature and target columns are defined.


**Reasoning**:
To verify the column names after manual technical indicator calculations, I need to add a print statement within the `load_and_prepare_data` function after the calculations and before dropping NaN values. This will allow me to inspect the DataFrame structure and column names at that point.



In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # --- Print DataFrame columns and first few rows for verification ---
    print("DataFrame columns after technical indicator calculation:")
    print(df.columns)
    print("\nFirst 5 rows of DataFrame after technical indicator calculation:")
    display(df.head())

    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)
    except FileNotFoundError as e:
        print(e)
        # Exit the script if the file is not found
        exit()

    # --- 2. Data Scaling ---
    # Create scaler
    scaler = MinMaxScaler()
    # Fit the scaler only on the training data, then transform all data
    # For simplicity, we transform the entire X here first, but in a strict process,
    # it should be split first then transformed.
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. Create Sequence Data ---
    TIME_STEPS = 10  # Use data from the past 10 days as input
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. Split Training and Test Sets (Time Series Method) ---
    # Ensure the training and test sets are not shuffled, maintaining time order
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"Total samples: {len(X_seq)}")
    print(f"Training set samples: {len(X_train)}")
    print(f"Test set samples: {len(X_test)}")

    # --- 5. Train Model ---
    print("\nTraining model...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)

    # Only attempt to save and evaluate if the model was built successfully
    if model is not None:
        model.save('stock_predictor_v2.h5')
        print("\nModel saved as stock_predictor_v2.h5")

        # --- 6. Evaluate Model on Test Set ---
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

        # --- 7. Make Prediction on New Data ---
        # Assuming we want to predict after the latest data point
        # We need to get the last `TIME_STEPS` days of data from the dataset
        # Ensure there are enough samples for the last sequence
        if len(X_scaled) >= TIME_STEPS:
            last_sequence = X_scaled.iloc[-TIME_STEPS:].values

            # Reshape it to the input format required by the model (1, time_steps, n_features)
            last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

            # Make prediction
            prediction = model.predict(last_sequence_reshaped)
            prediction_proba = prediction[0][0]

            print("\n--- Latest Data Prediction ---")
            print(f"Based on the last {TIME_STEPS} days of data...")
            print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
            if prediction_proba > 0.5:
                print("Prediction Result: 看涨")
            else:
                print("Prediction Result: 看跌")
        else:
            print(f"\n--- Latest Data Prediction ---")
            print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。


NameError: name 'X' is not defined

**Reasoning**:
The error indicates that the `AAPL_daily.csv` file is still missing, preventing the script from defining `X` and `y`. To complete the subtask of verifying the column names, I need a sample DataFrame that simulates the output of the `load_and_prepare_data` function up to the point of calculating the technical indicators. This will allow me to inspect the column names without relying on the external file. I will then manually check if the expected columns exist.



In [1]:
# Create a sample DataFrame to simulate the data after technical indicator calculation
data = {
    '日期': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
                         '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10',
                         '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14', '2023-01-15',
                         '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20']),
    '收盘价': [150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
}
sample_df = pd.DataFrame(data)
sample_df = sample_df.set_index('日期')

# Manually calculate technical indicators on the sample DataFrame
sample_df['MA5'] = sample_df['收盘价'].rolling(window=5).mean()
sample_df['MA10'] = sample_df['收盘价'].rolling(window=10).mean()

delta = sample_df['收盘价'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
avg_loss = loss.ewm(com=14-1, min_periods=14).mean()
rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
sample_df['RSI'] = 100 - (100 / (1 + rs))

exp1 = sample_df['收盘价'].ewm(span=12, adjust=False).mean()
exp2 = sample_df['收盘价'].ewm(span=26, adjust=False).mean()
sample_df['MACD'] = exp1 - exp2
sample_df['MACD_signal'] = sample_df['MACD'].ewm(span=9, adjust=False).mean()
sample_df['MACD_hist'] = sample_df['MACD'] - sample_df['MACD_signal']

# Print DataFrame columns for verification
print("Sample DataFrame columns after technical indicator calculation:")
print(sample_df.columns)

# Check if expected columns exist
expected_cols = ['MA5', 'MA10', 'RSI', 'MACD']
missing_cols = [col for col in expected_cols if col not in sample_df.columns]

if not missing_cols:
    print("\nAll expected technical indicator columns exist in the DataFrame.")
else:
    print(f"\nMissing technical indicator columns: {missing_cols}")

# Verify the feature_cols list in the original code
feature_cols_in_original_code = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
print(f"\nFeature columns defined in the original code: {feature_cols_in_original_code}")

# Although we cannot fully verify the '价格/MA5' and 'MA5/MA10' calculations without the full data,
# the presence of 'RSI' and 'MACD' can be confirmed.

NameError: name 'pd' is not defined

**Reasoning**:
The previous command failed because `pandas` was not imported in that specific code block. To fix this and successfully create the sample DataFrame and perform the checks, I need to include the necessary imports (`pandas` and `numpy`) in the code block.



In [2]:
import pandas as pd
import numpy as np

# Create a sample DataFrame to simulate the data after technical indicator calculation
data = {
    '日期': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
                         '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10',
                         '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14', '2023-01-15',
                         '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20']),
    '收盘价': [150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
}
sample_df = pd.DataFrame(data)
sample_df = sample_df.set_index('日期')

# Manually calculate technical indicators on the sample DataFrame
sample_df['MA5'] = sample_df['收盘价'].rolling(window=5).mean()
sample_df['MA10'] = sample_df['收盘价'].rolling(window=10).mean()

delta = sample_df['收盘价'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
avg_loss = loss.ewm(com=14-1, min_periods=14).mean()
rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
sample_df['RSI'] = 100 - (100 / (1 + rs))

exp1 = sample_df['收盘价'].ewm(span=12, adjust=False).mean()
exp2 = sample_df['收盘价'].ewm(span=26, adjust=False).mean()
sample_df['MACD'] = exp1 - exp2
sample_df['MACD_signal'] = sample_df['MACD'].ewm(span=9, adjust=False).mean()
sample_df['MACD_hist'] = sample_df['MACD'] - sample_df['MACD_signal']

# Print DataFrame columns for verification
print("Sample DataFrame columns after technical indicator calculation:")
print(sample_df.columns)

# Check if expected columns exist
expected_cols = ['MA5', 'MA10', 'RSI', 'MACD']
missing_cols = [col for col in expected_cols if col not in sample_df.columns]

if not missing_cols:
    print("\nAll expected technical indicator columns exist in the DataFrame.")
else:
    print(f"\nMissing technical indicator columns: {missing_cols}")

# Verify the feature_cols list in the original code
feature_cols_in_original_code = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
print(f"\nFeature columns defined in the original code: {feature_cols_in_original_code}")

# Although we cannot fully verify the '价格/MA5' and 'MA5/MA10' calculations without the full data,
# the presence of 'RSI' and 'MACD' can be confirmed.

Sample DataFrame columns after technical indicator calculation:
Index(['收盘价', 'MA5', 'MA10', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist'], dtype='object')

All expected technical indicator columns exist in the DataFrame.

Feature columns defined in the original code: ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']


## Clean data

### Subtask:
Keep the `dropna()` step to remove rows with NaN values introduced by the calculations.


## Run the modified code

### Subtask:
Execute the updated `IFLFDSq2QcVk` cell to check if the code runs without errors and produces the expected output.


**Reasoning**:
Execute the updated code cell to check if the manual technical indicator calculations and subsequent steps run without errors.



In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    # Use small epsilon to avoid inf values
    rs = avg_gain / (avg_loss + 1e-10)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # Drop all rows containing NaN values introduced by indicator and target variable calculations
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # Get input shape
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM layer, input_shape=(time_steps, number of features)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # Add Dropout to prevent overfitting
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. Load and Prepare Data ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)
    except FileNotFoundError as e:
        print(e)
        # Exit the script if the file is not found
        exit()

    # --- 2. Data Scaling ---
    # Create scaler
    scaler = MinMaxScaler()
    # Fit the scaler only on the training data, then transform all data
    # For simplicity, we transform the entire X here first, but in a strict process,
    # it should be split first then transformed.
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. Create Sequence Data ---
    TIME_STEPS = 10  # Use data from the past 10 days as input
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. Split Training and Test Sets (Time Series Method) ---
    # Ensure the training and test sets are not shuffled, maintaining time order
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"Total samples: {len(X_seq)}")
    print(f"Training set samples: {len(X_train)}")
    print(f"Test set samples: {len(X_test)}")

    # --- 5. Train Model ---
    print("\nTraining model...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)

    # Only attempt to save and evaluate if the model was built successfully
    if model is not None:
        model.save('stock_predictor_v2.h5')
        print("\nModel saved as stock_predictor_v2.h5")

        # --- 6. Evaluate Model on Test Set ---
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

        # --- 7. Make Prediction on New Data ---
        # Assuming we want to predict after the latest data point
        # We need to get the last `TIME_STEPS` days of data from the dataset
        # Ensure there are enough samples for the last sequence
        if len(X_scaled) >= TIME_STEPS:
            last_sequence = X_scaled.iloc[-TIME_STEPS:].values

            # Reshape it to the input format required by the model (1, time_steps, n_features)
            last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

            # Make prediction
            prediction = model.predict(last_sequence_reshaped)
            prediction_proba = prediction[0][0]

            print("\n--- Latest Data Prediction ---")
            print(f"Based on the last {TIME_STEPS} days of data...")
            print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
            if prediction_proba > 0.5:
                print("Prediction Result: Bullish")
            else:
                print("Prediction Result: Bearish")
        else:
            print(f"\n--- Latest Data Prediction ---")
            print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")


错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。


NameError: name 'X' is not defined

**Reasoning**:
The code failed because the `AAPL_daily.csv` file was not found, causing the script to exit before `X` was defined. To address this and allow the rest of the code to be tested with the manual calculations, I will add a check for the `FileNotFoundError` after the `load_and_prepare_data` call and only proceed with the remaining steps if the data loading is successful. This will prevent the `NameError` and allow the script to complete if the file is present in the environment where it is executed.



In [1]:
# Only proceed with the rest of the script if data loading was successful
if 'X' in locals() and 'y' in locals() and 'df_full' in locals():
    # --- 2. Data Scaling ---
    # Create scaler
    scaler = MinMaxScaler()
    # Fit the scaler only on the training data, then transform all data
    # For simplicity, we transform the entire X here first, but in a strict process,
    # it should be split first then transformed.
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # --- 3. Create Sequence Data ---
    TIME_STEPS = 10  # Use data from the past 10 days as input
    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

    # --- 4. Split Training and Test Sets (Time Series Method) ---
    # Ensure the training and test sets are not shuffled, maintaining time order
    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"Total samples: {len(X_seq)}")
    print(f"Training set samples: {len(X_train)}")
    print(f"Test set samples: {len(X_test)}")

    # --- 5. Train Model ---
    print("\nTraining model...")
    model = build_and_train_model(X_train, y_train, X_test, y_test)

    # Only attempt to save and evaluate if the model was built successfully
    if model is not None:
        model.save('stock_predictor_v2.h5')
        print("\nModel saved as stock_predictor_v2.h5")

        # --- 6. Evaluate Model on Test Set ---
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

        # --- 7. Make Prediction on New Data ---
        # Assuming we want to predict after the latest data point
        # We need to get the last `TIME_STEPS` days of data from the dataset
        # Ensure there are enough samples for the last sequence
        if len(X_scaled) >= TIME_STEPS:
            last_sequence = X_scaled.iloc[-TIME_STEPS:].values

            # Reshape it to the input format required by the model (1, time_steps, n_features)
            last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

            # Make prediction
            prediction = model.predict(last_sequence_reshaped)
            prediction_proba = prediction[0][0]

            print("\n--- Latest Data Prediction ---")
            print(f"Based on the last {TIME_STEPS} days of data...")
            print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
            if prediction_proba > 0.5:
                print("Prediction Result: Bullish")
            else:
                print("Prediction Result: Bearish")
        else:
            print(f"\n--- Latest Data Prediction ---")
            print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")
else:
    print("Data loading failed. Skipping model training and prediction.")


Data loading failed. Skipping model training and prediction.


## Run the modified code retry 1

### Subtask:
Retry running the modified `IFLFDSq2QcVk` cell after ensuring the `AAPL_daily.csv` file is available. The code now includes manual calculations for technical indicators and handles the `FileNotFoundError`.


**Reasoning**:
Retry running the modified `IFLFDSq2QcVk` cell after ensuring the `AAPL_daily.csv` file is available. The previous attempt failed because the file was not found. This run should execute the full script with the manual technical indicator calculations and the LSTM model training if the file is now present.



In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)

        # Only proceed with the rest of the script if data loading was successful
        # --- 2. Data Scaling ---
        # Create scaler
        scaler = MinMaxScaler()
        # Fit the scaler only on the training data, then transform all data
        # For simplicity, we transform the entire X here first, but in a strict process,
        # it should be split first then transformed.
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

        # --- 3. Create Sequence Data ---
        TIME_STEPS = 10  # Use data from the past 10 days as input
        X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

        # --- 4. Split Training and Test Sets (Time Series Method) ---
        # Ensure the training and test sets are not shuffled, maintaining time order
        split_index = int(len(X_seq) * 0.8)
        X_train, X_test = X_seq[:split_index], X_seq[split_index:]
        y_train, y_test = y_seq[:split_index], y_seq[split_index:]

        print(f"Total samples: {len(X_seq)}")
        print(f"Training set samples: {len(X_train)}")
        print(f"Test set samples: {len(X_test)}")

        # --- 5. Train Model ---
        print("\nTraining model...")
        model = build_and_train_model(X_train, y_train, X_test, y_test)

        # Only attempt to save and evaluate if the model was built successfully
        if model is not None:
            model.save('stock_predictor_v2.h5')
            print("\nModel saved as stock_predictor_v2.h5")

            # --- 6. Evaluate Model on Test Set ---
            loss, accuracy = model.evaluate(X_test, y_test)
            print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

            # --- 7. Make Prediction on New Data ---
            # Assuming we want to predict after the latest data point
            # We need to get the last `TIME_STEPS` days of data from the dataset
            # Ensure there are enough samples for the last sequence
            if len(X_scaled) >= TIME_STEPS:
                last_sequence = X_scaled.iloc[-TIME_STEPS:].values

                # Reshape it to the input format required by the model (1, time_steps, n_features)
                last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

                # Make prediction
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
                if prediction_proba > 0.5:
                    print("Prediction Result: Bullish")
                else:
                    print("Prediction Result: Bearish")
            else:
                print(f"\n--- Latest Data Prediction ---")
                print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


错误：找不到 'AAPL_daily.csv'。请确保文件在正确的路径下。


## Summary:

### Data Analysis Key Findings

*   The user successfully replaced the `pandas_ta` technical indicator calculations with manual implementations using pandas methods for MA5, MA10, RSI, and MACD.
*   The code was modified to include error handling for a missing input file (`AAPL_daily.csv`), which was the primary reason the script failed to run in initial attempts.
*   The script was not fully executed to test the entire workflow (including data scaling, sequence creation, model training, and prediction) due to the persistent absence of the required `AAPL_daily.csv` file.

### Insights or Next Steps

*   To fully verify the correctness of the manual technical indicator calculations and the subsequent model training process, the `AAPL_daily.csv` file needs to be provided in the execution environment.
*   Consider adding more robust error handling or input validation for the CSV file, such as checking for the presence of required columns ('日期', '收盘价') after loading.


In [2]:
# -*- coding: utf-8 -*-
"""
依赖检查和安装脚本
在运行股票预测程序前，请先运行此脚本检查环境
"""

import sys
import subprocess
import importlib

def check_python_version():
    """检查Python版本"""
    print("=== Python版本检查 ===")
    version = sys.version_info
    print(f"当前Python版本: {version.major}.{version.minor}.{version.micro}")

    if version.major < 3 or (version.major == 3 and version.minor < 7):
        print("❌ 需要Python 3.7或更高版本")
        return False
    else:
        print("✓ Python版本满足要求")
        return True

def install_package(package_name):
    """安装包"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        return True
    except subprocess.CalledProcessError:
        return False

def check_and_install_dependencies():
    """检查并安装依赖"""
    print("\n=== 依赖库检查 ===")

    # 必需的依赖库
    required_packages = [
        ("pandas", "pandas"),
        ("numpy", "numpy"),
        ("sklearn", "scikit-learn"),
        ("tensorflow", "tensorflow"),
    ]

    missing_packages = []

    for import_name, package_name in required_packages:
        try:
            importlib.import_module(import_name)
            print(f"✓ {package_name} 已安装")
        except ImportError:
            print(f"❌ {package_name} 未安装")
            missing_packages.append(package_name)

    # 安装缺失的包
    if missing_packages:
        print(f"\n正在安装缺失的依赖: {', '.join(missing_packages)}")
        for package in missing_packages:
            print(f"正在安装 {package}...")
            if install_package(package):
                print(f"✓ {package} 安装成功")
            else:
                print(f"❌ {package} 安装失败")
                return False

    return True

def test_tensorflow():
    """测试TensorFlow"""
    print("\n=== TensorFlow测试 ===")
    try:
        import tensorflow as tf
        print(f"TensorFlow版本: {tf.__version__}")

        # 测试基本功能
        a = tf.constant([1, 2, 3])
        b = tf.constant([4, 5, 6])
        c = tf.add(a, b)
        print("✓ TensorFlow基本功能正常")

        # 检查GPU
        if tf.config.list_physical_devices('GPU'):
            print("✓ GPU可用")
        else:
            print("ℹ 将使用CPU进行训练")

        return True
    except Exception as e:
        print(f"❌ TensorFlow测试失败: {e}")
        return False

def create_requirements_file():
    """创建requirements.txt文件"""
    requirements = """pandas>=1.3.0
numpy>=1.21.0
scikit-learn>=1.0.0
tensorflow>=2.8.0
matplotlib>=3.3.0
"""

    with open("requirements.txt", "w") as f:
        f.write(requirements)

    print("\n✓ requirements.txt 文件已创建")
    print("你也可以使用以下命令安装依赖:")
    print("pip install -r requirements.txt")

def main():
    """主函数"""
    print("股票预测程序 - 环境检查工具")
    print("=" * 50)

    # 检查Python版本
    if not check_python_version():
        return

    # 检查并安装依赖
    if not check_and_install_dependencies():
        print("\n❌ 依赖安装失败，请手动安装")
        create_requirements_file()
        return

    # 测试TensorFlow
    if not test_tensorflow():
        print("\n❌ TensorFlow测试失败")
        return

    print("\n" + "=" * 50)
    print("✓ 环境检查完成！")
    print("所有依赖都已正确安装，你现在可以运行股票预测程序了。")
    print("\n建议执行步骤：")
    print("1. 运行改进版股票预测程序")
    print("2. 程序会自动生成示例数据（如果没有现有数据）")
    print("3. 查看训练结果和预测")

if __name__ == "__main__":
    main()

股票预测程序 - 环境检查工具
=== Python版本检查 ===
当前Python版本: 3.11.13
✓ Python版本满足要求

=== 依赖库检查 ===
✓ pandas 已安装
✓ numpy 已安装
✓ scikit-learn 已安装
✓ tensorflow 已安装

=== TensorFlow测试 ===
TensorFlow版本: 2.18.0
✓ TensorFlow基本功能正常
ℹ 将使用CPU进行训练

✓ 环境检查完成！
所有依赖都已正确安装，你现在可以运行股票预测程序了。

建议执行步骤：
1. 运行改进版股票预测程序
2. 程序会自动生成示例数据（如果没有现有数据）
3. 查看训练结果和预测


# Task
Fetch historical daily stock data for Apple (AAPL) using either `akshare` or `tushare`, save it to a CSV file named "AAPL_daily.csv" with columns '日期' and '收盘价', and then run the code in cell with id IFLFDSq2QcVk.

## Install data fetching library

### Subtask:
Install either the `akshare` or `tushare` library using pip.


**Reasoning**:
Install the `akshare` library using pip as it is generally easier for basic data fetching without an API key.



In [4]:
!pip install akshare

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 90.8 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5668 sha256=91816a8c23eca40d2487874e37db3f3394bab1cfbec38cfc49ca36985fcf5e92
  Stored in directory: /root/.cache/pip/wheels/7d/b8/16/48bbd715040679311fa68cb564ad24a97d9a67fea5d4c848c8
Successfully built jsonpath


## Fetch data

### Subtask:
Fetch historical daily data for a specific stock (e.g., Apple, usually represented by a code like 'AAPL') using the chosen library.


**Reasoning**:
Import the akshare library and use the appropriate function to fetch historical daily stock data for 'AAPL'. Store the fetched data in a pandas DataFrame.



In [5]:
import akshare as ak

# Define start and end dates
start_date = "20100101"
end_date = "20241231"

# Fetch historical daily data for Apple (AAPL)
# Using 'stock_us_daily_qfq' for US stocks daily data with front-adjusted values
stock_data = ak.stock_us_daily_qfq(symbol="AAPL", adjust="qfq")

# Display the first few rows and info to verify the data
print("Fetched Stock Data:")
display(stock_data.head())
stock_data.info()

AttributeError: module 'akshare' has no attribute 'stock_us_daily_qfq'

**Reasoning**:
The previous command failed because the function `stock_us_daily_qfq` was not found in the imported `akshare` module. I need to find the correct function within `akshare` to fetch US stock daily data with front-adjusted values. I will explore the `akshare` documentation or available functions to identify the correct one and then use it to fetch the data.



In [6]:
# Attempt to find the correct function for US stock daily data with front-adjusted values in akshare

# A common pattern for US stocks in akshare is 'stock_us_hist'.
# Let's try using 'stock_us_hist' and check its parameters and output.

# Define start and end dates
start_date = "20100101"
end_date = "20241231"

try:
    # Try using 'stock_us_hist' with adjust='qfq'
    stock_data = ak.stock_us_hist(symbol="AAPL", period="daily", adjust="qfq")

    # Display the first few rows and info to verify the data
    print("Fetched Stock Data using stock_us_hist:")
    display(stock_data.head())
    stock_data.info()

except AttributeError:
    print("The function 'stock_us_hist' with adjust='qfq' was not found. Exploring other options.")
    # If the above fails, we might need to explore other akshare functions
    # or check the akshare documentation for the correct function name and usage.
    # For the scope of this task, if this attempt fails, I will report failure.


TypeError: 'NoneType' object is not subscriptable

**Reasoning**:
The traceback indicates that the `ak.stock_us_hist` function is returning `None` for `"data"` or `"klines"`, suggesting an issue with the data source or the parameters. This might be due to incorrect date format, symbol, or service availability. I will try a different `akshare` function that is known to work for US stocks and has clearer parameter requirements. `stock_us_zhishu_daily` seems like a possible alternative for daily US stock index data, although the original task was for individual stock data. If this also fails, it might indicate a broader issue with `akshare`'s US stock data availability or require consulting the `akshare` documentation more thoroughly. I will keep the same start and end dates but ensure they are in the expected 'YYYY-MM-DD' format if the function requires it, although the previous functions used 'YYYYMMDD'.



In [7]:
# Try using a different akshare function for US stock data
# Let's attempt to use 'stock_us_hist' again but with date format 'YYYY-MM-DD'
# as the traceback comes from within the akshare library code, the issue might be internal.

# Define start and end dates in 'YYYY-MM-DD' format just in case
start_date_ymd = "2010-01-01"
end_date_ymd = "2024-12-31"

try:
    # Try using 'stock_us_hist' with the 'YYYY-MM-DD' date format
    stock_data = ak.stock_us_hist(symbol="AAPL", period="daily", start_date=start_date_ymd, end_date=end_date_ymd, adjust="qfq")

    # Display the first few rows and info to verify the data
    print("Fetched Stock Data using stock_us_hist with YYYY-MM-DD dates:")
    display(stock_data.head())
    stock_data.info()

except AttributeError:
    print("The function 'stock_us_hist' with adjust='qfq' was not found after retrying date format.")
except TypeError:
     print("TypeError encountered with stock_us_hist. The function might not be working as expected or parameters are incorrect.")
except Exception as e:
    print(f"An unexpected error occurred while fetching data with stock_us_hist: {e}")


TypeError encountered with stock_us_hist. The function might not be working as expected or parameters are incorrect.


**Reasoning**:
The `ak.stock_us_hist` function appears to be consistently failing, possibly due to issues with the data source or internal `akshare` implementation for US stocks at this time. Since the primary goal is to fetch US stock data, and `akshare` is not working reliably, I will try using `yfinance`, which is a widely used and generally robust library for fetching financial data from Yahoo Finance. This involves installing the library, importing it, and using its methods to download the historical data for AAPL.



In [8]:
# Try using the yfinance library as an alternative
!pip install yfinance

import yfinance as yf

# Define start and end dates
start_date = "2010-01-01"
end_date = "2024-12-31"

try:
    # Fetch historical daily data for Apple (AAPL) using yfinance
    stock_data = yf.download("AAPL", start=start_date, end=end_date)

    # Display the first few rows and info to verify the data
    print("Fetched Stock Data using yfinance:")
    display(stock_data.head())
    stock_data.info()

except Exception as e:
    print(f"An error occurred while fetching data with yfinance: {e}")


/tmp/ipython-input-8-436107281.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download("AAPL", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Fetched Stock Data using yfinance:


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.431896,6.446623,6.382908,6.414465,493729600
2010-01-05,6.443017,6.479383,6.409056,6.449630,601904800
2010-01-06,6.340532,6.468563,6.333920,6.443017,552160000
2010-01-07,6.328811,6.371489,6.282828,6.363975,477131200
2010-01-08,6.370886,6.371487,6.283128,6.320395,447610800


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3773 entries, 2010-01-04 to 2024-12-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   3773 non-null   float64
 1   (High, AAPL)    3773 non-null   float64
 2   (Low, AAPL)     3773 non-null   float64
 3   (Open, AAPL)    3773 non-null   float64
 4   (Volume, AAPL)  3773 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 176.9 KB


## Save data to csv

### Subtask:
Save the fetched data to a CSV file named `AAPL_daily.csv` with the required columns ('日期' and '收盘价').


**Reasoning**:
Select the 'Close' column, rename it to '收盘价', reset the index to convert 'Date' to a column, rename 'Date' to '日期', and save the resulting DataFrame to 'AAPL_daily.csv' without the index.



In [9]:
# Select the 'Close' column
close_price_df = stock_data['Close']

# Rename the 'Close' column to '收盘价'
close_price_df = close_price_df.rename('收盘价')

# Convert the Series to a DataFrame
close_price_df = close_price_df.to_frame()

# Reset the index to turn the 'Date' index into a column
close_price_df = close_price_df.reset_index()

# Rename the 'Date' column to '日期'
close_price_df = close_price_df.rename(columns={'Date': '日期'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

TypeError: 'str' object is not callable

**Reasoning**:
The error "TypeError: 'str' object is not callable" occurs because I'm trying to use the string '收盘价' as a callable function within the `rename` method when applied to a pandas Series. To rename a Series, I should use the `.name` attribute. After renaming the Series, I should convert it to a DataFrame and then rename the index.



In [10]:
# Select the 'Close' column
close_price_df = stock_data['Close']

# Rename the 'Close' Series to '收盘价' using the .name attribute
close_price_df.name = '收盘价'

# Convert the Series to a DataFrame
close_price_df = close_price_df.to_frame()

# Reset the index to turn the 'Date' index into a column
close_price_df = close_price_df.reset_index()

# Rename the 'index' column (which was the original 'Date' index) to '日期'
close_price_df = close_price_df.rename(columns={'index': '日期'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

AttributeError: 'DataFrame' object has no attribute 'to_frame'

**Reasoning**:
The error "AttributeError: 'DataFrame' object has no attribute 'to_frame'" indicates that `close_price_df` is already a DataFrame, not a Series, when I try to call `.to_frame()`. This is because selecting a single column from a DataFrame using single brackets `df['col']` returns a Series, but if the original `stock_data` was a DataFrame with a MultiIndex (as suggested by the variable explorer and previous outputs showing `Ticker AAPL` and `Date`), selecting a column might sometimes result in a DataFrame depending on how it was created. To fix this, I will remove the `.to_frame()` call as `close_price_df` should already be a DataFrame after selecting the 'Close' column from the MultiIndex DataFrame. Then I will rename the columns correctly.



In [11]:
# Select the 'Close' column. This should return a DataFrame because the original is MultiIndex.
close_price_df = stock_data['Close']

# Reset the index to turn the 'Date' index into a column
close_price_df = close_price_df.reset_index()

# Rename the columns: the first column is the 'Date' index after reset, and the second is 'Close'.
# The original index levels were 'Date' and 'Ticker'. After reset_index(), these become columns.
# The 'Close' column retains its name from the selection.
# The columns will be 'Date', 'Ticker', and 'Close'. We only need 'Date' and 'Close'.
# Let's rename 'Date' to '日期' and 'Close' to '收盘价'.
close_price_df = close_price_df.rename(columns={'Date': '日期', 'Close': '收盘价'})

# Keep only the required columns '日期' and '收盘价'
close_price_df = close_price_df[['日期', '收盘价']]

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

KeyError: "['收盘价'] not in index"

**Reasoning**:
The error "KeyError: "['收盘价'] not in index"" indicates that after renaming, the column is not correctly named '收盘价'. Looking back at the previous step, I renamed the column using `.rename(columns={'Close': '收盘价'})`. This should have correctly renamed the column. The issue might be in the line `close_price_df = close_price_df[['日期', '收盘价']]`, where I am trying to select the column using the new name immediately after renaming, but perhaps the rename hasn't fully taken effect in the way I expect, or there's an issue with how the columns are structured after `reset_index` on the MultiIndex DataFrame.

Let's re-examine the structure after `reset_index`. When you reset the index of a MultiIndex DataFrame, the index levels become regular columns. In the case of `stock_data`, the index levels are 'Date' and 'Ticker'. The columns are 'Close', 'High', 'Low', 'Open', 'Volume'. So, after `reset_index()`, the columns should be 'Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'.

I only need the 'Date' and 'Close' columns. I should select these two columns first and then rename them.



In [12]:
# Select the 'Date' index and the 'Close' column from the original MultiIndex DataFrame
# This will result in a DataFrame with 'Date' as a regular column and 'Close' as another column.
close_price_df = stock_data.reset_index()[['Date', 'Close']]

# Rename the columns to '日期' and '收盘价'
close_price_df = close_price_df.rename(columns={'Date': '日期', 'Close': '收盘价'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.


## Verify the csv file

### Subtask:
Check if the `AAPL_daily.csv` file is created correctly with the necessary columns and data format.


**Reasoning**:
Read the AAPL_daily.csv file into a pandas DataFrame and perform the necessary checks as outlined in the instructions.



In [13]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('AAPL_daily.csv')

# Print the first 5 rows of the DataFrame
print("First 5 rows of AAPL_daily.csv:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of AAPL_daily.csv:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in AAPL_daily.csv:")
df.info()

First 5 rows of AAPL_daily.csv:


,日期,收盘价
0,NaN,AAPL
1,2010-01-04,6.431896209716797
2,2010-01-05,6.443017482757568
3,2010-01-06,6.340531826019287
4,2010-01-07,6.328811168670654



Column names of AAPL_daily.csv:
Index(['日期', '收盘价'], dtype='object')

Data types of columns in AAPL_daily.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3774 entries, 0 to 3773
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日期      3773 non-null   object
 1   收盘价     3774 non-null   object
dtypes: object(2)
memory usage: 59.1+ KB


**Reasoning**:
The previous step successfully read the CSV and displayed its structure. The output shows that the column names are correct ('日期' and '收盘价'). However, the first row contains header-like values ('NaN' and 'AAPL') which are not actual data, and the data types are 'object' instead of appropriate numeric and datetime types. This indicates an issue with how the CSV was saved or read, potentially caused by including the header row as data. The `df_full` dataframe from the previous step in the history shows the same issue. The instruction is to check if the file is created correctly. It is not created correctly due to the extra row. I will modify the previous code to skip the first row when reading the CSV, which should resolve the issue with the extra header row and incorrect data types. Then I will re-run the checks.



In [14]:
# Read the CSV file into a pandas DataFrame, skipping the first row
df = pd.read_csv('AAPL_daily.csv', skiprows=1)

# Print the first 5 rows of the DataFrame
print("First 5 rows of AAPL_daily.csv after skipping the first row:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of AAPL_daily.csv after skipping the first row:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in AAPL_daily.csv after skipping the first row:")
df.info()

First 5 rows of AAPL_daily.csv after skipping the first row:


,Unnamed: 0,AAPL
0,2010-01-04,6.431896
1,2010-01-05,6.443017
2,2010-01-06,6.340532
3,2010-01-07,6.328811
4,2010-01-08,6.370886



Column names of AAPL_daily.csv after skipping the first row:
Index(['Unnamed: 0', 'AAPL'], dtype='object')

Data types of columns in AAPL_daily.csv after skipping the first row:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3773 non-null   object 
 1   AAPL        3773 non-null   float64
dtypes: float64(1), object(1)
memory usage: 59.1+ KB


**Reasoning**:
Skipping the first row when reading the CSV helped remove the extra header row, but the column names are still incorrect ('Unnamed: 0' and 'AAPL') because the original header row was also skipped. To correctly read the data with the proper column names ('日期' and '收盘价'), I need to read the first row to get the headers and then read the rest of the data. However, based on the previous history, it seems the `to_csv` step in the previous subtask might have saved the DataFrame incorrectly, potentially including the index name or an extra row. I will re-examine the output of the previous `to_csv` step and regenerate the CSV file with the correct headers and no extra rows by explicitly setting `header=True` and ensuring `index=False`. Then I will re-read the CSV and perform the verification steps.



In [15]:
# Re-create the DataFrame with correct column names and save it to CSV

# The stock_data DataFrame from the previous subtask had the correct data but a MultiIndex.
# We need to select the correct columns and flatten the index before saving.
# Based on the history, 'stock_data' is a MultiIndex DataFrame.
# Assuming 'stock_data' is still available in the environment from the previous successful fetch.

# Select the 'Close' price and reset the index to make 'Date' a column
close_price_df = stock_data['Close'].reset_index()

# Rename the columns to '日期' and '收盘价'
close_price_df = close_price_df.rename(columns={'Date': '日期', 'Close': '收盘价'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
# Explicitly set header=True to ensure the column names are written as the first row
close_price_df.to_csv('AAPL_daily.csv', index=False, header=True)

print("Data re-saved to AAPL_daily.csv with corrected format.")

# Now, read the corrected CSV file into a pandas DataFrame
df = pd.read_csv('AAPL_daily.csv')

# Print the first 5 rows of the DataFrame
print("\nFirst 5 rows of AAPL_daily.csv after correction:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of AAPL_daily.csv after correction:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in AAPL_daily.csv after correction:")
df.info()

Data re-saved to AAPL_daily.csv with corrected format.

First 5 rows of AAPL_daily.csv after correction:


,日期,AAPL
0,2010-01-04,6.431896
1,2010-01-05,6.443017
2,2010-01-06,6.340532
3,2010-01-07,6.328811
4,2010-01-08,6.370886



Column names of AAPL_daily.csv after correction:
Index(['日期', 'AAPL'], dtype='object')

Data types of columns in AAPL_daily.csv after correction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      3773 non-null   object 
 1   AAPL    3773 non-null   float64
dtypes: float64(1), object(1)
memory usage: 59.1+ KB


**Reasoning**:
The CSV file still has 'AAPL' as the second column name instead of '收盘价', although the data format and headers are otherwise correct now. This is likely because in the previous step, I re-saved the `close_price_df` which had already been renamed but the original `stock_data` had 'Close' under a MultiIndex level 'AAPL'. When resetting the index and selecting 'Close', the column name became 'AAPL' instead of 'Close'. I need to explicitly rename the 'AAPL' column to '收盘价' after resetting the index and before saving to CSV. Then I will perform the verification steps again.



In [16]:
# Re-create the DataFrame with correct column names and save it to CSV again

# Select the 'Close' price and reset the index to make 'Date' a column
close_price_df = stock_data['Close'].reset_index()

# Rename the 'Date' column to '日期' and the 'AAPL' column to '收盘价'
# Based on the previous output, the column name after reset_index() is 'AAPL'
close_price_df = close_price_df.rename(columns={'Date': '日期', 'AAPL': '收盘价'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
# Explicitly set header=True to ensure the column names are written as the first row
close_price_df.to_csv('AAPL_daily.csv', index=False, header=True)

print("Data re-saved to AAPL_daily.csv with corrected column names.")

# Now, read the corrected CSV file into a pandas DataFrame
df = pd.read_csv('AAPL_daily.csv')

# Print the first 5 rows of the DataFrame
print("\nFirst 5 rows of AAPL_daily.csv after correction:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of AAPL_daily.csv after correction:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in AAPL_daily.csv after correction:")
df.info()

Data re-saved to AAPL_daily.csv with corrected column names.

First 5 rows of AAPL_daily.csv after correction:


,日期,收盘价
0,2010-01-04,6.431896
1,2010-01-05,6.443017
2,2010-01-06,6.340532
3,2010-01-07,6.328811
4,2010-01-08,6.370886



Column names of AAPL_daily.csv after correction:
Index(['日期', '收盘价'], dtype='object')

Data types of columns in AAPL_daily.csv after correction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      3773 non-null   object 
 1   收盘价     3773 non-null   float64
dtypes: float64(1), object(1)
memory usage: 59.1+ KB


## Run the main program

### Subtask:
Execute the original code cell (`IFLFDSq2QcVk`) which now expects the `AAPL_daily.csv` file to be present.


**Reasoning**:
Execute the updated code cell with the manual technical indicator calculations and file existence check. If the AAPL_daily.csv file is now present, the script should run to completion, including model training and prediction.



In [17]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    加载CSV数据，计算技术指标，并创建目标变量。
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- 计算技术指标 (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 10-day Simple Moving Average ('MA10')
    df['MA10'] = df['收盘价'].rolling(window=10).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']


    # --- 创建特征 ---
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA10'] = df['MA5'] / df['MA10']

    # --- 创建目标变量：未来5天后价格是否上涨 ---
    # shift(-5) 会将未来第5天的数据移到当前行
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- 清理数据 ---
    # 删除所有因为计算指标和目标变量而产生的含有NaN的行
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    feature_cols = ['价格/MA5', 'MA5/MA10', 'RSI', 'MACD']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. 创建时间序列样本 =====
def create_sequences(X, y, time_steps=10):
    """
    将数据转换为适用于LSTM的序列样本。
    :param X: 特征数据集
    :param y: 目标数据集
    :param time_steps: 每个样本包含的时间步长（回看天数）
    :return: 序列化的 X 和对应的 y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. 训练模型 =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    构建、编译和训练LSTM模型。
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # 获取输入形状
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM层，input_shape=(时间步长, 特征数量)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # 添加Dropout防止过拟合
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # 输出层，用于二元分类
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== 主程序 =====
if __name__ == "__main__":
    # --- 1. 加载和准备数据 ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)

        # Only proceed with the rest of the script if data loading was successful
        # --- 2. Data Scaling ---
        # Create scaler
        scaler = MinMaxScaler()
        # Fit the scaler only on the training data, then transform all data
        # For simplicity, we transform the entire X here first, but in a strict process,
        # it should be split first then transformed.
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

        # --- 3. Create Sequence Data ---
        TIME_STEPS = 10  # Use data from the past 10 days as input
        X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

        # --- 4. Split Training and Test Sets (Time Series Method) ---
        # Ensure the training and test sets are not shuffled, maintaining time order
        split_index = int(len(X_seq) * 0.8)
        X_train, X_test = X_seq[:split_index], X_seq[split_index:]
        y_train, y_test = y_seq[:split_index], y_seq[split_index:]

        print(f"Total samples: {len(X_seq)}")
        print(f"Training set samples: {len(X_train)}")
        print(f"Test set samples: {len(X_test)}")

        # --- 5. Train Model ---
        print("\nTraining model...")
        model = build_and_train_model(X_train, y_train, X_test, y_test)

        # Only attempt to save and evaluate if the model was built successfully
        if model is not None:
            model.save('stock_predictor_v2.h5')
            print("\nModel saved as stock_predictor_v2.h5")

            # --- 6. Evaluate Model on Test Set ---
            loss, accuracy = model.evaluate(X_test, y_test)
            print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

            # --- 7. Make Prediction on New Data ---
            # Assuming we want to predict after the latest data point
            # We need to get the last `TIME_STEPS` days of data from the dataset
            # Ensure there are enough samples for the last sequence
            if len(X_scaled) >= TIME_STEPS:
                last_sequence = X_scaled.iloc[-TIME_STEPS:].values

                # Reshape it to the input format required by the model (1, time_steps, n_features)
                last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

                # Make prediction
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
                if prediction_proba > 0.5:
                    print("Prediction Result: Bullish")
                else:
                    print("Prediction Result: Bearish")
            else:
                print(f"\n--- Latest Data Prediction ---")
                print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Total samples: 3745
Training set samples: 2996
Test set samples: 749

Training model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


模型结构概要：


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,301 (48.05 KB)

 Trainable params: 12,301 (48.05 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5738 - loss: 0.6834 - val_accuracy: 0.5461 - val_loss: 0.6916
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5938 - loss: 0.6763 - val_accuracy: 0.5461 - val_loss: 0.6887
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5866 - loss: 0.6788 - val_accuracy: 0.5461 - val_loss: 0.6915
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5794 - loss: 0.6811 - val_accuracy: 0.5461 - val_loss: 0.6957
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5899 - loss: 0.6776 - val_accuracy: 0.5461 - val_loss: 0.6907
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5971 - loss: 0.6738 - val_accuracy: 0.5461 - val_loss: 0.6898
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5904 - loss: 0.6769 - val_accuracy: 0.5461 - val_loss: 0.6908
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5874 - loss: 0.6784 - val_accuracy: 0.5461 - val


Model saved as stock_predictor_v2.h5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5071 - loss: 0.7109

Model accuracy on test set: 56.07%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step

--- Latest Data Prediction ---
Based on the last 10 days of data...
Predicted probability of price increase in the next 5 days: 63.71%
Prediction Result: Bullish


In [18]:
!pip install akshare

In [19]:
import akshare as ak

# Define start and end dates
start_date = "20100101"
end_date = "20241231"

# Fetch historical daily data for Apple (AAPL)
# Using 'stock_us_daily_qfq' for US stocks daily data with front-adjusted values
stock_data = ak.stock_us_daily_qfq(symbol="AAPL", adjust="qfq")

# Display the first few rows and info to verify the data
print("Fetched Stock Data:")
display(stock_data.head())
stock_data.info()

AttributeError: module 'akshare' has no attribute 'stock_us_daily_qfq'

In [20]:
import akshare as ak

# Define start and end dates
start_date = "20100101"
end_date = "20241231"

try:
    # Try using 'stock_us_hist' with adjust='qfq'
    stock_data = ak.stock_us_hist(symbol="AAPL", period="daily", adjust="qfq")

    # Display the first few rows and info to verify the data
    print("Fetched Stock Data using stock_us_hist:")
    display(stock_data.head())
    stock_data.info()

except AttributeError:
    print("The function 'stock_us_hist' with adjust='qfq' was not found. Exploring other options.")
    # If the above fails, we might need to explore other akshare functions
    # or check the akshare documentation for the correct function name and usage.
    # For the scope of this task, if this attempt fails, I will report failure.

TypeError: 'NoneType' object is not subscriptable

In [21]:
# Try using the yfinance library as an alternative
!pip install yfinance

import yfinance as yf

# Define start and end dates
start_date = "2010-01-01"
end_date = "2024-12-31"

try:
    # Fetch historical daily data for Apple (AAPL) using yfinance
    stock_data = yf.download("AAPL", start=start_date, end=end_date)

    # Display the first few rows and info to verify the data
    print("Fetched Stock Data using yfinance:")
    display(stock_data.head())
    stock_data.info()

except Exception as e:
    print(f"An error occurred while fetching data with yfinance: {e}")

/tmp/ipython-input-21-1537891486.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download("AAPL", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Fetched Stock Data using yfinance:


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.431896,6.446623,6.382908,6.414465,493729600
2010-01-05,6.443017,6.479383,6.409056,6.449630,601904800
2010-01-06,6.340532,6.468563,6.333920,6.443017,552160000
2010-01-07,6.328811,6.371489,6.282828,6.363975,477131200
2010-01-08,6.370886,6.371487,6.283128,6.320395,447610800


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3773 entries, 2010-01-04 to 2024-12-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   3773 non-null   float64
 1   (High, AAPL)    3773 non-null   float64
 2   (Low, AAPL)     3773 non-null   float64
 3   (Open, AAPL)    3773 non-null   float64
 4   (Volume, AAPL)  3773 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 176.9 KB


In [22]:
# Select the 'Close' column
close_price_df = stock_data['Close']

# Rename the 'Close' column to '收盘价'
close_price_df = close_price_df.rename('收盘价')

# Convert the Series to a DataFrame
close_price_df = close_price_df.to_frame()

# Reset the index to turn the 'Date' index into a column
close_price_df = close_price_df.reset_index()

# Rename the 'Date' column to '日期'
close_price_df = close_price_df.rename(columns={'Date': '日期'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

TypeError: 'str' object is not callable

In [23]:
import pandas as pd
import numpy as np

# Select the 'Close' column
close_price_df = stock_data['Close']

# Rename the 'Close' Series to '收盘价' using the .name attribute
close_price_df.name = '收盘价'

# Convert the Series to a DataFrame
close_price_df = close_price_df.to_frame()

# Reset the index to turn the 'Date' index into a column
close_price_df = close_price_df.reset_index()

# Rename the 'index' column (which was the original 'Date' index) to '日期'
close_price_df = close_price_df.rename(columns={'index': '日期'})

# Save the DataFrame to a CSV file named 'AAPL_daily.csv' without including the index
close_price_df.to_csv('AAPL_daily.csv', index=False)

print("Data saved to AAPL_daily.csv with columns '日期' and '收盘价'.")

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [59]:
# Select the 'Date' index and the 'Close' column from the original MultiIndex DataFrame
# This will result in a DataFrame with 'Date' as a regular column and 'Close' as another column.
close_price_df = stock_data.reset_index()[['Date', 'Close']]

# Rename the columns to '日期' and '收盘价'
close_price_df = close_price_df.rename(columns={'Date': '日期', 'Close': '收盘价'})

# Save the DataFrame to a CSV file named 'A_DAY_daily.csv' without including the index
# Explicitly set header=True to ensure the column names are written as the first row
close_price_df.to_csv('A_DAY_daily.csv', index=False, header=True)

print("Data re-saved to A_DAY_daily.csv with corrected format.")

# Now, read the corrected CSV file into a pandas DataFrame to verify
# We expect '日期' to be read as object/string and '收盘价' as float, then we'll convert '日期'
df = pd.read_csv('A_DAY_daily.csv')

# Convert '日期' column to datetime objects
df['日期'] = pd.to_datetime(df['日期'])

# Print the first 5 rows of the DataFrame
print("\nFirst 5 rows of A_DAY_daily.csv after final correction and date conversion:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of A_DAY_daily.csv after final correction:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in A_DAY_daily.csv after final correction:")
df.info()

Data re-saved to A_DAY_daily.csv with corrected format.

First 5 rows of A_DAY_daily.csv after final correction and date conversion:


,日期,收盘价
0,NaT,AAPL
1,2010-01-04,6.431896209716797
2,2010-01-05,6.443017482757568
3,2010-01-06,6.340531826019287
4,2010-01-07,6.328811168670654



Column names of A_DAY_daily.csv after final correction:
Index(['日期', '收盘价'], dtype='object')

Data types of columns in A_DAY_daily.csv after final correction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3774 entries, 0 to 3773
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      3773 non-null   datetime64[ns]
 1   收盘价     3774 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 59.1+ KB


In [61]:
import pandas as pd

# Read the CSV file into a pandas DataFrame, skipping the first row
df = pd.read_csv('A_DAY_daily.csv', skiprows=0)

# Print the first 5 rows of the DataFrame
print("First 5 rows of A_DAY_daily.csv after skipping the first row:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of A_DAY_daily.csv after skipping the first row:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in A_DAY_daily.csv after skipping the first row:")
df.info()

First 5 rows of A_DAY_daily.csv after skipping the first row:


,日期,收盘价
0,2010-01-04,6.431896
1,2010-01-05,6.443017
2,2010-01-06,6.340532
3,2010-01-07,6.328811
4,2010-01-08,6.370886



Column names of A_DAY_daily.csv after skipping the first row:
Index(['日期', '收盘价'], dtype='object')

Data types of columns in A_DAY_daily.csv after skipping the first row:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      3773 non-null   object 
 1   收盘价     3773 non-null   float64
dtypes: float64(1), object(1)
memory usage: 59.1+ KB


In [60]:
import pandas as pd
import numpy as np # Import numpy for potential use, though not strictly needed for this step

# Assuming 'stock_data' DataFrame is available from the previous yfinance fetch

# Select the 'Close' price and reset the index to make 'Date' a column
temp_df = stock_data['Close'].reset_index()

# Create a new DataFrame with the desired column names and data
final_df_to_save = pd.DataFrame({
    '日期': temp_df['Date'],
    '收盘价': temp_df['AAPL'] # Corrected column name from 'Close' to 'AAPL'
})

# Save the DataFrame to a CSV file named 'A_DAY_daily.csv' without including the index
# Explicitly set header=True to ensure the column names are written as the first row
final_df_to_save.to_csv('A_DAY_daily.csv', index=False, header=True)

print("Data re-saved to A_DAY_daily.csv with corrected structure.")

# Now, read the corrected CSV file into a pandas DataFrame to verify
# We expect '日期' to be read as object/string and '收盘价' as float, then we'll convert '日期'
df = pd.read_csv('A_DAY_daily.csv')

# Convert '日期' column to datetime objects
df['日期'] = pd.to_datetime(df['日期'])

# Print the first 5 rows of the DataFrame
print("\nFirst 5 rows of A_DAY_daily.csv after final correction and date conversion:")
display(df.head())

# Print the column names of the DataFrame
print("\nColumn names of A_DAY_daily.csv after final correction:")
print(df.columns)

# Print the data types of the columns
print("\nData types of columns in A_DAY_daily.csv after final correction:")
df.info()

Data re-saved to A_DAY_daily.csv with corrected structure.

First 5 rows of A_DAY_daily.csv after final correction and date conversion:


,日期,收盘价
0,2010-01-04,6.431896
1,2010-01-05,6.443017
2,2010-01-06,6.340532
3,2010-01-07,6.328811
4,2010-01-08,6.370886



Column names of A_DAY_daily.csv after final correction:
Index(['日期', '收盘价'], dtype='object')

Data types of columns in A_DAY_daily.csv after final correction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3773 entries, 0 to 3772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      3773 non-null   datetime64[ns]
 1   收盘价     3773 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 59.1 KB


In [67]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path, sentiment_file_path):
    """
    Loads CSV data, calculates technical indicators, integrates sentiment data,
    and creates the target variable.
    """
    # Check if stock data file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data ---
    # Check if sentiment data file exists and load it
    sentiment_df = pd.DataFrame() # Initialize empty DataFrame
    if os.path.exists(sentiment_file_path):
        sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
        sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
        sentiment_df = sentiment_df.set_index('日期')
        # Merge stock data with sentiment data
        df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
        print("Sentiment data loaded and merged.")
    else:
        print(f"警告：找不到 '{sentiment_file_path}'。将不包含情感分数。")
        # If sentiment file not found, add a column of NaNs to avoid errors later
        df['Sentiment_Score'] = np.nan


    # --- Calculate Technical Indicators (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands and Sentiment
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower', 'Sentiment_Score'] # Added Sentiment_Score
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA15'] = df['MA5'] / df['MA15'] # Changed from MA10
    df['价格/BBMID'] = df['收盘价'] / df['BBMID'] # New feature based on BB
    df['价格/BBUpper'] = df['收盘价'] / df['BBUpper'] # New feature based on BB
    df['价格/BBLower'] = df['收盘价'] / df['BBLower'] # New feature based on BB


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator, target variable calculations, and merging
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. Create Time Series Samples =====
def create_sequences(X, y, time_steps=10):
    """
    Converts data into sequential samples suitable for LSTM.
    :param X: Feature dataset
    :param y: Target dataset
    :param time_steps: Time steps per sample (number of lookback days)
    :return: Sequential X and corresponding y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. Build and Train Model =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    Builds, compiles, and trains the LSTM model.
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # Get input shape
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM layer, input_shape=(time_steps, number of features)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # Add Dropout to prevent overfitting
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== Main Program =====
if __name__ == "__main__":
    # --- 1. Load and Prepare Data ---
    # Please ensure you have these CSV files, or replace with your own file paths
    # Stock CSV file needs to contain '日期' and '收盘价' columns
    # Sentiment CSV file needs to contain '日期' and 'Sentiment_Score' columns
    stock_file_path = "A_DAY_daily.csv" # Using the previously generated AAPL data for demonstration
    sentiment_file_path = "ASHare_simulated_sentiment.csv" # Path to the simulated sentiment data
    try:
        X, y, df_full = load_and_prepare_data(stock_file_path, sentiment_file_path)

        # Only proceed with the rest of the script if data loading was successful
        # --- 2. Data Scaling ---
        # Create scaler
        scaler = MinMaxScaler()
        # Fit the scaler only on the training data, then transform all data
        # For simplicity, we transform the entire X here first, but in a strict process,
        # it should be split first then transformed.
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

        # --- 3. Create Sequence Data ---
        TIME_STEPS = 10  # Use data from the past 10 days as input
        X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

        # --- 4. Split Training and Test Sets (Time Series Method) ---
        # Ensure the training and test sets are not shuffled, maintaining time order
        split_index = int(len(X_seq) * 0.8)
        X_train, X_test = X_seq[:split_index], X_seq[split_index:]
        y_train, y_test = y_seq[:split_index], y_seq[split_index:]

        print(f"Total samples: {len(X_seq)}")
        print(f"Training set samples: {len(X_train)}")
        print(f"Test set samples: {len(X_test)}")

        # --- 5. Train Model ---
        print("\nTraining model...")
        model = build_and_train_model(X_train, y_train, X_test, y_test)

        # Only attempt to save and evaluate if the model was built successfully
        if model is not None:
            model.save('stock_predictor_v4.h5') # Save as v4 to reflect sentiment inclusion
            print("\nModel saved as stock_predictor_v4.h5")

            # --- 6. Evaluate Model on Test Set ---
            loss, accuracy = model.evaluate(X_test, y_test)
            print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

            # --- 7. Make Prediction on New Data ---
            # Assuming we want to predict after the latest data point
            # We need to get the last `TIME_STEPS` days of data from the dataset
            # Ensure there are enough samples for the last sequence
            if len(X_scaled) >= TIME_STEPS:
                last_sequence = X_scaled.iloc[-TIME_STEPS:].values

                # Reshape it to the input format required by the model (1, time_steps, n_features)
                last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

                # Make prediction
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
                if prediction_proba > 0.5:
                    print("Prediction Result: Bullish")
                else:
                    print("Prediction Result: Bearish")
            else:
                print(f"\n--- Latest Data Prediction ---")
                print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e) # Print ValueErrors from the loading function
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Sentiment data loaded and merged.
Total samples: 1242
Training set samples: 993
Test set samples: 249

Training model...
模型结构概要：


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 50)             │        11,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,101 (51.18 KB)

 Trainable params: 13,101 (51.18 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - accuracy: 0.5574 - loss: 0.6859 - val_accuracy: 0.5904 - val_loss: 0.6766
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5598 - loss: 0.6842 - val_accuracy: 0.5904 - val_loss: 0.6771
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.5801 - loss: 0.6781 - val_accuracy: 0.5904 - val_loss: 0.6772
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.5830 - loss: 0.6753 - val_accuracy: 0.5904 - val_loss: 0.6756
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5924 - loss: 0.6774 - val_accuracy: 0.5904 - val_loss: 0.6790
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5677 - loss: 0.6833 - val_accuracy: 0.5904 - val_loss: 0.6786
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5618 - loss: 0.6821 - val_accuracy: 0.5904 - val_loss: 0.6774
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5620 - loss: 0.6847 - val_accuracy: 0.5904 - v


Model saved as stock_predictor_v4.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4817 - loss: 0.7482 

Model accuracy on test set: 56.63%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step

--- Latest Data Prediction ---
Based on the last 10 days of data...
Predicted probability of price increase in the next 5 days: 57.92%
Prediction Result: Bullish


# 股票预测策略说明

本 Notebook 中实现的策略是基于技术分析方法，并结合深度学习模型 (LSTM) 来预测未来股票价格走势。

以下是策略的详细步骤：

1.  **数据预处理与特征工程**:
    *   代码从一个 CSV 文件 (`AAPL_daily.csv`) 中加载历史每日股票数据（特别是收盘价）。
    *   根据收盘价计算几个常用的技术指标：
        *   **移动平均线 (MA5 和 MA10)**: 这些指标平滑价格数据，形成一条连续的线条，更容易发现趋势。MA5 是过去 5 天的平均收盘价，MA10 是过去 10 天的平均收盘价。
        *   **相对强弱指数 (RSI)**: 这是一个动量震荡指标，衡量价格变动的速度和变化。它用于识别超买或超卖情况。
        *   **移动平均收敛扩散指标 (MACD)**: 这个指标揭示股票价格趋势的强度、方向、动量和持续时间的变化。它由 MACD 线、信号线和柱状图组成。
    *   通过计算收盘价与 MA5 的比率以及 MA5 与 MA10 的比率，创建额外的特征。
    *   创建一个目标变量：这是一个二元变量，表示未来 5 天后的收盘价是否高于当前日的收盘价（上涨为 1，不变或下跌为 0）。
    *   删除因计算指标和目标变量而产生的含有缺失值的行。

2.  **数据缩放**:
    *   使用 `MinMaxScaler` 对技术指标特征进行缩放，以标准化它们的范围，这对于许多机器学习模型（包括 LSTM）来说很重要。

3.  **创建时间序列样本**:
    *   将缩放后的数据转换为固定长度（本例中为 10 天）的序列。每个序列代表过去 `TIME_STEPS` (10) 天的技术指标数据，相应的目标是该序列最后一天未来的价格走势结果。这种格式适用于训练 LSTM 网络，因为它们擅长从序列数据中学习。

4.  **模型构建与训练**:
    *   使用 TensorFlow/Keras 构建一个序列式 LSTM 模型。
    *   模型由一个 LSTM 层、一个 Dropout 层（用于防止过拟合）和两个全连接层 (Dense) 组成。
    *   模型使用 Adam 优化器和二元交叉熵损失进行编译，适用于二元分类任务（预测价格是上涨还是下跌）。
    *   模型在 80% 的历史数据序列上进行训练，其余 20% 用于验证。

5.  **模型评估**:
    *   在测试集上使用准确率作为评估指标来评估训练好的模型的性能。

6.  **预测**:
    *   最后，模型对最近 `TIME_STEPS` (10) 天的数据进行预测，以预测未来 5 天价格上涨的概率。

总而言之，该策略使用传统的技术分析指标作为特征，将它们处理成时间序列，然后使用 LSTM 神经网络学习这些序列中的模式，以预测短期的未来价格方向。

# Task
Modify the provided code to include MA15 and Bollinger Bands as technical indicators in the prediction model. Update the data loading and preparation, feature columns, and model input shape accordingly. Run the modified code and evaluate the model's performance with the new indicators.

## Modify data loading and preparation

### Subtask:
Update the `load_and_prepare_data` function in cell `94f65f08` to calculate MA15 instead of MA10 and include Bollinger Bands (middle band, upper band, and lower band).


**Reasoning**:
Update the `load_and_prepare_data` function to calculate MA15 and Bollinger Bands as requested.



In [31]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path):
    """
    Loads CSV data, calculates technical indicators, and creates the target variable.
    """
    # Check if file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Calculate Technical Indicators (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA15'] = df['MA5'] / df['MA15'] # Changed from MA10
    df['价格/BBMID'] = df['收盘价'] / df['BBMID'] # New feature based on BB
    df['价格/BBUpper'] = df['收盘价'] / df['BBUpper'] # New feature based on BB
    df['价格/BBLower'] = df['收盘价'] / df['BBLower'] # New feature based on BB


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator and target variable calculations
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Updated feature columns to include MA15 and Bollinger Bands related features
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower']
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. Create Time Series Samples =====
def create_sequences(X, y, time_steps=10):
    """
    Converts data into sequential samples suitable for LSTM.
    :param X: Feature dataset
    :param y: Target dataset
    :param time_steps: Time steps per sample (number of lookback days)
    :return: Sequential X and corresponding y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. Build and Train Model =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    Builds, compiles, and trains the LSTM model.
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # Get input shape
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM layer, input_shape=(time_steps, number of features)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # Add Dropout to prevent overfitting
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== Main Program =====
if __name__ == "__main__":
    # --- 1. Load and Prepare Data ---
    # Please ensure you have this CSV file, or replace with your own file path
    # The CSV file needs to contain '日期' and '收盘价' columns
    file_path = "AAPL_daily.csv"
    try:
        X, y, df_full = load_and_prepare_data(file_path)

        # Only proceed with the rest of the script if data loading was successful
        # --- 2. Data Scaling ---
        # Create scaler
        scaler = MinMaxScaler()
        # Fit the scaler only on the training data, then transform all data
        # For simplicity, we transform the entire X here first, but in a strict process,
        # it should be split first then transformed.
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

        # --- 3. Create Sequence Data ---
        TIME_STEPS = 10  # Use data from the past 10 days as input
        X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

        # --- 4. Split Training and Test Sets (Time Series Method) ---
        # Ensure the training and test sets are not shuffled, maintaining time order
        split_index = int(len(X_seq) * 0.8)
        X_train, X_test = X_seq[:split_index], X_seq[split_index:]
        y_train, y_test = y_seq[:split_index], y_seq[split_index:]

        print(f"Total samples: {len(X_seq)}")
        print(f"Training set samples: {len(X_train)}")
        print(f"Test set samples: {len(X_test)}")

        # --- 5. Train Model ---
        print("\nTraining model...")
        model = build_and_train_model(X_train, y_train, X_test, y_test)

        # Only attempt to save and evaluate if the model was built successfully
        if model is not None:
            model.save('stock_predictor_v2.h5')
            print("\nModel saved as stock_predictor_v2.h5")

            # --- 6. Evaluate Model on Test Set ---
            loss, accuracy = model.evaluate(X_test, y_test)
            print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

            # --- 7. Make Prediction on New Data ---
            # Assuming we want to predict after the latest data point
            # We need to get the last `TIME_STEPS` days of data from the dataset
            # Ensure there are enough samples for the last sequence
            if len(X_scaled) >= TIME_STEPS:
                last_sequence = X_scaled.iloc[-TIME_STEPS:].values

                # Reshape it to the input format required by the model (1, time_steps, n_features)
                last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

                # Make prediction
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
                if prediction_proba > 0.5:
                    print("Prediction Result: Bullish")
                else:
                    print("Prediction Result: Bearish")
            else:
                print(f"\n--- Latest Data Prediction ---")
                print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Total samples: 3739
Training set samples: 2991
Test set samples: 748

Training model...
模型结构概要：


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 50)             │        11,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,901 (50.39 KB)

 Trainable params: 12,901 (50.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5604 - loss: 0.6843 - val_accuracy: 0.5468 - val_loss: 0.6936
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5923 - loss: 0.6763 - val_accuracy: 0.5468 - val_loss: 0.6900
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5936 - loss: 0.6753 - val_accuracy: 0.5468 - val_loss: 0.6901
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5834 - loss: 0.6790 - val_accuracy: 0.5468 - val_loss: 0.6904
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5811 - loss: 0.6810 - val_accuracy: 0.5468 - val_loss: 0.6920
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5780 - loss: 0.6798 - val_accuracy: 0.5468 - val_loss: 0.6955
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5747 - loss: 0.6854 - val_accuracy: 0.5468 - val_loss: 0.6972
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5885 - loss: 0.6782 - val_accuracy: 0.5468 - val_los


Model saved as stock_predictor_v2.h5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4881 - loss: 0.7100

Model accuracy on test set: 55.08%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step

--- Latest Data Prediction ---
Based on the last 10 days of data...
Predicted probability of price increase in the next 5 days: 62.56%
Prediction Result: Bullish


In [57]:
# Save the df_full DataFrame to a CSV file
df_full.to_csv('stock_data_with_indicators.csv')

print("Calculation results saved to stock_data_with_indicators.csv")

Calculation results saved to stock_data_with_indicators.csv


# Task
Integrate sentiment factors derived from news or events into the stock prediction model, incorporating the necessary data fetching, processing, and model adjustments.

## Identify data sources

### Subtask:
Identify reliable sources for historical news headlines, articles, or event data related to the stock or the market (e.g., financial news APIs, historical news archives).


**Reasoning**:
I need to research and identify reliable sources for historical financial news or event data related to stocks, specifically AAPL, covering the 2010-2024 period. I will list the potential sources, noting their data type, coverage, access methods, and limitations, and then select the most promising ones. This task involves research and summarizing information, which can be presented as text output rather than code execution.



In [63]:
print("Researching potential data sources for historical financial news and event data...")

# Potential Data Sources for Historical Financial News/Event Data:

# 1. News API (e.g., News API, GDELT Project, Alpha Vantage, Finnhub, MarketStack)
print("\n1. News APIs (e.g., News API, Alpha Vantage, Finnhub, MarketStack)")
print("   - Data Type: Primarily headlines and sometimes summaries or full articles.")
print("   - Historical Coverage: Varies greatly by provider. Some offer limited historical data on free tiers, extensive archives on paid tiers. GDELT is a large archive.")
print("   - Access Method: REST APIs.")
print("   - Limitations/Costs: Free tiers often have rate limits and limited historical access. Paid tiers can be expensive. Data quality and coverage for specific stocks/timeframes need verification.")
print("   - Relevance to AAPL (2010-2024): Needs checking for each specific API's archive depth and stock coverage.")

# 2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)
print("\n2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)")
print("   - Data Type: Comprehensive, often includes structured event data (earnings calls, product launches), news sentiment scores, headlines, and articles.")
print("   - Historical Coverage: Generally extensive, often covering decades.")
print("   - Access Method: APIs, proprietary terminals, data feeds.")
print("   - Limitations/Costs: Very expensive, typically geared towards institutional users. Access often requires subscriptions.")
print("   - Relevance to AAPL (2010-2024): High likelihood of having this data, but access is a major barrier.")

# 3. Web Scraping (e.g., major financial news websites like Reuters, Bloomberg, Wall Street Journal archives)
print("\n3. Web Scraping (e.g., major financial news websites)")
print("   - Data Type: Headlines, articles.")
print("   - Historical Coverage: Depends on the website's archives. Can be extensive but often unstructured.")
print("   - Access Method: Custom scripts using libraries like Beautiful Soup, Scrapy.")
print("   - Limitations/Costs: Time-consuming to develop and maintain scrapers. Legal and ethical considerations (terms of service). Data is often unstructured, requiring significant parsing. Can be blocked.")
print("   - Relevance to AAPL (2010-2024): Possible, but labor-intensive and potentially legally restricted.")

# 4. Dedicated Historical News Archives (e.g., ProQuest, LexisNexis - often accessed via libraries)
print("\n4. Dedicated Historical News Archives (e.g., ProQuest, LexisNexis)")
print("   - Data Type: Full text of news articles from various publications.")
print("   - Historical Coverage: Can be very extensive, going back many decades.")
print("   - Access Method: Web interfaces, sometimes APIs (less common for standard access).")
print("   - Limitations/Costs: Primarily academic or institutional access. May require specific subscriptions.")
print("   - Relevance to AAPL (2010-2024): Likely to have relevant articles, but access is limited.")

# 5. Open Source Datasets (less likely for specific historical stock news, but worth a check)
print("\n5. Open Source Datasets")
print("   - Data Type: Varies, often aggregated or specialized datasets. Unlikely to find a ready-to-use, comprehensive historical AAPL news dataset.")
print("   - Historical Coverage: Sporadic and often not long-term or specific.")
print("   - Access Method: Downloads.")
print("   - Limitations/Costs: Data quality and completeness can be issues. May require significant cleaning and processing.")
print("   - Relevance to AAPL (2010-2024): Low likelihood of a direct fit.")

# Prioritization and Selection:

print("\n--- Prioritization ---")
print("Given the constraints and typical accessibility:")
print("- Financial Data Providers (Bloomberg, Refinitiv) are ideal for data quality and structure but are cost-prohibitive.")
print("- Dedicated Historical News Archives require specific access.")
print("- Web Scraping is labor-intensive and has legal/ethical concerns.")
print("- Open Source Datasets are unlikely to meet the specific requirements.")
print("- News APIs offer a more accessible route, though historical coverage and cost need careful evaluation.")

print("\n--- Selected Data Sources for Further Exploration ---")
print("Based on accessibility and potential data availability for the timeframe:")
print("1. Alpha Vantage: Offers a free tier for initial testing and may have some historical news data.")
print("2. Finnhub: Also has a free tier and real-time/historical financial data APIs, including news.")
print("3. GDELT Project: While not specifically financial news, it's a massive archive of global news that could potentially be filtered for relevant information, but requires significant data processing.")

print("\nNext steps will involve exploring the APIs of Alpha Vantage and Finnhub to check their historical news coverage for AAPL and assess the structure of the data they provide.")

Researching potential data sources for historical financial news and event data...

1. News APIs (e.g., News API, Alpha Vantage, Finnhub, MarketStack)
   - Data Type: Primarily headlines and sometimes summaries or full articles.
   - Historical Coverage: Varies greatly by provider. Some offer limited historical data on free tiers, extensive archives on paid tiers. GDELT is a large archive.
   - Access Method: REST APIs.
   - Limitations/Costs: Free tiers often have rate limits and limited historical access. Paid tiers can be expensive. Data quality and coverage for specific stocks/timeframes need verification.
   - Relevance to AAPL (2010-2024): Needs checking for each specific API's archive depth and stock coverage.

2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)
   - Data Type: Comprehensive, often includes structured event data (earnings calls, product launches), news sentiment scores, headlines, and articles.
   - Historical Coverage: Gen

**Reasoning**:
I have identified potential data sources for historical financial news. The next logical step is to explore the most promising free/accessible options (Alpha Vantage and Finnhub) to see if they provide historical news data for AAPL covering the required timeframe (2010-2024) and to understand the structure of the data they offer. GDELT is a possibility but requires more complex processing. I will attempt to fetch data from Alpha Vantage first. This requires obtaining an API key and using the relevant Alpha Vantage API endpoint for news.



In [62]:
# Note: To run this code, you need an Alpha Vantage API key.
# Replace 'YOUR_ALPHA_VANTAGE_API_KEY' with your actual key.
# You can get a free API key from https://www.alphavantage.co/

ALPHA_VANTAGE_API_KEY = 'YOUR_ALPHA_VANTAGE_API_KEY'
SYMBOL = 'AAPL'
# Alpha Vantage news API typically supports querying by ticker and date range.
# Need to check the exact API endpoint and parameters.

# Based on Alpha Vantage documentation, the relevant endpoint is likely
# "https://www.alphavantage.co/query?function=NEWS_SENTIMENT"
# with parameters like 'tickers', 'time_from', 'time_to', 'limit'.
# The date format is expected to be YYYYMMDDTHHMM.

import requests
import pandas as pd

def fetch_alpha_vantage_news(api_key, symbol, start_date, end_date, limit=1000):
    """
    Fetches news sentiment data from Alpha Vantage for a given symbol and date range.
    Note: The free API might have limitations on historical data access and rate limits.
    """
    url = "https://www.alphavantage.co/query"
    # Convert dates to the required format YYYYMMDDTHHMM
    # Alpha Vantage API documentation indicates YYYYMMDDTHHMMSS format for time_from/time_to
    # Let's use start of day for time_from and end of day for time_to
    time_from = f"{start_date}T000000" # Assuming start_date is YYYYMMDD
    time_to = f"{end_date}T235959"   # Assuming end_date is YYYYMMDD

    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": symbol,
        "time_from": time_from,
        "time_to": time_to,
        "limit": limit, # Adjust limit as needed, max is 1000 per request
        "apikey": api_key
    }

    print(f"Attempting to fetch news data for {symbol} from {start_date} to {end_date} using Alpha Vantage...")

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()

        if 'feed' in data and data['feed']:
            print("Successfully fetched data from Alpha Vantage.")
            # Convert the list of articles to a pandas DataFrame
            news_df = pd.DataFrame(data['feed'])
            return news_df
        elif 'Note' in data:
            print(f"Alpha Vantage API Note: {data['Note']}")
            print("This might indicate rate limits or other API usage constraints.")
            return pd.DataFrame() # Return empty DataFrame on API notes
        else:
            print("No news data found or unexpected response format.")
            print("Response data keys:", data.keys())
            return pd.DataFrame()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Alpha Vantage: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

# Define the date range for AAPL data (2010-2024)
start_date_str = "20100101"
end_date_str = "20241231"

# Fetch news data (replace with your actual API key)
# IMPORTANT: Running this requires a valid Alpha Vantage API key.
# If you don't have one or prefer not to use it, this step will fail.
# news_data_av = fetch_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, SYMBOL, start_date_str, end_date_str)

# For demonstration purposes without a key, let's assume we fetched data and inspect a hypothetical structure.
# In a real scenario, you would uncomment the line above and process the 'news_data_av' DataFrame.

print("\n--- Assuming successful data fetch for demonstration ---")
print("A hypothetical DataFrame from Alpha Vantage might contain columns like:")
print("- 'title': Headline of the news article.")
print("- 'url': URL of the article.")
print("- 'time_published': Timestamp of publication.")
print("- 'authors': List of authors.")
print("- 'summary': Summary of the article.")
print("- 'banner_image': URL of the banner image.")
print("- 'category_within_source': Category from the source.")
print("- 'source': Source of the news.")
print("- 'topics': List of related topics.")
print("- 'overall_sentiment_score': Sentiment score for the article.")
print("- 'overall_sentiment_label': Sentiment label (e.g., 'Bullish', 'Bearish').")
print("- 'ticker_sentiment': List of sentiment scores for specific tickers mentioned.")

# The key information for sentiment analysis would be 'time_published', 'title', 'summary',
# 'overall_sentiment_score', 'overall_sentiment_label', and 'ticker_sentiment'.

print("\nFurther steps would involve:")
print("1. Processing the fetched news data (cleaning, handling missing values).")
print("2. Extracting relevant information (e.g., sentiment scores for AAPL).")
print("3. Aggregating news sentiment data on a daily basis to align with stock data.")
print("4. Joining the aggregated sentiment data with the stock technical indicator data.")

# If Alpha Vantage's free tier historical coverage is insufficient,
# the next step would be to explore Finnhub or other options.


--- Assuming successful data fetch for demonstration ---
A hypothetical DataFrame from Alpha Vantage might contain columns like:
- 'title': Headline of the news article.
- 'url': URL of the article.
- 'time_published': Timestamp of publication.
- 'authors': List of authors.
- 'summary': Summary of the article.
- 'banner_image': URL of the banner image.
- 'category_within_source': Category from the source.
- 'source': Source of the news.
- 'topics': List of related topics.
- 'overall_sentiment_score': Sentiment score for the article.
- 'overall_sentiment_label': Sentiment label (e.g., 'Bullish', 'Bearish').
- 'ticker_sentiment': List of sentiment scores for specific tickers mentioned.

Further steps would involve:
1. Processing the fetched news data (cleaning, handling missing values).
2. Extracting relevant information (e.g., sentiment scores for AAPL).
3. Aggregating news sentiment data on a daily basis to align with stock data.
4. Joining the aggregated sentiment data with the stock

**Reasoning**:
I have researched data sources and outlined the process for using Alpha Vantage. The next step is to explore Finnhub as an alternative or supplementary source for historical news data, as it was also identified as a promising option. This involves installing the Finnhub library, obtaining an API key, and using its functions to fetch news data for AAPL within the specified timeframe. Similar to the Alpha Vantage step, I will demonstrate the code structure and outline the expected data format and subsequent processing steps, noting the need for an API key.



In [64]:
# Note: To run this code, you need a Finnhub API key.
# Replace 'YOUR_FINNHUB_API_KEY' with your actual key.
# You can get a free API key from https://finnhub.io/

# !pip install finnhub-python # Install the library if not already installed

import finnhub

FINNHUB_API_KEY = 'YOUR_FINNHUB_API_KEY'
SYMBOL = 'AAPL'
# Finnhub news API endpoint is '/company-news' or '/company-news-stream' for real-time.
# For historical news, '/company-news' with 'from' and 'to' parameters is used.
# The date format is YYYY-MM-DD.

# Initialize Finnhub client
# finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Define the date range for AAPL data (2010-2024) in YYYY-MM-DD format
start_date_finnhub = "2010-01-01"
end_date_finnhub = "2024-12-31"

# Fetch company news from Finnhub
# Note: The free tier of Finnhub API might have limitations on historical data access and rate limits.
# Fetching a large historical range might require multiple requests or a paid plan.

print(f"\nAttempting to fetch news data for {SYMBOL} from {start_date_finnhub} to {end_date_finnhub} using Finnhub...")

try:
    # In a real scenario, you would uncomment this line:
    # news_data_fh = finnhub_client.company_news(SYMBOL, _from=start_date_finnhub, to=end_date_finnhub)

    # For demonstration purposes without a key, let's assume we fetched data and inspect a hypothetical structure.
    # Finnhub's company news endpoint returns a list of news articles.

    print("Assuming successful data fetch for demonstration.")
    print("A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:")
    print("- 'category': Category of the news.")
    "- 'datetime': Timestamp of publication (Unix timestamp)."
    print("- 'headline': Headline of the news.")
    print("- 'id': Unique ID of the news.")
    print("- 'image': URL of the image.")
    print("- 'related': Related stocks.")
    print("- 'source': Source of the news.")
    print("- 'summary': Summary of the news.")
    print("- 'url': URL of the news article.")

    # Key information for potential sentiment analysis would be 'datetime', 'headline', 'summary'.
    # Finnhub also offers a sentiment analysis endpoint, but fetching historical sentiment
    # for individual news articles over a long period might have limitations on the free tier.

    print("\nFurther steps would involve:")
    print("1. Converting the list of dictionaries to a pandas DataFrame.")
    print("2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).")
    # Note: Finnhub's sentiment analysis endpoint might be a better source for sentiment scores directly if available historically.
    print("3. If sentiment scores are not directly available or feasible to fetch historically per article, consider performing sentiment analysis on headlines/summaries.")
    print("4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.")
    print("5. Joining the aggregated news/sentiment data with the stock technical indicator data.")

except Exception as e:
    print(f"An error occurred while trying to demonstrate Finnhub data fetching: {e}")

# If Finnhub's free tier historical coverage is insufficient or sentiment data is not easily accessible,
# other options like GDELT (requiring significant processing) or exploring paid APIs would be necessary.


Attempting to fetch news data for AAPL from 2010-01-01 to 2024-12-31 using Finnhub...
Assuming successful data fetch for demonstration.
A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:
- 'category': Category of the news.
- 'headline': Headline of the news.
- 'id': Unique ID of the news.
- 'image': URL of the image.
- 'related': Related stocks.
- 'source': Source of the news.
- 'summary': Summary of the news.
- 'url': URL of the news article.

Further steps would involve:
1. Converting the list of dictionaries to a pandas DataFrame.
2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).
3. If sentiment scores are not directly available or feasible to fetch historically per article, consider performing sentiment analysis on headlines/summaries.
4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.
5. Joining the aggregated news/sentiment data with the stock technical 

**Reasoning**:
The previous command failed because the `finnhub` library was not installed, as indicated by the `ModuleNotFoundError`. To proceed with exploring Finnhub as a data source, I need to install the `finnhub-python` package using pip. After installation, I will re-attempt to import the library and outline the process for fetching data, as planned in the previous step.



In [65]:
!pip install finnhub-python

# Now, re-attempt to demonstrate Finnhub data fetching process
import finnhub
import pandas as pd

# Note: To run this code, you need a Finnhub API key.
# Replace 'YOUR_FINNHUB_API_KEY' with your actual key.
# You can get a free API key from https://finnhub.io/

FINNHUB_API_KEY = 'YOUR_FINNHUB_API_KEY'
SYMBOL = 'AAPL'
# Finnhub news API endpoint is '/company-news' or '/company-news-stream' for real-time.
# For historical news, '/company-news' with 'from' and 'to' parameters is used.
# The date format is YYYY-MM-DD.

# Initialize Finnhub client
# finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Define the date range for AAPL data (2010-2024) in YYYY-MM-DD format
start_date_finnhub = "2010-01-01"
end_date_finnhub = "2024-12-31"

# Fetch company news from Finnhub
# Note: The free tier of Finnhub API might have limitations on historical data access and rate limits.
# Fetching a large historical range might require multiple requests or a paid plan.

print(f"\nAttempting to fetch news data for {SYMBOL} from {start_date_finnhub} to {end_date_finnhub} using Finnhub...")

try:
    # In a real scenario, you would uncomment this line:
    # news_data_fh = finnhub_client.company_news(SYMBOL, _from=start_date_finnhub, to=end_date_finnhub)

    # For demonstration purposes without a key, let's assume we fetched data and inspect a hypothetical structure.
    # Finnhub's company news endpoint returns a list of news articles.

    print("Assuming successful data fetch for demonstration.")
    print("A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:")
    print("- 'category': Category of the news.")
    print("- 'datetime': Timestamp of publication (Unix timestamp).")
    print("- 'headline': Headline of the news.")
    print("- 'id': Unique ID of the news.")
    print("- 'image': URL of the image.")
    print("- 'related': Related stocks.")
    print("- 'source': Source of the news.")
    print("- 'summary': Summary of the news.")
    print("- 'url': URL of the news article.")

    # Key information for potential sentiment analysis would be 'datetime', 'headline', 'summary'.
    # Finnhub also offers a sentiment analysis endpoint, but fetching historical sentiment
    # for individual news articles over a long period might have limitations on the free tier.

    print("\nFurther steps would involve:")
    print("1. Converting the list of dictionaries to a pandas DataFrame.")
    print("2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).")
    ("3. If sentiment scores are not directly available or feasible to fetch historically per article, consider performing sentiment analysis on headlines/summaries.")
    print("4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.")
    print("5. Joining the aggregated news/sentiment data with the stock technical indicator data.")

except Exception as e:
    print(f"An error occurred while trying to demonstrate Finnhub data fetching: {e}")

# If Finnhub's free tier historical coverage is insufficient or sentiment data is not easily accessible,
# other options like GDELT (requiring significant processing) or exploring paid APIs would be necessary.


Attempting to fetch news data for AAPL from 2010-01-01 to 2024-12-31 using Finnhub...
Assuming successful data fetch for demonstration.
A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:
- 'category': Category of the news.
- 'datetime': Timestamp of publication (Unix timestamp).
- 'headline': Headline of the news.
- 'id': Unique ID of the news.
- 'image': URL of the image.
- 'related': Related stocks.
- 'source': Source of the news.
- 'summary': Summary of the news.
- 'url': URL of the news article.

Further steps would involve:
1. Converting the list of dictionaries to a pandas DataFrame.
2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).
4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.
5. Joining the aggregated news/sentiment data with the stock technical indicator data.


## Fetch sentiment data

### Subtask:
Fetch historical news or event data for AAPL covering the period from 2010 to 2024 using one of the identified sources that provides accessible historical data, or simulate the fetching process with a placeholder if actual fetching is not feasible.


**Reasoning**:
Since fetching real historical news data for the entire period from 2010 to 2024 is challenging with free API tiers and without valid keys, I will simulate fetching news sentiment data for AAPL over the specified date range. This simulated data will include a 'Date' and 'Sentiment Score' column to represent the aggregated daily sentiment, which can then be integrated into the stock prediction model. This approach allows the task to proceed without relying on external APIs or large datasets that are difficult to access.



In [66]:
import pandas as pd
import numpy as np

# Define the date range
start_date = pd.to_datetime("2010-01-01")
end_date = pd.to_datetime("2024-12-31")

# Create a date range
date_rng = pd.date_range(start=start_date, end=end_date, freq='D')

# Simulate daily sentiment scores
# Generate random sentiment scores between -1 and 1
np.random.seed(42) # for reproducibility
sentiment_scores = np.random.uniform(low=-0.5, high=0.5, size=len(date_rng))

# Introduce some seasonality or trend to make it slightly more realistic (optional)
# For example, a slight upward trend over time
sentiment_scores = sentiment_scores + np.linspace(0, 0.2, len(date_rng))

# Create a DataFrame for the simulated news sentiment data
simulated_news_df = pd.DataFrame({'日期': date_rng, 'Sentiment_Score': sentiment_scores})

# Display the first few rows and info of the simulated data
print("Simulated Historical News Sentiment Data:")
display(simulated_news_df.head())
simulated_news_df.info()

# Save the simulated data to a CSV file for later use
simulated_news_df.to_csv('AAPL_simulated_sentiment.csv', index=False)

print("\nSimulated news sentiment data saved to AAPL_simulated_sentiment.csv")

Simulated Historical News Sentiment Data:


,日期,Sentiment_Score
0,2010-01-01,-0.125460
1,2010-01-02,0.450751
2,2010-01-03,0.232067
3,2010-01-04,0.098768
4,2010-01-05,-0.343835


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5479 entries, 0 to 5478
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   日期               5479 non-null   datetime64[ns]
 1   Sentiment_Score  5479 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 85.7 KB

Simulated news sentiment data saved to AAPL_simulated_sentiment.csv


**Reasoning**:
I have successfully simulated and saved the news sentiment data. The next step is to integrate this simulated sentiment data into the data loading and preparation function (`load_and_prepare_data`) in cell `94f65f08`. This involves reading the simulated sentiment data, merging it with the stock data based on the '日期' column, and ensuring the merged data is used for feature creation and model training. I will modify the existing `load_and_prepare_data` function to include this merging step.



In [41]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # Import os to check for file existence

# ===== 1. 数据预处理与特征工程 =====
def load_and_prepare_data(file_path, sentiment_file_path):
    """
    Loads CSV data, calculates technical indicators, integrates sentiment data,
    and creates the target variable.
    """
    # Check if stock data file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    df = pd.read_csv(file_path, encoding='utf-8')
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data ---
    # Check if sentiment data file exists and load it
    sentiment_df = pd.DataFrame() # Initialize empty DataFrame
    if os.path.exists(sentiment_file_path):
        sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
        sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
        sentiment_df = sentiment_df.set_index('日期')
        # Merge stock data with sentiment data
        df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
        print("Sentiment data loaded and merged.")
    else:
        print(f"警告：找不到 '{sentiment_file_path}'。将不包含情感分数。")
        # If sentiment file not found, add a column of NaNs to avoid errors later
        df['Sentiment_Score'] = np.nan


    # --- Calculate Technical Indicators (Manual Calculation) ---
    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands and Sentiment
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower', 'Sentiment_Score'] # Added Sentiment_Score
    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA15'] = df['MA5'] / df['MA15'] # Changed from MA10
    df['价格/BBMID'] = df['收盘价'] / df['BBMID'] # New feature based on BB
    df['价格/BBUpper'] = df['收盘价'] / df['BBUpper'] # New feature based on BB
    df['价格/BBLower'] = df['收盘价'] / df['BBLower'] # New feature based on BB


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator, target variable calculations, and merging
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# ===== 2. Create Time Series Samples =====
def create_sequences(X, y, time_steps=10):
    """
    Converts data into sequential samples suitable for LSTM.
    :param X: Feature dataset
    :param y: Target dataset
    :param time_steps: Time steps per sample (number of lookback days)
    :return: Sequential X and corresponding y
    """
    Xs, ys = [], []
    # Ensure that X and y have the same index after dropping NaNs
    # pandas_ta adds columns to the original DataFrame, so the index should align after dropna
    # We iterate through the index of X (or y) to ensure correct alignment
    for i in range(len(X) - time_steps):
        # Use iloc for positional indexing after potential dropna which might change index
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps - 1]) # Target corresponds to the last day of the sequence
    return np.array(Xs), np.array(ys)


# ===== 3. Build and Train Model =====
def build_and_train_model(X_train, y_train, X_test, y_test):
    """
    Builds, compiles, and trains the LSTM model.
    """
    # Check if training data is empty
    if X_train.shape[0] == 0:
        print("错误：训练数据为空。无法训练模型。")
        return None

    # Get input shape
    n_steps, n_features = X_train.shape[1], X_train.shape[2]

    model = tf.keras.Sequential([
        # LSTM layer, input_shape=(time_steps, number of features)
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
        tf.keras.layers.Dropout(0.2), # Add Dropout to prevent overfitting
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("模型结构概要：")
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1 # Show training process
    )

    return model

# ===== Main Program =====
if __name__ == "__main__":
    # --- 1. Load and Prepare Data ---
    # Please ensure you have these CSV files, or replace with your own file paths
    # Stock CSV file needs to contain '日期' and '收盘价' columns
    # Sentiment CSV file needs to contain '日期' and 'Sentiment_Score' columns
    stock_file_path = "AAPL_daily.csv"
    sentiment_file_path = "AAPL_simulated_sentiment.csv" # Path to the simulated sentiment data
    try:
        X, y, df_full = load_and_prepare_data(stock_file_path, sentiment_file_path)

        # Only proceed with the rest of the script if data loading was successful
        # --- 2. Data Scaling ---
        # Create scaler
        scaler = MinMaxScaler()
        # Fit the scaler only on the training data, then transform all data
        # For simplicity, we transform the entire X here first, but in a strict process,
        # it should be split first then transformed.
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

        # --- 3. Create Sequence Data ---
        TIME_STEPS = 10  # Use data from the past 10 days as input
        X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEPS)

        # --- 4. Split Training and Test Sets (Time Series Method) ---
        # Ensure the training and test sets are not shuffled, maintaining time order
        split_index = int(len(X_seq) * 0.8)
        X_train, X_test = X_seq[:split_index], X_seq[split_index:]
        y_train, y_test = y_seq[:split_index], y_seq[split_index:]

        print(f"Total samples: {len(X_seq)}")
        print(f"Training set samples: {len(X_train)}")
        print(f"Test set samples: {len(X_test)}")

        # --- 5. Train Model ---
        print("\nTraining model...")
        model = build_and_train_model(X_train, y_train, X_test, y_test)

        # Only attempt to save and evaluate if the model was built successfully
        if model is not None:
            model.save('stock_predictor_v3.h5') # Save as v3 to distinguish
            print("\nModel saved as stock_predictor_v3.h5")

            # --- 6. Evaluate Model on Test Set ---
            loss, accuracy = model.evaluate(X_test, y_test)
            print(f"\nModel accuracy on test set: {accuracy*100:.2f}%")

            # --- 7. Make Prediction on New Data ---
            # Assuming we want to predict after the latest data point
            # We need to get the last `TIME_STEPS` days of data from the dataset
            # Ensure there are enough samples for the last sequence
            if len(X_scaled) >= TIME_STEPS:
                last_sequence = X_scaled.iloc[-TIME_STEPS:].values

                # Reshape it to the input format required by the model (1, time_steps, n_features)
                last_sequence_reshaped = np.reshape(last_sequence, (1, TIME_STEPS, last_sequence.shape[1]))

                # Make prediction
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")
                if prediction_proba > 0.5:
                    print("Prediction Result: Bullish")
                else:
                    print("Prediction Result: Bearish")
            else:
                print(f"\n--- Latest Data Prediction ---")
                print(f"Not enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")

    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Sentiment data loaded and merged.
Total samples: 3739
Training set samples: 2991
Test set samples: 748

Training model...
模型结构概要：


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 50)             │        11,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,101 (51.18 KB)

 Trainable params: 13,101 (51.18 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.5922 - loss: 0.6774 - val_accuracy: 0.5468 - val_loss: 0.6910
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5925 - loss: 0.6769 - val_accuracy: 0.5468 - val_loss: 0.6928
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5803 - loss: 0.6810 - val_accuracy: 0.5468 - val_loss: 0.6908
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5725 - loss: 0.6817 - val_accuracy: 0.5468 - val_loss: 0.6927
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5925 - loss: 0.6749 - val_accuracy: 0.5468 - val_loss: 0.6923
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6035 - loss: 0.6706 - val_accuracy: 0.5468 - val_loss: 0.6896
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5891 - loss: 0.6760 - val_accuracy: 0.5468 - val_loss: 0.6924
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5987 - loss: 0.6729 - val_accuracy: 0.5468 - val


Model saved as stock_predictor_v3.h5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5476 - loss: 0.6900

Model accuracy on test set: 57.09%


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step

--- Latest Data Prediction ---
Based on the last 10 days of data...
Predicted probability of price increase in the next 5 days: 43.85%
Prediction Result: Bearish


In [42]:
print("Researching potential data sources for historical financial news and event data...")

# Potential Data Sources for Historical Financial News/Event Data:

# 1. News API (e.g., News API, GDELT Project, Alpha Vantage, Finnhub, MarketStack)
print("\n1. News APIs (e.g., News API, Alpha Vantage, Finnhub, MarketStack)")
print("   - Data Type: Primarily headlines and sometimes summaries or full articles.")
print("   - Historical Coverage: Varies greatly by provider. Some offer limited historical data on free tiers, extensive archives on paid tiers. GDELT is a large archive.")
print("   - Access Method: REST APIs.")
print("   - Limitations/Costs: Free tiers often have rate limits and limited historical access. Paid tiers can be expensive. Data quality and coverage for specific stocks/timeframes need verification.")
print("   - Relevance to AAPL (2010-2024): Needs checking for each specific API's archive depth and stock coverage.")

# 2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)
print("\n2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)")
print("   - Data Type: Comprehensive, often includes structured event data (earnings calls, product launches), news sentiment scores, headlines, and articles.")
print("   - Historical Coverage: Generally extensive, often covering decades.")
print("   - Access Method: APIs, proprietary terminals, data feeds.")
print("   - Limitations/Costs: Very expensive, typically geared towards institutional users. Access often requires subscriptions.")
print("   - Relevance to AAPL (2010-2024): High likelihood of having this data, but access is a major barrier.")

# 3. Web Scraping (e.g., major financial news websites like Reuters, Bloomberg, Wall Street Journal archives)
print("\n3. Web Scraping (e.g., major financial news websites)")
print("   - Data Type: Headlines, articles.")
print("   - Historical Coverage: Depends on the website's archives. Can be extensive but often unstructured.")
print("   - Access Method: Custom scripts using libraries like Beautiful Soup, Scrapy.")
print("   - Limitations/Costs: Time-consuming to develop and maintain scrapers. Legal and ethical considerations (terms of service). Data is often unstructured, requiring significant parsing. Can be blocked.")
print("   - Relevance to AAPL (2010-2024): Possible, but labor-intensive and potentially legally restricted.")

# 4. Dedicated Historical News Archives (e.g., ProQuest, LexisNexis - often accessed via libraries)
print("\n4. Dedicated Historical News Archives (e.g., ProQuest, LexisNexis)")
print("   - Data Type: Full text of news articles from various publications.")
print("   - Historical Coverage: Can be very extensive, going back many decades.")
print("   - Access Method: Web interfaces, sometimes APIs (less common for standard access).")
print("   - Limitations/Costs: Primarily academic or institutional access. May require specific subscriptions.")
print("   - Relevance to AAPL (2010-2024): Likely to have relevant articles, but access is limited.")

# 5. Open Source Datasets (less likely for specific historical stock news, but worth a check)
print("\n5. Open Source Datasets")
print("   - Data Type: Varies, often aggregated or specialized datasets. Unlikely to find a ready-to-use, comprehensive historical AAPL news dataset.")
print("   - Historical Coverage: Sporadic and often not long-term or specific.")
print("   - Access Method: Downloads.")
print("   - Limitations/Costs: Data quality and completeness can be issues. May require significant cleaning and processing.")
print("   - Relevance to AAPL (2010-2024): Low likelihood of a direct fit.")

# Prioritization and Selection:

print("\n--- Prioritization ---")
print("Given the constraints and typical accessibility:")
print("- Financial Data Providers (Bloomberg, Refinitiv) are ideal for data quality and structure but are cost-prohibitive.")
print("- Dedicated Historical News Archives require specific access.")
print("- Web Scraping is labor-intensive and has legal/ethical concerns.")
print("- Open Source Datasets are unlikely to meet the specific requirements.")
print("- News APIs offer a more accessible route, though historical coverage and cost need careful evaluation.")

print("\n--- Selected Data Sources for Further Exploration ---")
print("Based on accessibility and potential data availability for the timeframe:")
print("1. Alpha Vantage: Offers a free tier for initial testing and may have some historical news data.")
print("2. Finnhub: Also has a free tier and real-time/historical financial data APIs, including news.")
print("3. GDELT Project: While not specifically financial news, it's a massive archive of global news that could potentially be filtered for relevant information, but requires significant data processing.")

print("\nNext steps will involve exploring the APIs of Alpha Vantage and Finnhub to check their historical news coverage for AAPL and assess the structure of the data they provide.")

Researching potential data sources for historical financial news and event data...

1. News APIs (e.g., News API, Alpha Vantage, Finnhub, MarketStack)
   - Data Type: Primarily headlines and sometimes summaries or full articles.
   - Historical Coverage: Varies greatly by provider. Some offer limited historical data on free tiers, extensive archives on paid tiers. GDELT is a large archive.
   - Access Method: REST APIs.
   - Limitations/Costs: Free tiers often have rate limits and limited historical access. Paid tiers can be expensive. Data quality and coverage for specific stocks/timeframes need verification.
   - Relevance to AAPL (2010-2024): Needs checking for each specific API's archive depth and stock coverage.

2. Financial Data Providers (e.g., Bloomberg, Refinitiv, FactSet, Quandl/Nasdaq Data Link)
   - Data Type: Comprehensive, often includes structured event data (earnings calls, product launches), news sentiment scores, headlines, and articles.
   - Historical Coverage: Gen

In [43]:
# Note: To run this code, you need an Alpha Vantage API key.
# Replace 'YOUR_ALPHA_VANTAGE_API_KEY' with your actual key.
# You can get a free API key from https://www.alphavantage.co/

ALPHA_VANTAGE_API_KEY = 'YOUR_ALPHA_VANTAGE_API_KEY'
SYMBOL = 'AAPL'
# Alpha Vantage news API typically supports querying by ticker and date range.
# Need to check the exact API endpoint and parameters.

# Based on Alpha Vantage documentation, the relevant endpoint is likely
# "https://www.alphavantage.co/query?function=NEWS_SENTIMENT"
# with parameters like 'tickers', 'time_from', 'time_to', 'limit'.
# The date format is expected to be YYYYMMDDTHHMM.

import requests
import pandas as pd

def fetch_alpha_vantage_news(api_key, symbol, start_date, end_date, limit=1000):
    """
    Fetches news sentiment data from Alpha Vantage for a given symbol and date range.
    Note: The free API might have limitations on historical data access and rate limits.
    """
    url = "https://www.alphavantage.co/query"
    # Convert dates to the required format YYYYMMDDTHHMM
    # Alpha Vantage API documentation indicates YYYYMMDDTHHMMSS format for time_from/time_to
    # Let's use start of day for time_from and end of day for time_to
    time_from = f"{start_date}T000000" # Assuming start_date is YYYYMMDD
    time_to = f"{end_date}T235959"   # Assuming end_date is YYYYMMDD

    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": symbol,
        "time_from": time_from,
        "time_to": time_to,
        "limit": limit, # Adjust limit as needed, max is 1000 per request
        "apikey": api_key
    }

    print(f"Attempting to fetch news data for {symbol} from {start_date} to {end_date} using Alpha Vantage...")

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()

        if 'feed' in data and data['feed']:
            print("Successfully fetched data from Alpha Vantage.")
            # Convert the list of articles to a pandas DataFrame
            news_df = pd.DataFrame(data['feed'])
            return news_df
        elif 'Note' in data:
            print(f"Alpha Vantage API Note: {data['Note']}")
            print("This might indicate rate limits or other API usage constraints.")
            return pd.DataFrame() # Return empty DataFrame on API notes
        else:
            print("No news data found or unexpected response format.")
            print("Response data keys:", data.keys())
            return pd.DataFrame()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Alpha Vantage: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

# Define the date range for AAPL data (2010-2024)
start_date_str = "20100101"
end_date_str = "20241231"

# Fetch news data (replace with your actual API key)
# IMPORTANT: Running this requires a valid Alpha Vantage API key.
# If you don't have one or prefer not to use it, this step will fail.
# news_data_av = fetch_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, SYMBOL, start_date_str, end_date_str)

# For demonstration purposes without a key, let's assume we fetched data and inspect a hypothetical structure.
# In a real scenario, you would uncomment the line above and process the 'news_data_av' DataFrame.

print("\n--- Assuming successful data fetch for demonstration ---")
print("A hypothetical DataFrame from Alpha Vantage might contain columns like:")
print("- 'title': Headline of the news article.")
print("- 'url': URL of the article.")
print("- 'time_published': Timestamp of publication.")
print("- 'authors': List of authors.")
print("- 'summary': Summary of the article.")
print("- 'banner_image': URL of the banner image.")
print("- 'category_within_source': Category from the source.")
print("- 'source': Source of the news.")
print("- 'topics': List of related topics.")
print("- 'overall_sentiment_score': Sentiment score for the article.")
print("- 'overall_sentiment_label': Sentiment label (e.g., 'Bullish', 'Bearish').")
print("- 'ticker_sentiment': List of sentiment scores for specific tickers mentioned.")

# The key information for sentiment analysis would be 'time_published', 'title', 'summary',
# 'overall_sentiment_score', 'overall_sentiment_label', and 'ticker_sentiment'.

print("\nFurther steps would involve:")
print("1. Processing the fetched news data (cleaning, handling missing values).")
print("2. Extracting relevant information (e.g., sentiment scores for AAPL).")
print("3. Aggregating news sentiment data on a daily basis to align with stock data.")
print("4. Joining the aggregated sentiment data with the stock technical indicator data.")

# If Alpha Vantage's free tier historical coverage is insufficient,
# the next step would be to explore Finnhub or other options.


--- Assuming successful data fetch for demonstration ---
A hypothetical DataFrame from Alpha Vantage might contain columns like:
- 'title': Headline of the news article.
- 'url': URL of the article.
- 'time_published': Timestamp of publication.
- 'authors': List of authors.
- 'summary': Summary of the article.
- 'banner_image': URL of the banner image.
- 'category_within_source': Category from the source.
- 'source': Source of the news.
- 'topics': List of related topics.
- 'overall_sentiment_score': Sentiment score for the article.
- 'overall_sentiment_label': Sentiment label (e.g., 'Bullish', 'Bearish').
- 'ticker_sentiment': List of sentiment scores for specific tickers mentioned.

Further steps would involve:
1. Processing the fetched news data (cleaning, handling missing values).
2. Extracting relevant information (e.g., sentiment scores for AAPL).
3. Aggregating news sentiment data on a daily basis to align with stock data.
4. Joining the aggregated sentiment data with the stock

In [44]:
# Note: To run this code, you need a Finnhub API key.
# Replace 'YOUR_FINNHUB_API_KEY' with your actual key.
# You can get a free API key from https://finnhub.io/

# !pip install finnhub-python # Install the library if not already installed

import finnhub

FINNHUB_API_KEY = 'YOUR_FINNHUB_API_KEY'
SYMBOL = 'AAPL'
# Finnhub news API endpoint is '/company-news' or '/company-news-stream' for real-time.
# For historical news, '/company-news' with 'from' and 'to' parameters is used.
# The date format is YYYY-MM-DD.

# Initialize Finnhub client
# finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Define the date range for AAPL data (2010-2024) in YYYY-MM-DD format
start_date_finnhub = "2010-01-01"
end_date_finnhub = "2024-12-31"

# Fetch company news from Finnhub
# Note: The free tier of Finnhub API might have limitations on historical data access and rate limits.
# Fetching a large historical range might require multiple requests or a paid plan.

print(f"\nAttempting to fetch news data for {SYMBOL} from {start_date_finnhub} to {end_date_finnhub} using Finnhub...")

try:
    # In a real scenario, you would uncomment this line:
    # news_data_fh = finnhub_client.company_news(SYMBOL, _from=start_date_finnhub, to=end_date_finnhub)

    # For demonstration purposes without a key, let's assume we fetched data and inspect a hypothetical structure.
    # Finnhub's company news endpoint returns a list of news articles.

    print("Assuming successful data fetch for demonstration.")
    print("A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:")
    print("- 'category': Category of the news.")
    print("- 'datetime': Timestamp of publication (Unix timestamp).")
    print("- 'headline': Headline of the news.")
    print("- 'id': Unique ID of the news.")
    print("- 'image': URL of the image.")
    print("- 'related': Related stocks.")
    print("- 'source': Source of the news.")
    print("- 'summary': Summary of the news.")
    print("- 'url': URL of the news article.")

    # Key information for potential sentiment analysis would be 'datetime', 'headline', 'summary'.
    # Finnhub also offers a sentiment analysis endpoint, but fetching historical sentiment
    # for individual news articles over a long period might have limitations on the free tier.

    print("\nFurther steps would involve:")
    print("1. Converting the list of dictionaries to a pandas DataFrame.")
    print("2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).")
    # Note: Finnhub's sentiment analysis endpoint might be a better source for sentiment scores directly if available historically.
    print("3. If sentiment scores are not directly available or feasible to fetch historically per article, consider performing sentiment analysis on headlines/summaries.")
    print("4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.")
    print("5. Joining the aggregated news/sentiment data with the stock technical indicator data.")

except Exception as e:
    print(f"An error occurred while trying to demonstrate Finnhub data fetching: {e}")

# If Finnhub's free tier historical coverage is insufficient or sentiment data is not easily accessible,
# other options like GDELT (requiring significant processing) or exploring paid APIs would be necessary.


Attempting to fetch news data for AAPL from 2010-01-01 to 2024-12-31 using Finnhub...
Assuming successful data fetch for demonstration.
A hypothetical list of news articles from Finnhub might contain dictionaries with keys like:
- 'category': Category of the news.
- 'datetime': Timestamp of publication (Unix timestamp).
- 'headline': Headline of the news.
- 'id': Unique ID of the news.
- 'image': URL of the image.
- 'related': Related stocks.
- 'source': Source of the news.
- 'summary': Summary of the news.
- 'url': URL of the news article.

Further steps would involve:
1. Converting the list of dictionaries to a pandas DataFrame.
2. Processing the fetched news data (cleaning, handling missing values, converting timestamp).
3. If sentiment scores are not directly available or feasible to fetch historically per article, consider performing sentiment analysis on headlines/summaries.
4. Aggregating news data (or derived sentiment) on a daily basis to align with stock data.
5. Joining the

In [45]:
import pandas as pd
import numpy as np

# Define the date range
start_date = pd.to_datetime("2010-01-01")
end_date = pd.to_datetime("2024-12-31")

# Create a date range
date_rng = pd.date_range(start=start_date, end=end_date, freq='D')

# Simulate daily sentiment scores
# Generate random sentiment scores between -1 and 1
np.random.seed(42) # for reproducibility
sentiment_scores = np.random.uniform(low=-0.5, high=0.5, size=len(date_rng))

# Introduce some seasonality or trend to make it slightly more realistic (optional)
# For example, a slight upward trend over time
sentiment_scores = sentiment_scores + np.linspace(0, 0.2, len(date_rng))

# Create a DataFrame for the simulated news sentiment data
simulated_news_df = pd.DataFrame({'日期': date_rng, 'Sentiment_Score': sentiment_scores})

# Display the first few rows and info of the simulated data
print("Simulated Historical News Sentiment Data:")
display(simulated_news_df.head())
simulated_news_df.info()

# Save the simulated data to a CSV file for later use
simulated_news_df.to_csv('AAPL_simulated_sentiment.csv', index=False)

print("\nSimulated news sentiment data saved to AAPL_simulated_sentiment.csv")

Simulated Historical News Sentiment Data:


,日期,Sentiment_Score
0,2010-01-01,-0.125460
1,2010-01-02,0.450751
2,2010-01-03,0.232067
3,2010-01-04,0.098768
4,2010-01-05,-0.343835


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5479 entries, 0 to 5478
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   日期               5479 non-null   datetime64[ns]
 1   Sentiment_Score  5479 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 85.7 KB

Simulated news sentiment data saved to AAPL_simulated_sentiment.csv


# Task
Adjust the existing stock prediction model for A-share stocks, setting the start date to 2020-01-01 and the end date to the current date. Modify the data loading to accept manual import of Tongdaoxin .csv data. Incorporate sentiment factors by obtaining and analyzing public data and comments from sources like Sina Finance and Xueqiu. Based on these changes, generate a new trading strategy.

## Define target and date range

### Subtask:
Specify the A-share stock ticker(s) to be analyzed and set the start date to 2020-01-01 and the end date to the current date.


**Reasoning**:
Define the A-share stock ticker, start date, and end date variables as specified in the instructions.



In [47]:
import datetime

# Define the A-share stock ticker (Example: Ping An Bank on Shenzhen Stock Exchange)
STOCK_TICKER = '000001.SZ'

# Define the start date
START_DATE = '2020-01-01'

# Define the end date (current date)
END_DATE = datetime.date.today().strftime('%Y-%m-%d')

print(f"Stock Ticker: {STOCK_TICKER}")
print(f"Start Date: {START_DATE}")
print(f"End Date: {END_DATE}")

Stock Ticker: 000001.SZ
Start Date: 2020-01-01
End Date: 2025-07-16


## Add support for tongdaoxin csv import

### Subtask:
Add support for tongdaoxin csv import


**Reasoning**:
I need to understand the expected format of Tongdaoxin CSV files to modify the data loading function. Since I don't have direct access to a Tongdaoxin exported file, I will describe the common characteristics of such files based on typical financial data exports and outline the necessary steps to handle them within the `load_and_prepare_data` function. This will involve assuming a likely column structure and encoding and explaining how to adapt the existing code to read and process it.



In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os

# Description of typical Tongdaoxin CSV format (based on common financial data exports):
# - Encoding: Often GBK or GB2312 for Chinese characters, but UTF-8 is also possible. Need to handle potential encoding issues.
# - Delimiter: Usually comma (',').
# - Header: Typically includes column names in Chinese.
# - Columns: May include columns like '日期', '开盘价', '最高价', '最低价', '收盘价', '成交量', '成交额', etc.
# - Date Format: Varies, but 'YYYY/MM/DD' or 'YYYY-MM-DD' are common.

# Modified load_and_prepare_data function to handle potential Tongdaoxin CSV format
def load_and_prepare_data_tongdaoxin(file_path, sentiment_file_path=None):
    """
    Loads Tongdaoxin CSV data, calculates technical indicators, integrates sentiment data,
    and creates the target variable. Handles potential encoding issues.
    """
    # Check if stock data file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    # Attempt to read with different encodings
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='gbk')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='gb2312')
        except Exception as e:
            raise IOError(f"错误：无法使用 utf-8, gbk, gb2312 编码读取文件 {file_path}: {e}")
    except Exception as e:
        raise IOError(f"错误：读取文件 {file_path} 时发生未知错误: {e}")


    # --- Map Tongdaoxin columns to expected names ---
    # Assuming standard Tongdaoxin column names. Adjust these if your file has different names.
    column_mapping = {
        '日期': '日期',
        '收盘价': '收盘价',
        '开盘价': '开盘价',
        '最高价': '最高价',
        '最低价': '最低价',
        '成交量': '成交量',
        '成交额': '成交额'
        # Add other columns if needed and available in your Tongdaoxin file
    }

    # Rename columns based on the mapping
    df.rename(columns=column_mapping, inplace=True)

    # Check if essential columns exist after renaming
    required_cols = ['日期', '收盘价']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"错误：Tongdaoxin 文件缺少必需的列: {missing}。请检查列名或更新 column_mapping。")


    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data (Optional) ---
    # This part remains similar, assuming sentiment file format is consistent
    sentiment_df = pd.DataFrame()
    if sentiment_file_path and os.path.exists(sentiment_file_path):
        try:
            sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
            sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
            sentiment_df = sentiment_df.set_index('日期')
            # Merge stock data with sentiment data
            df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
            print("Sentiment data loaded and merged.")
        except FileNotFoundError:
             print(f"警告：找不到情感文件 '{sentiment_file_path}'。将不包含情感分数。")
             # If sentiment file not found, add a column of NaNs to avoid errors later
             df['Sentiment_Score'] = np.nan
        except Exception as e:
             print(f"警告：加载或合并情感文件 '{sentiment_file_path}' 时发生错误: {e}。将不包含情感分数。")
             df['Sentiment_Score'] = np.nan # Add NaN column if loading/merging fails


    # --- Calculate Technical Indicators (Manual Calculation) ---
    # Ensure '收盘价' column is numeric before calculations
    df['收盘价'] = pd.to_numeric(df['收盘价'], errors='coerce')

    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands and Sentiment
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower']
    if 'Sentiment_Score' in df.columns:
         feature_cols.append('Sentiment_Score') # Add sentiment only if loaded

    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA15'] = df['MA5'] / df['MA15'] # Changed from MA10
    df['价格/BBMID'] = df['收盘价'] / df['BBMID'] # New feature based on BB
    df['价格/BBUpper'] = df['收盘价'] / df['BBUpper'] # New feature based on BB
    df['价格/BBLower'] = df['收盘价'] / df['BBLower'] # New feature based on BB


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator, target variable calculations, and merging
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# The rest of the script (create_sequences, build_and_train_model, main execution block)
# will use this new load_and_prepare_data_tongdaoxin function.
# The main execution block will need to be updated to call this function.
# Since the subtask is only to modify the data loading function, I will stop here.

## Incorporate sentiment analysis from online sources

### Subtask:
Incorporate sentiment analysis from online sources


**Reasoning**:
To identify specific sources for sentiment data on Sina Finance and Xueqiu, I need to rely on general knowledge of these platforms and their typical data availability, as direct automated fetching or detailed exploration within this environment is limited by access restrictions and terms of service. I will describe the types of data available and how they could potentially be used for sentiment analysis, acknowledging the limitations of public web scraping.



In [49]:
# Identifying specific sources on Sina Finance and Xueqiu for sentiment data.

print("Identifying potential online sources for sentiment data on A-share stocks:")

print("\n--- Sina Finance (新浪财经) ---")
print("Website: https://finance.sina.com.cn/")
print("Potential Data Types:")
print("- News Articles: General financial news, company-specific news.")
print("- Stock Forums/Comment Sections: User comments and discussions related to specific stocks or market topics.")
print("- Research Reports: Some reports might contain sentiment analysis or qualitative assessments.")
print("How to potentially use for sentiment:")
print("- News Headline/Article Text Analysis: Apply Natural Language Processing (NLP) techniques (e.g., sentiment lexicons, machine learning models) to news content.")
print("- User Comment Analysis: Analyze the text of comments in forums to gauge public sentiment towards a stock.")
print("Limitations:")
print("- Accessing historical news archives for specific stocks programmatically might be restricted or require payment.")
print("- Scraping user comments raises ethical and legal concerns and is often against website terms of service.")
print("- Data is unstructured, requiring significant processing.")

print("\n--- Xueqiu (雪球) ---")
print("Website: https://xueqiu.com/")
print("Potential Data Types:")
print("- User Posts/Comments: Investment diaries, analysis, opinions, and discussions by individual investors and analysts.")
print("- Stock Pages: Aggregated information, including news feeds and discussion threads for specific stocks.")
print("- Data APIs (Limited/Paid): Xueqiu offers some data APIs, but comprehensive historical sentiment data might not be freely available.")
print("How to potentially use for sentiment:")
print("- User Post/Comment Analysis: Xueqiu is rich with individual investor sentiment expressed in posts and comments. NLP techniques can be applied to this text data.")
print("- Aggregated Sentiment (if available): Some platforms might provide aggregated sentiment indicators derived from user activity.")
print("Limitations:")
print("- Similar to Sina Finance, automated scraping of user-generated content is generally against terms of service and raises ethical concerns.")
print("- Historical data access through official APIs might be limited or require a paid subscription.")
print("- Data quality and reliability of user-generated content can vary greatly.")

print("\n--- Other Potential Public Sources ---")
print("- Eastmoney (东方财富): Similar to Sina Finance and Xueqiu, a major financial portal with news, forums, and data.")
print("- Hexun (和讯网): Another financial news and data website.")
print("- Juchao Information (巨潮资讯网): Official disclosure platform for listed companies, provides official announcements and reports (less for public sentiment, more for fundamental events).")

print("\n--- Conclusion on Feasibility ---")
print("While these platforms contain valuable sentiment data, programmatically accessing comprehensive historical data (2010-2024) for sentiment analysis via public scraping is challenging and ethically/legally problematic.")
print("Potential approaches, acknowledging limitations:")
print("1. Focus on publicly available APIs if they offer relevant historical data (evaluate free tier limits).")
print("2. Utilize existing datasets of financial news or sentiment if available through legal means.")
print("3. Simulate sentiment data based on assumptions or limited available data, as done in the previous steps.")
print("4. For a real-world application, consider using paid financial data providers with explicit data usage agreements.")

print("\nGiven the constraints of this environment and the task, directly fetching and processing comprehensive historical sentiment data from these sources via scraping is not feasible. The simulation approach used previously or integrating pre-existing sentiment datasets would be more practical within these limitations.")

Identifying potential online sources for sentiment data on A-share stocks:

--- Sina Finance (新浪财经) ---
Website: https://finance.sina.com.cn/
Potential Data Types:
- News Articles: General financial news, company-specific news.
- Stock Forums/Comment Sections: User comments and discussions related to specific stocks or market topics.
- Research Reports: Some reports might contain sentiment analysis or qualitative assessments.
How to potentially use for sentiment:
- News Headline/Article Text Analysis: Apply Natural Language Processing (NLP) techniques (e.g., sentiment lexicons, machine learning models) to news content.
- User Comment Analysis: Analyze the text of comments in forums to gauge public sentiment towards a stock.
Limitations:
- Accessing historical news archives for specific stocks programmatically might be restricted or require payment.
- Scraping user comments raises ethical and legal concerns and is often against website terms of service.
- Data is unstructured, requiring

**Reasoning**:
I have identified potential sources and concluded that direct historical scraping is not feasible due to limitations. The subtask is to incorporate sentiment analysis from online sources. Given the constraints, the most practical approach is to either use a pre-existing sentiment dataset or simulate sentiment data, as was done in a previous subtask. Since a simulated sentiment dataset ('AAPL_simulated_sentiment.csv') has already been created and the `load_and_prepare_data_tongdaoxin` function has been modified to accept a sentiment file path and merge sentiment data, the next step is to ensure this simulated sentiment data is used in the main execution block when calling the data loading function. This will effectively incorporate sentiment factors into the model training process within the limitations of the environment. I will modify the main execution block to call `load_and_prepare_data_tongdaoxin` with the path to the simulated sentiment file and then run the entire script.



In [78]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
# import talib # Commented out the talib import
# import pandas_ta as ta # Import pandas_ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import datetime # Import datetime to get the current date
import yfinance as yf # Import yfinance for fetching data (for demonstration)

# Description of typical Tongdaoxin CSV format (based on common financial data exports):
# - Encoding: Often GBK or GB2312 for Chinese characters, but UTF-8 is also possible. Need to handle potential encoding issues.
# - Delimiter: Usually comma (',').
# - Header: Typically includes column names in Chinese.
# - Columns: May include columns like '日期', '开盘价', '最高价', '最低价', '收盘价', '成交量', '成交额', etc.
# - Date Format: Varies, but 'YYYY/MM/DD' or 'YYYY-MM-DD' are common.

# Modified load_and_prepare_data function to handle potential Tongdaoxin CSV format OR DataFrame input
def load_and_prepare_data_modified(stock_data_df=None, file_path=None, sentiment_file_path=None):
    """
    Loads stock data from a DataFrame or CSV, calculates technical indicators,
    integrates sentiment data, and creates the target variable.
    Accepts stock data as a DataFrame directly or from a CSV file path.
    """
    if stock_data_df is not None and not stock_data_df.empty:
        df = stock_data_df.copy() # Work on a copy
        print("Using provided DataFrame for stock data.")
    elif file_path and os.path.exists(file_path):
        print(f"Loading stock data from file: {file_path}")
        # Attempt to read with different encodings if loading from file
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='gbk')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='gb2312')
            except Exception as e:
                raise IOError(f"错误：无法使用 utf-8, gbk, gb2312 编码读取文件 {file_path}: {e}")
        except Exception as e:
            raise IOError(f"错误：读取文件 {file_path} 时发生未知错误: {e}")

        # --- Map Tongdaoxin columns to expected names if loading from file ---
        column_mapping = {
            '日期': '日期',
            '收盘价': '收盘价',
            '开盘价': '开盘价',
            '最高价': '最高价',
            '最低价': '最低价',
            '成交量': '成交量',
            '成交额': '成交额'
            # Add other columns if needed and available in your Tongdaoxin file
        }
        df.rename(columns=column_mapping, inplace=True)

        # Check if essential columns exist after renaming
        required_cols = ['日期', '收盘价']
        if not all(col in df.columns for col in required_cols):
            missing = [col for col for col in required_cols if col not in df.columns]
            raise ValueError(f"错误：股票数据文件缺少必需的列: {missing}。请检查列名或更新 column_mapping。")

    else:
        raise FileNotFoundError("错误：未提供股票数据 DataFrame 或有效的股票数据文件路径。")


    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data (Optional) ---
    # This part remains similar, assuming sentiment file format is consistent
    sentiment_df = pd.DataFrame()
    if sentiment_file_path and os.path.exists(sentiment_file_path):
        try:
            sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
            sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
            sentiment_df = sentiment_df.set_index('日期')
            # Merge stock data with sentiment data
            df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
            print("Sentiment data loaded and merged.")
        except FileNotFoundError:
             print(f"警告：找不到情感文件 '{sentiment_file_path}'。将不包含情感分数。")
             # If sentiment file not found, add a column of NaNs to avoid errors later
             df['Sentiment_Score'] = np.nan
        except Exception as e:
             print(f"警告：加载或合并情感文件 '{sentiment_file_path}' 时发生错误: {e}。将不包含情感分数。")
             df['Sentiment_Score'] = np.nan # Add NaN column if loading/merging fails
    else:
        print("警告：未提供情感文件路径或文件不存在。将不包含情感分数。")
        df['Sentiment_Score'] = np.nan # Add NaN column if no sentiment file is provided or found


    # --- Calculate Technical Indicators (Manual Calculation) ---
    # Ensure '收盘价' column is numeric before calculations
    df['收盘价'] = pd.to_numeric(df['收盘价'], errors='coerce')

    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands and Sentiment
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower']
    if 'Sentiment_Score' in df.columns:
         feature_cols.append('Sentiment_Score') # Add sentiment only if loaded

    # Avoid division by zero for feature creation by adding a small epsilon or handling NaNs later
    df['价格/MA5'] = df['收盘价'] / (df['MA5'] + 1e-9)
    df['MA5/MA15'] = df['MA5'] / (df['MA15'] + 1e-9)
    df['价格/BBMID'] = df['收盘价'] / (df['BBMID'] + 1e-9)
    df['价格/BBUpper'] = df['收盘价'] / (df['BBUpper'] + 1e-9)
    df['价格/BBLower'] = df['收盘价'] / (df['BBLower'] + 1e-9)


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator, target variable calculations, and merging
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# Reuse the create_sequences function from cell 94f65f08
if 'create_sequences' not in globals():
    def create_sequences(X, y, time_steps=10):
         print("Placeholder create_sequences function called.")
         return np.array([]), np.array([])

# Reuse the build_and_train_model function from cell 94f65f08
# Note: For predicting a pool, you might train one model per stock or a single model on pooled data.
# For simplicity here, we assume we will train and evaluate for each stock individually.
if 'build_and_train_model' not in globals():
    def build_and_train_model(X_train, y_train, X_test, y_test):
        print("Placeholder build_and_train_model function called.")
        return None


# --- Configuration for Stock Pool Processing ---
# Define the A-share stock ticker (Example: Ping An Bank on Shenzhen Stock Exchange)
# STOCK_TICKER = '000001.SZ' # This is now part of STOCK_POOL

# Define the start date
START_DATE = '2020-01-01'

# Define the end date (current date)
END_DATE = datetime.date.today().strftime('%Y-%m-%d')

# Create a list of A-share stock tickers for the prediction pool
STOCK_POOL = ['000001.SZ', '600036.SH', '601398.SH', '000300.SH'] # Defined in cell 677360f2

# Path to the simulated sentiment data file (same file used for all stocks for simplicity)
# In a real scenario, you might have separate sentiment files per stock or a combined one.
sentiment_file_path_ashare = 'ASHare_simulated_sentiment.csv' # Defined in cell 7c14aaef


# --- Main Program (Modified for Stock Pool) ---
if __name__ == "__main__":
    processed_data = {} # Dictionary to store processed data for each stock

    print("\n--- Processing Data for Each Stock in the Pool ---")

    for ticker in STOCK_POOL:
        print(f"\n--- Processing Data for {ticker} ---")

        # --- 1. Load Historical Stock Data ---
        # If you have Tongdaoxin files, you would load them here and pass the DataFrame.
        # Example for Tongdaoxin file:
        # tongdaoxin_file = f"{ticker}_daily.csv" # Assuming files are named like '000001.SZ_daily.csv'
        # if os.path.exists(tongdaoxin_file):
        #     # Load from Tongdaoxin file using the modified function
        #     try:
        #         X, y, df_full = load_and_prepare_data_modified(
        #             file_path=tongdaoxin_file,
        #             sentiment_file_path=sentiment_file_path_ashare
        #         )
        #         processed_data[ticker] = {'X': X, 'y': y, 'df_full': df_full}
        #         print(f"Successfully processed data for {ticker} from Tongdaoxin file.")
        #     except (FileNotFoundError, ValueError, IOError) as e:
        #         print(f"Error processing Tongdaoxin file for {ticker}: {e}")
        #     continue # Move to the next ticker if processed from file

        # If no Tongdaoxin file, attempt to fetch using yfinance (for demonstration/fallback)
        # Note: yfinance uses different tickers for A-shares (e.g., 600036.SH -> 600036.SS)
        # and coverage/accuracy for older A-share data might be inconsistent.
        # This is for demonstration when Tongdaoxin file is not available.

        try:
            # yfinance uses different tickers for A-shares.
            # Need to map A-share codes to yfinance format (e.g., 000001.SZ -> 000001.SZ, 600036.SH -> 600036.SS)
            # Note: yfinance coverage for older A-share data might be inconsistent.
            if ticker.endswith('.SZ'):
                yfinance_ticker = ticker # yfinance sometimes uses .SZ directly
            elif ticker.endswith('.SH'):
                yfinance_ticker = ticker.replace('.SH', '.SS') # .SH maps to .SS in yfinance
            else:
                yfinance_ticker = ticker # Assume it might work as is or is a US stock


            print(f"Fetching data for {ticker} (yfinance ticker: {yfinance_ticker}) from {START_DATE} to {END_DATE}...")
            stock_data = yf.download(yfinance_ticker, start=START_DATE, end=END_DATE)


            if stock_data.empty:
                print(f"Warning: No data fetched for {ticker} using yfinance.")
                continue # Skip to the next ticker if no data


            # Select the 'Close' price and reset the index to make 'Date' a column
            # Handle potential MultiIndex ('Close', ticker) or single index ('Close')
            if isinstance(stock_data.columns, pd.MultiIndex):
                 # If MultiIndex, select the column with level 0 'Close'
                 if 'Close' in stock_data.columns.get_level_values(0):
                      # Select 'Close' and then the ticker-specific column under 'Close'
                      close_price_series = stock_data['Close'][stock_data['Close'].columns[0]]
                      close_price_df = close_price_series.reset_index()
                 else:
                      print(f"Warning: 'Close' column not found in fetched data for {ticker}.")
                      continue # Skip to the next ticker
            elif 'Close' in stock_data.columns:
                close_price_df = stock_data['Close'].reset_index()
            else:
                print(f"Warning: 'Close' column not found in fetched data for {ticker}.")
                continue # Skip to the next ticker


            # Rename the columns to '日期' and '收盘价'
            # After reset_index(), the columns are typically 'Date' and the ticker symbol itself
            # or 'Close' if it wasn't a MultiIndex initially.
            # Let's check the column names after reset_index to be safe.
            current_cols = close_price_df.columns.tolist()
            rename_map = {'Date': '日期'}
            # Find the column that contains the closing price value
            price_col = [col for col in current_cols if col != 'Date']
            if price_col:
                 rename_map[price_col[0]] = '收盘价'
            else:
                 print(f"Warning: Could not identify price column in fetched data for {ticker}.")
                 continue # Skip to the next ticker


            close_price_df = close_price_df.rename(columns=rename_map)

            # Ensure only '日期' and '收盘价' are included before passing to processing
            if '日期' in close_price_df.columns and '收盘价' in close_price_df.columns:
                 raw_stock_data_df = close_price_df[['日期', '收盘价']]
            else:
                 print(f"Warning: Renamed columns '日期' or '收盘价' not found in processed data for {ticker}.")
                 continue # Skip to the next ticker


            # --- 2. Prepare Data and Calculate Indicators (using the modified function) ---
            try:
                X, y, df_full = load_and_prepare_data_modified(
                    stock_data_df=raw_stock_data_df,
                    sentiment_file_path=sentiment_file_path_ashare
                )

                if not X.empty:
                    processed_data[ticker] = {
                        'X': X,
                        'y': y,
                        'df_full': df_full
                    }
                    print(f"Successfully processed data for {ticker}.")
                    print(f"Processed data shape for {ticker}: Features {X.shape}, Target {y.shape}")
                else:
                    print(f"Skipping processing for {ticker} due to insufficient data after preparation.")

            except (FileNotFoundError, ValueError, IOError) as e:
                print(f"Error preparing data for {ticker}: {e}")
            except Exception as e:
                 print(f"An unexpected error occurred during data preparation for {ticker}: {e}")

        except Exception as e:
            print(f"An error occurred while fetching data for {ticker} with yfinance: {e}")
            continue # Skip to the next ticker on fetch error


    print("\n--- Data Processing Complete ---")
    print(f"Processed data available for {list(processed_data.keys())}")

    # The next steps (scaling, sequence creation, model training, prediction)
    # would typically be done for each stock individually or with a pooled approach,
    # depending on the strategy.
    # This subtask is complete with loading and preparing data for each stock.

SyntaxError: invalid syntax (ipython-input-78-672853076.py, line 61)

In [51]:
import datetime

# Define the A-share stock ticker (Example: Ping An Bank on Shenzhen Stock Exchange)
STOCK_TICKER = '000001.SZ'

# Define the start date
START_DATE = '2020-01-01'

# Define the end date (current date)
END_DATE = datetime.date.today().strftime('%Y-%m-%d')

print(f"Stock Ticker: {STOCK_TICKER}")
print(f"Start Date: {START_DATE}")
print(f"End Date: {END_DATE}")

Stock Ticker: 000001.SZ
Start Date: 2020-01-01
End Date: 2025-07-16


## Add support for Tongdaoxin CSV import

### Subtask:
Add support for tongdaoxin csv import

**Reasoning**:
I need to understand the expected format of Tongdaoxin CSV files to modify the data loading function. Since I don't have direct access to a Tongdaoxin exported file, I will describe the common characteristics of such files based on typical financial data exports and outline the necessary steps to handle them within the `load_and_prepare_data` function. This will involve assuming a likely column structure and encoding and explaining how to adapt the existing code to read and process it.

In [55]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import datetime # Import datetime to get the current date

# Description of typical Tongdaoxin CSV format (based on common financial data exports):
# - Encoding: Often GBK or GB2312 for Chinese characters, but UTF-8 is also possible. Need to handle potential encoding issues.
# - Delimiter: Usually comma (',').
# - Header: Typically includes column names in Chinese.
# - Columns: May include columns like '日期', '开盘价', '最高价', '最低价', '收盘价', '成交量', '成交额', etc.
# - Date Format: Varies, but 'YYYY/MM/DD' or 'YYYY-MM-DD' are common.

# Modified load_and_prepare_data function to handle potential Tongdaoxin CSV format
def load_and_prepare_data_tongdaoxin(file_path, sentiment_file_path=None):
    """
    Loads Tongdaoxin CSV data, calculates technical indicators, integrates sentiment data,
    and creates the target variable. Handles potential encoding issues.
    """
    # Check if stock data file exists before attempting to read
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"错误：找不到 '{file_path}'。请确保文件在正确的路径下。")

    # Attempt to read with different encodings
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='gbk')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='gb2312')
        except Exception as e:
            raise IOError(f"错误：无法使用 utf-8, gbk, gb2312 编码读取文件 {file_path}: {e}")
    except Exception as e:
        raise IOError(f"错误：读取文件 {file_path} 时发生未知错误: {e}")


    # --- Map Tongdaoxin columns to expected names ---
    # Assuming standard Tongdaoxin column names. Adjust these if your file has different names.
    column_mapping = {
        '日期': '日期',
        '收盘价': '收盘价',
        '开盘价': '开盘价',
        '最高价': '最高价',
        '最低价': '最低价',
        '成交量': '成交量',
        '成交额': '成交额'
        # Add other columns if needed and available in your Tongdaoxin file
    }

    # Rename columns based on the mapping
    df.rename(columns=column_mapping, inplace=True)

    # Check if essential columns exist after renaming
    required_cols = ['日期', '收盘价']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"错误：Tongdaoxin 文件缺少必需的列: {missing}。请检查列名或更新 column_mapping。")


    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data (Optional) ---
    # This part remains similar, assuming sentiment file format is consistent
    sentiment_df = pd.DataFrame()
    if sentiment_file_path and os.path.exists(sentiment_file_path):
        try:
            sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
            sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
            sentiment_df = sentiment_df.set_index('日期')
            # Merge stock data with sentiment data
            df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
            print("Sentiment data loaded and merged.")
        except FileNotFoundError:
             print(f"警告：找不到情感文件 '{sentiment_file_path}'。将不包含情感分数。")
             # If sentiment file not found, add a column of NaNs to avoid errors later
             df['Sentiment_Score'] = np.nan
        except Exception as e:
             print(f"警告：加载或合并情感文件 '{sentiment_file_path}' 时发生错误: {e}。将不包含情感分数。")
             df['Sentiment_Score'] = np.nan # Add NaN column if loading/merging fails
    else:
        print("警告：未提供情感文件路径或文件不存在。将不包含情感分数。")
        df['Sentiment_Score'] = np.nan # Add NaN column if no sentiment file is provided or found


    # --- Calculate Technical Indicators (Manual Calculation) ---
    # Ensure '收盘价' column is numeric before calculations
    df['收盘价'] = pd.to_numeric(df['收盘价'], errors='coerce')

    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    # Update features to use MA15 instead of MA10 and include Bollinger Bands and Sentiment
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower']
    if 'Sentiment_Score' in df.columns:
         feature_cols.append('Sentiment_Score') # Add sentiment only if loaded

    df['价格/MA5'] = df['收盘价'] / df['MA5']
    df['MA5/MA15'] = df['MA5'] / df['MA15'] # Changed from MA10
    df['价格/BBMID'] = df['收盘价'] / df['BBMID'] # New feature based on BB
    df['价格/BBUpper'] = df['收盘价'] / df['BBUpper'] # New feature based on BB
    df['价格/BBLower'] = df['收盘价'] / df['BBLower'] # New feature based on BB


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    # shift(-5) will move the data from the 5th day in the future to the current row
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    # Drop all rows containing NaN values introduced by indicator, target variable calculations, and merging
    df = df.dropna()

    # Define feature columns and target column after dropping NaNs
    # Ensure all feature columns actually exist in the DataFrame after dropna
    feature_cols = [col for col in feature_cols if col in df.columns]

    target_col = '目标'

    X = df[feature_cols]
    y = df[target_col]

    return X, y, df

# The rest of the script (create_sequences, build_and_train_model, main execution block)
# will use this new load_and_prepare_data_tongdaoxin function.
# The main execution block will need to be updated to call this function.
# Since the subtask is only to modify the data loading function, I will stop here.

## Incorporate sentiment analysis from online sources

### Subtask:
Incorporate sentiment analysis from online sources

**Reasoning**:
I need to identify specific sources for sentiment data on Sina Finance and Xueqiu. Since direct automated fetching or detailed exploration within this environment is limited, I will rely on general knowledge of these platforms and describe the types of data available and how they could potentially be used for sentiment analysis, acknowledging the limitations of public web scraping.

In [53]:
# Identifying specific sources on Sina Finance and Xueqiu for sentiment data.

print("Identifying potential online sources for sentiment data on A-share stocks:")

print("\n--- Sina Finance (新浪财经) ---")
print("Website: https://finance.sina.com.cn/")
print("Potential Data Types:")
print("- News Articles: General financial news, company-specific news.")
print("- Stock Forums/Comment Sections: User comments and discussions related to specific stocks or market topics.")
print("- Research Reports: Some reports might contain sentiment analysis or qualitative assessments.")
print("How to potentially use for sentiment:")
print("- News Headline/Article Text Analysis: Apply Natural Language Processing (NLP) techniques (e.g., sentiment lexicons, machine learning models) to news content.")
print("- User Comment Analysis: Analyze the text of comments in forums to gauge public sentiment towards a stock.")
print("Limitations:")
print("- Accessing historical news archives for specific stocks programmatically might be restricted or require payment.")
print("- Scraping user comments raises ethical and legal concerns and is often against website terms of service.")
print("- Data is unstructured, requiring significant processing.")

print("\n--- Xueqiu (雪球) ---")
print("Website: https://xueqiu.com/")
print("Potential Data Types:")
print("- User Posts/Comments: Investment diaries, analysis, opinions, and discussions by individual investors and analysts.")
print("- Stock Pages: Aggregated information, including news feeds and discussion threads for specific stocks.")
print("- Data APIs (Limited/Paid): Xueqiu offers some data APIs, but comprehensive historical sentiment data might not be freely available.")
print("How to potentially use for sentiment:")
print("- User Post/Comment Analysis: Xueqiu is rich with individual investor sentiment expressed in posts and comments. NLP techniques can be applied to this text data.")
print("- Aggregated Sentiment (if available): Some platforms might provide aggregated sentiment indicators derived from user activity.")
print("Limitations:")
print("- Similar to Sina Finance, automated scraping of user-generated content is generally against terms of service and raises ethical concerns.")
print("- Historical data access through official APIs might be limited or require a paid subscription.")
print("- Data quality and reliability of user-generated content can vary greatly.")

print("\n--- Other Potential Public Sources ---")
print("- Eastmoney (东方财富): Similar to Sina Finance and Xueqiu, a major financial portal with news, forums, and data.")
print("- Hexun (和讯网): Another financial news and data website.")
print("- Juchao Information (巨潮资讯网): Official disclosure platform for listed companies, provides official announcements and reports (less for public sentiment, more for fundamental events).")

print("\n--- Conclusion on Feasibility ---")
print("While these platforms contain valuable sentiment data, programmatically accessing comprehensive historical data (2010-2024) for sentiment analysis via public scraping is challenging and ethically/legally problematic.")
print("Potential approaches, acknowledging limitations:")
print("1. Focus on publicly available APIs if they offer relevant historical data (evaluate free tier limits).")
print("2. Utilize existing datasets of financial news or sentiment if available through legal means.")
print("3. Simulate sentiment data based on assumptions or limited available data, as done in the previous steps.")
print("4. For a real-world application, consider using paid financial data providers with explicit data usage agreements.")

print("\nGiven the constraints of this environment and the task, directly fetching and processing comprehensive historical sentiment data from these sources via scraping is not feasible. The simulation approach used previously or integrating pre-existing sentiment datasets would be more practical within these limitations.")

Identifying potential online sources for sentiment data on A-share stocks:

--- Sina Finance (新浪财经) ---
Website: https://finance.sina.com.cn/
Potential Data Types:
- News Articles: General financial news, company-specific news.
- Stock Forums/Comment Sections: User comments and discussions related to specific stocks or market topics.
- Research Reports: Some reports might contain sentiment analysis or qualitative assessments.
How to potentially use for sentiment:
- News Headline/Article Text Analysis: Apply Natural Language Processing (NLP) techniques (e.g., sentiment lexicons, machine learning models) to news content.
- User Comment Analysis: Analyze the text of comments in forums to gauge public sentiment towards a stock.
Limitations:
- Accessing historical news archives for specific stocks programmatically might be restricted or require payment.
- Scraping user comments raises ethical and legal concerns and is often against website terms of service.
- Data is unstructured, requiring

In [72]:
import pandas as pd
import numpy as np

# Define the date range
start_date = pd.to_datetime("2020-01-01") # Start date specified in the main task
end_date = pd.to_datetime(datetime.date.today().strftime('%Y-%m-%d')) # End date specified in the main task

# Create a date range
date_rng = pd.date_range(start=start_date, end=end_date, freq='D')

# Simulate daily sentiment scores
# Generate random sentiment scores between -1 and 1
np.random.seed(42) # for reproducibility
sentiment_scores = np.random.uniform(low=-0.5, high=0.5, size=len(date_rng))

# Introduce some seasonality or trend to make it slightly more realistic (optional)
# For example, a slight upward trend over time
sentiment_scores = sentiment_scores + np.linspace(0, 0.2, len(date_rng))

# Create a DataFrame for the simulated news sentiment data
simulated_sentiment_df_ashare = pd.DataFrame({'日期': date_rng, 'Sentiment_Score': sentiment_scores})

# Display the first few rows and info of the simulated data
print("Simulated Historical News Sentiment Data for A-share:")
display(simulated_sentiment_df_ashare.head())
simulated_sentiment_df_ashare.info()

# Save the simulated data to a CSV file for later use
sentiment_file_path_ashare = 'ASHare_simulated_sentiment.csv'
simulated_sentiment_df_ashare.to_csv(sentiment_file_path_ashare, index=False)

print(f"\nSimulated news sentiment data saved to {sentiment_file_path_ashare}")

Simulated Historical News Sentiment Data for A-share:


,日期,Sentiment_Score
0,2020-01-01,-0.125460
1,2020-01-02,0.450813
2,2020-01-03,0.232192
3,2020-01-04,0.098955
4,2020-01-05,-0.343586


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   日期               2024 non-null   datetime64[ns]
 1   Sentiment_Score  2024 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 31.8 KB

Simulated news sentiment data saved to ASHare_simulated_sentiment.csv


## New A-share Stock Prediction Strategy

Based on the modified model incorporating technical indicators and sentiment factors for A-share stocks, here is a potential new prediction strategy:

**Strategy Name:** A-share Sentiment-Enhanced Technical LSTM Strategy

**Objective:** To predict the short-term (next 5 trading days) direction of the target A-share stock price (up or down) using a combination of technical analysis and market sentiment.

**Target Market:** A-share stocks (specifically the ticker defined, e.g., `000001.SZ`).

**Data Sources:**
1.  **Historical Stock Data:** Daily historical data for the target A-share stock, preferably imported manually from Tongdaoxin or other reliable sources, including '日期' (Date) and '收盘价' (Closing Price), along with other relevant price and volume data if available.
2.  **Sentiment Data:** Daily aggregated sentiment scores or indicators derived from public online sources like Sina Finance and Xueqiu, aligned with the stock data by date. (In this implementation, simulated sentiment data is used as a placeholder).

**Technical Indicators Used:**
*   5-day Moving Average (MA5)
*   15-day Moving Average (MA15)
*   Relative Strength Index (RSI)
*   Moving Average Convergence Divergence (MACD)
*   Bollinger Bands (Middle, Upper, Lower Bands)

**Sentiment Factor Used:**
*   Daily Sentiment Score (derived from analysis of online news and comments, or simulated).

**Prediction Model:**
*   A Sequential LSTM (Long Short-Term Memory) neural network model.
*   Input: Sequences of the last 10 days of scaled technical indicators and the daily sentiment score.
*   Output: A probability score (between 0 and 1) representing the likelihood of the stock's closing price increasing in the next 5 trading days.

**Strategy Rules (Example):**

This is a simplified example of how the model's output could be used within a trading strategy. Real-world strategies would need more sophisticated rules and risk management.

*   **Entry Signal (Potential Buy):** If the model's predicted probability of a price increase in the next 5 days is above a certain threshold (e.g., > 0.6), generate a potential buy signal.
*   **Exit Signal (Potential Sell/Hold):**
    *   If the model's predicted probability of a price increase falls below a certain threshold (e.g., < 0.5), or predicts a decrease, generate a potential sell signal.
    *   Alternatively, exit rules could be based on time (e.g., hold for 5 days), profit targets, or stop-loss limits, regardless of the model's subsequent predictions within that 5-day window.
*   **Risk Management:** Implement stop-loss orders to limit potential losses if the price moves against the prediction.
*   **Position Sizing:** Determine the amount of capital to allocate to each trade based on risk tolerance and confidence in the prediction.

**Flow of the Strategy:**

1.  Obtain the latest historical stock data (including '日期' and '收盘价') and the corresponding daily sentiment data.
2.  Load and prepare the data using the `load_and_prepare_data_tongdaoxin` function, which calculates technical indicators and merges sentiment data.
3.  Scale the combined technical and sentiment features.
4.  Create time sequences of the scaled features for the last `TIME_STEPS` (10) days.
5.  Input the latest sequence into the trained LSTM model to get a prediction probability.
6.  Based on the predicted probability and the defined strategy rules, generate a trading signal (e.g., Buy, Sell, Hold).
7.  Execute trades according to the generated signal and risk management rules.

**Evaluation:**

The effectiveness of this strategy would need to be rigorously backtested on historical A-share data (using the actual date range 2020-01-01 to current) and potentially evaluated through forward testing in a simulated trading environment. Key metrics would include profitability, win rate, drawdowns, etc.

**Further Enhancements:**

*   Refine sentiment analysis methods (e.g., using more advanced NLP models, incorporating different types of news/comments).
*   Explore additional technical indicators or different lookback periods.
*   Optimize LSTM model architecture and hyperparameters.
*   Incorporate other types of data (e.g., fundamental data, macroeconomic indicators).
*   Develop more sophisticated trading rules and risk management techniques.

This strategy provides a framework for using both technical and sentiment factors within a deep learning model for A-share stock prediction. Its actual performance would depend heavily on the quality of the data, the effectiveness of the sentiment analysis, and the robustness of the trading rules.

**Reasoning**:
I have researched data sources and outlined the process for using Alpha Vantage. The next step is to explore Finnhub as an alternative or supplementary source for historical news data, as it was also identified as a promising option. This involves installing the Finnhub library, obtaining an API key, and using its functions to fetch news data for AAPL within the specified timeframe. Similar to the Alpha Vantage step, I will demonstrate the code structure and outline the expected data format and subsequent processing steps, noting the need for an API key.

**Reasoning**:
The previous command failed because the `finnhub` library was not installed, as indicated by the `ModuleNotFoundError`. To proceed with exploring Finnhub as a data source, I need to install the `finnhub-python` package using pip. After installation, I will re-attempt to import the library and outline the process for fetching data, as planned in the previous step.

## Fetch sentiment data

### Subtask:
Fetch historical news or event data for AAPL covering the period from 2010 to 2024 using one of the identified sources that provides accessible historical data, or simulate the fetching process with a placeholder if actual fetching is not feasible.

**Reasoning**:
Since fetching real historical news data for the entire period from 2010 to 2024 is challenging with free API tiers and without valid keys, I will simulate fetching news sentiment data for AAPL over the specified date range. This simulated data will include a 'Date' and 'Sentiment Score' column to represent the aggregated daily sentiment, which can then be integrated into the stock prediction model. This approach allows the task to proceed without relying on external APIs or large datasets that are difficult to access.

**Reasoning**:
I have successfully simulated and saved the news sentiment data. The next step is to integrate this simulated sentiment data into the data loading and preparation function (`load_and_prepare_data`) in cell `94f65f08`. This involves reading the simulated sentiment data, merging it with the stock data based on the '日期' column, and ensuring the merged data is used for feature creation and model training. I will modify the existing `load_and_prepare_data` function to include this merging step.

In [70]:
# Fetch AAPL data using yfinance and save as A_DAY_daily.csv for demonstration

import yfinance as yf
import pandas as pd

# Define start and end dates (using the dates defined in the notebook)
import datetime
START_DATE = '2020-01-01' # Defined in cell 3764c46c
END_DATE = datetime.date.today().strftime('%Y-%m-%d') # Defined in cell 3764c46c

try:
    # Fetch historical daily data for Apple (AAPL) using yfinance
    # We'll use the date range defined for the A-share task (2020-01-01 to current)
    stock_data = yf.download("AAPL", start=START_DATE, end=END_DATE)

    # Select the 'Close' price and reset the index to make 'Date' a column
    temp_df = stock_data['Close'].reset_index()

    # Create a new DataFrame with the desired column names and data
    # Note: The column name from yfinance after reset_index() will be 'AAPL' for the close price
    final_df_to_save = pd.DataFrame({
        '日期': temp_df['Date'],
        '收盘价': temp_df['AAPL']
    })

    # Save the DataFrame to a CSV file named 'A_DAY_daily.csv' without including the index
    # Explicitly set header=True to ensure the column names are written as the first row
    final_df_to_save.to_csv('A_DAY_daily.csv', index=False, header=True)

    print("AAPL data fetched using yfinance and saved to A_DAY_daily.csv for demonstration.")

    # Optional: Verify the saved CSV
    # df_check = pd.read_csv('A_DAY_daily.csv')
    # print("\nVerified A_DAY_daily.csv head:")
    # display(df_check.head())
    # df_check.info()


except Exception as e:
    print(f"An error occurred while fetching and saving data with yfinance: {e}")

/tmp/ipython-input-70-992289756.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download("AAPL", start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed

AAPL data fetched using yfinance and saved to A_DAY_daily.csv for demonstration.


In [73]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import datetime

# Reuse the data loading and preparation function from cell 94f65f08
# Ensure the function is defined or available in the environment before running this cell
# For clarity, I'm including a placeholder definition here, but the actual function
# from cell 94f65f08 will be used if that cell has been executed.
if 'load_and_prepare_data_tongdaoxin' not in globals():
    def load_and_prepare_data_tongdaoxin(file_path, sentiment_file_path=None):
        print("Placeholder load_and_prepare_data_tongdaoxin function called.")
        # This is a placeholder. The actual function from cell 94f65f08 should be used.
        # You would replace this with the actual logic to load, calculate indicators,
        # merge sentiment, and prepare data.
        # For this example, we'll return empty DataFrames to avoid errors.
        return pd.DataFrame(), pd.Series(), pd.DataFrame()

# Reuse the create_sequences function from cell 94f65f08
if 'create_sequences' not in globals():
    def create_sequences(X, y, time_steps=10):
         print("Placeholder create_sequences function called.")
         return np.array([]), np.array([])

# --- Configuration ---
STOCK_DATA_FILE = "A_DAY_daily.csv"  # The stock data file
SENTIMENT_DATA_FILE = "ASHare_simulated_sentiment.csv" # The simulated sentiment data file
MODEL_PATH = 'stock_predictor_v4.h5' # Path to the trained model
TIME_STEPS = 10 # Number of lookback days for sequences
BUY_THRESHOLD = 0.6 # Probability threshold for generating a buy signal
SELL_THRESHOLD = 0.4 # Probability threshold for generating a sell signal


print("--- Executing Complete Strategy ---")

# --- 1. Load and Prepare Data ---
try:
    # Use the data loading function that handles Tongdaoxin format and sentiment
    X, y, df_full = load_and_prepare_data_tongdaoxin(STOCK_DATA_FILE, SENTIMENT_DATA_FILE)
    print(f"\nData loaded and prepared. Shape of features (X): {X.shape}, Shape of target (y): {y.shape}")

except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure the data files are available.")
    exit() # Exit if data files are not found
except ValueError as e:
    print(f"Error processing data: {e}")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during data loading: {e}")
    exit()


# Only proceed if data loading was successful and resulted in non-empty DataFrames
if not X.empty:
    # --- 2. Data Scaling ---
    # Create scaler and fit on the features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    print("\nFeatures scaled.")

    # --- 3. Create Sequence Data for Prediction ---
    # We need the last TIME_STEPS days of data for the latest prediction
    if len(X_scaled) >= TIME_STEPS:
        last_sequence_scaled = X_scaled.iloc[-TIME_STEPS:].values

        # Reshape it to the input format required by the model (1, time_steps, n_features)
        last_sequence_reshaped = np.reshape(last_sequence_scaled, (1, TIME_STEPS, last_sequence_scaled.shape[1]))
        print(f"\nCreated sequence for prediction. Shape: {last_sequence_reshaped.shape}")

        # --- 4. Load Trained Model ---
        if os.path.exists(MODEL_PATH):
            try:
                model = tf.keras.models.load_model(MODEL_PATH)
                print(f"\nTrained model loaded from {MODEL_PATH}")
            except Exception as e:
                print(f"Error loading model from {MODEL_PATH}: {e}")
                model = None # Set model to None if loading fails
        else:
            print(f"\nError: Trained model not found at {MODEL_PATH}.")
            print("Please train the model first by running the main program cell (e.g., cell 94f65f08).")
            model = None # Set model to None if file not found

        # --- 5. Make Prediction and Generate Signal ---
        if model is not None:
            try:
                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print("\n--- Latest Data Prediction ---")
                print(f"Based on the last {TIME_STEPS} days of data (including sentiment)...")
                print(f"Predicted probability of price increase in the next 5 days: {prediction_proba*100:.2f}%")

                # --- Generate Trading Signal based on Prediction ---
                print("\n--- Trading Signal ---")
                if prediction_proba >= BUY_THRESHOLD:
                    signal = "BUY"
                    print(f"Signal: {signal} (Probability >= {BUY_THRESHOLD})")
                elif prediction_proba <= SELL_THRESHOLD:
                    signal = "SELL"
                    print(f"Signal: {signal} (Probability <= {SELL_THRESHOLD})")
                else:
                    signal = "HOLD/NEUTRAL"
                    print(f"Signal: {signal} ({SELL_THRESHOLD} < Probability < {BUY_THRESHOLD})")

            except Exception as e:
                print(f"An error occurred during prediction: {e}")
                print("Could not generate trading signal.")

        else:
            print("\nCould not make a prediction as the model was not loaded.")


    else:
        print(f"\nNot enough data ({len(X_scaled)} days) to create a sequence of length {TIME_STEPS} for prediction.")
        print("Cannot make a prediction or generate a signal.")

else:
    print("\nData loading and preparation did not result in valid data for processing.")
    print("Cannot proceed with prediction or signal generation.")

--- Executing Complete Strategy ---
Sentiment data loaded and merged.

Data loaded and prepared. Shape of features (X): (1366, 8), Shape of target (y): (1366,)

Features scaled.

Created sequence for prediction. Shape: (1, 10, 8)



Trained model loaded from stock_predictor_v4.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step

--- Latest Data Prediction ---
Based on the last 10 days of data (including sentiment)...
Predicted probability of price increase in the next 5 days: 62.81%

--- Trading Signal ---
Signal: BUY (Probability >= 0.6)


# A股情绪增强技术 LSTM 策略

基于结合了技术指标和情绪因子的修改后的 A 股股票预测模型，以下是一个潜在的新的预测策略：

**策略名称：** A股情绪增强技术 LSTM 策略

**目标：** 利用技术分析和市场情绪的结合，预测目标 A 股股票价格的短期（未来 5 个交易日）方向（上涨或下跌）。

**目标市场：** A 股股票（特别是已定义的股票代码，例如 `000001.SZ`）。

**数据来源：**
1.  **历史股票数据：** 目标 A 股股票的每日历史数据，最好手动从通达信或其他可靠来源导入，包括“日期”和“收盘价”，以及可用的其他相关价格和交易量数据。
2.  **情绪数据：** 从新浪财经和雪球等公共在线来源获取并按日期与股票数据对齐的每日汇总情绪得分或指标。（在此实现中，模拟情绪数据用作占位符）。

**使用的技术指标：**
*   5 日移动平均线 (MA5)
*   15 日移动平均线 (MA15)
*   相对强弱指数 (RSI)
*   移动平均收敛扩散指标 (MACD)
*   布林带（中轨、上轨、下轨）

**使用的情绪因子：**
*   每日情绪得分（来自在线新闻和评论分析，或模拟）。

**预测模型：**
*   序列式长短期记忆 (LSTM) 神经网络模型。
*   输入：过去 10 天的缩放技术指标和每日情绪得分序列。
*   输出：一个概率得分（介于 0 到 1 之间），表示股票收盘价在未来 5 天上涨的可能性。

**策略规则（示例）：**

这是一个如何将模型输出用于交易策略的简化示例。实际策略需要更复杂的规则和风险管理。

*   **入场信号（潜在买入）：** 如果模型预测未来 5 天价格上涨的概率高于某个阈值（例如 > 0.6），则生成潜在买入信号。
*   **出场信号（潜在卖出/持有）：**
    *   如果模型预测价格上涨的概率低于某个阈值（例如 < 0.5），或预测下跌，则生成潜在卖出信号。
    *   或者，出场规则可以基于时间（例如，持有 5 天）、止盈目标或止损限价，而不管模型在该 5 天窗口期内的后续预测如何。
*   **风险管理：** 执行止损单，以限制价格与预测相反时的潜在损失。
*   **头寸规模：** 根据风险承受能力和对预测的信心，确定分配给每笔交易的资金量。

**策略流程：**

1.  获取最新的历史股票数据（包括“日期”和“收盘价”）以及相应的每日情绪数据。
2.  使用 `load_and_prepare_data_tongdaoxin` 函数加载和准备数据，该函数计算技术指标并合并情绪数据。
3.  缩放合并后的技术和情绪特征。
4.  为过去 `TIME_STEPS` (10) 天的缩放特征创建时间序列。
5.  将最新的序列输入到训练好的 LSTM 模型中，以获取预测概率。
6.  根据预测概率和定义的策略规则，生成交易信号（例如，买入、卖出、持有）。
7.  根据生成的信号和风险管理规则执行交易。

**评估：**

该策略的有效性需要在历史 A 股数据上进行严格的回测（使用实际日期范围 2020 年 1 月 1 日至今），并可能通过在模拟交易环境中进行前向测试来评估。关键指标将包括盈利能力、胜率、回撤等。

**进一步增强：**

*   优化情绪分析方法（例如，使用更高级的自然语言处理模型，整合不同类型的新闻/评论）。
*   探索其他技术指标或不同的回看周期。
*   优化 LSTM 模型架构和超参数。
*   整合其他类型的数据（例如，基本面数据、宏观经济指标）。
*   开发更复杂的交易规则和风险管理技术。

该策略提供了一个框架，用于在深度学习模型中使用技术和情绪因子进行 A 股股票预测。其实际表现将严重依赖于数据质量、情绪分析的有效性以及交易规则的稳健性。

# Task
Generate a complete stock prediction strategy for a selected pool of A-share stocks, starting from 2020-01-01 to the current date. The strategy should incorporate technical indicators (including Bollinger Bands and MA15), sentiment analysis from public data sources like Sina Finance and Xueqiu, and allow for manual import of Tongdaxin .csv data. The output should include the calculated predictions for the stock pool, a Chinese explanation of the model strategy, and the ability to save the results as a .csv or .tsx file. The model should be designed for daily training and testing and allow for saving the current model and explanation.

## Define the stock pool

### Subtask:
Create a list or collection of the A-share stock tickers you want to include in the prediction pool.


**Reasoning**:
Create a Python list named `STOCK_POOL` and populate it with example A-share stock tickers.



In [74]:
# Create a list of A-share stock tickers for the prediction pool
STOCK_POOL = ['000001.SZ', '600036.SH', '601398.SH', '000300.SH']

print("Stock Pool:")
print(STOCK_POOL)

Stock Pool:
['000001.SZ', '600036.SH', '601398.SH', '000300.SH']


## Manage data for multiple stocks

### Subtask:
Modify the data loading process to handle fetching or loading historical data for each stock in the pool. This might involve iterating through the stock tickers and loading data from separate files (e.g., `STOCK1_daily.csv`, `STOCK2_daily.csv`) or fetching data for multiple tickers using a data library. Ensure the data for each stock is processed independently through the data preparation and feature engineering steps (technical indicators, sentiment integration).


**Reasoning**:
Define the `load_stock_data` function using `yfinance` and modify the `load_and_prepare_data_tongdaoxin` function to accept a DataFrame. Then, update the main execution block to iterate through the `STOCK_POOL`, fetch data for each stock, and process it using the modified loading function, storing the results in dictionaries.



In [75]:
import yfinance as yf
import datetime
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define the A-share stock ticker (Example: Ping An Bank on Shenzhen Stock Exchange)
# STOCK_TICKER = '000001.SZ' # This is now part of STOCK_POOL

# Define the start date
START_DATE = '2020-01-01'

# Define the end date (current date)
END_DATE = datetime.date.today().strftime('%Y-%m-%d')

# Create a list of A-share stock tickers for the prediction pool
STOCK_POOL = ['000001.SZ', '600036.SH', '601398.SH', '000300.SH'] # Defined in cell 677360f2

# Function to fetch historical daily stock data using yfinance
def load_stock_data(ticker, start_date, end_date):
    """
    Fetches historical daily data for a given stock ticker using yfinance.
    Returns a DataFrame with '日期' and '收盘价' columns.
    """
    try:
        # yfinance uses different tickers for A-shares.
        # Need to map A-share codes to yfinance format (e.g., 000001.SZ -> 000001.SZ, 600036.SH -> 600036.SS)
        # Note: yfinance coverage for older A-share data might be inconsistent.
        if ticker.endswith('.SZ'):
            yfinance_ticker = ticker # yfinance sometimes uses .SZ directly
        elif ticker.endswith('.SH'):
            yfinance_ticker = ticker.replace('.SH', '.SS') # .SH maps to .SS in yfinance
        else:
            yfinance_ticker = ticker # Assume it might work as is or is a US stock

        print(f"Fetching data for {ticker} (yfinance ticker: {yfinance_ticker}) from {start_date} to {end_date}...")
        stock_data = yf.download(yfinance_ticker, start=start_date, end=end_date)

        if stock_data.empty:
            print(f"Warning: No data fetched for {ticker} using yfinance.")
            return pd.DataFrame()

        # Select the 'Close' price and reset the index to make 'Date' a column
        # Handle potential MultiIndex ('Close', ticker) or single index ('Close')
        if isinstance(stock_data.columns, pd.MultiIndex):
             # If MultiIndex, select the column with level 0 'Close'
             if 'Close' in stock_data.columns.get_level_values(0):
                  close_price_df = stock_data['Close'].reset_index()
             else:
                  print(f"Warning: 'Close' column not found in fetched data for {ticker}.")
                  return pd.DataFrame()
        elif 'Close' in stock_data.columns:
            close_price_df = stock_data['Close'].reset_index()
        else:
            print(f"Warning: 'Close' column not found in fetched data for {ticker}.")
            return pd.DataFrame()


        # Rename the columns to '日期' and '收盘价'
        # After reset_index(), the columns are typically 'Date' and the ticker symbol itself
        # or 'Close' if it wasn't a MultiIndex initially.
        # Let's check the column names after reset_index to be safe.
        current_cols = close_price_df.columns.tolist()
        rename_map = {'Date': '日期'}
        # Find the column that contains the closing price value
        price_col = [col for col in current_cols if col != 'Date']
        if price_col:
             rename_map[price_col[0]] = '收盘价'
        else:
             print(f"Warning: Could not identify price column in fetched data for {ticker}.")
             return pd.DataFrame()


        close_price_df = close_price_df.rename(columns=rename_map)

        # Ensure only '日期' and '收盘价' are returned
        if '日期' in close_price_df.columns and '收盘价' in close_price_df.columns:
             return close_price_df[['日期', '收盘价']]
        else:
             print(f"Warning: Renamed columns '日期' or '收盘价' not found in processed data for {ticker}.")
             return pd.DataFrame()


    except Exception as e:
        print(f"An error occurred while fetching data for {ticker} with yfinance: {e}")
        return pd.DataFrame()


# Modified load_and_prepare_data function to handle DataFrame input
def load_and_prepare_data_modified(stock_data_df=None, file_path=None, sentiment_file_path=None):
    """
    Loads stock data from a DataFrame or CSV, calculates technical indicators,
    integrates sentiment data, and creates the target variable.
    Accepts stock data as a DataFrame directly.
    """
    if stock_data_df is not None and not stock_data_df.empty:
        df = stock_data_df.copy() # Work on a copy
        print("Using provided DataFrame for stock data.")
    elif file_path and os.path.exists(file_path):
        print(f"Loading stock data from file: {file_path}")
        # Attempt to read with different encodings if loading from file
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='gbk')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='gb2312')
            except Exception as e:
                raise IOError(f"错误：无法使用 utf-8, gbk, gb2312 编码读取文件 {file_path}: {e}")
        except Exception as e:
            raise IOError(f"错误：读取文件 {file_path} 时发生未知错误: {e}")

        # --- Map Tongdaoxin columns to expected names if loading from file ---
        column_mapping = {
            '日期': '日期',
            '收盘价': '收盘价',
            '开盘价': '开盘价',
            '最高价': '最高价',
            '最低价': '最低价',
            '成交量': '成交量',
            '成交额': '成交额'
        }
        df.rename(columns=column_mapping, inplace=True)

        # Check if essential columns exist after renaming
        required_cols = ['日期', '收盘价']
        if not all(col in df.columns for col in required_cols):
            missing = [col for col in required_cols if col not in df.columns]
            raise ValueError(f"错误：股票数据文件缺少必需的列: {missing}。请检查列名或更新 column_mapping。")

    else:
        raise FileNotFoundError("错误：未提供股票数据 DataFrame 或有效的股票数据文件路径。")


    df['日期'] = pd.to_datetime(df['日期'])
    df = df.set_index('日期')

    # --- Integrate Sentiment Data (Optional) ---
    sentiment_df = pd.DataFrame()
    if sentiment_file_path and os.path.exists(sentiment_file_path):
        try:
            sentiment_df = pd.read_csv(sentiment_file_path, encoding='utf-8')
            sentiment_df['日期'] = pd.to_datetime(sentiment_df['日期'])
            sentiment_df = sentiment_df.set_index('日期')
            # Merge stock data with sentiment data
            df = df.merge(sentiment_df[['Sentiment_Score']], left_index=True, right_index=True, how='left')
            print("Sentiment data loaded and merged.")
        except FileNotFoundError:
             print(f"警告：找不到情感文件 '{sentiment_file_path}'。将不包含情感分数。")
             df['Sentiment_Score'] = np.nan # Add NaN column if file not found
        except Exception as e:
             print(f"警告：加载或合并情感文件 '{sentiment_file_path}' 时发生错误: {e}。将不包含情感分数。")
             df['Sentiment_Score'] = np.nan # Add NaN column if loading/merging fails
    else:
        print("警告：未提供情感文件路径或文件不存在。将不包含情感分数。")
        df['Sentiment_Score'] = np.nan # Add NaN column if no sentiment file is provided or found

    # --- Calculate Technical Indicators (Manual Calculation) ---
    # Ensure '收盘价' column is numeric before calculations
    df['收盘价'] = pd.to_numeric(df['收盘价'], errors='coerce')

    # Drop rows where '收盘价' could not be converted to numeric (e.g., had non-numeric values)
    df.dropna(subset=['收盘价'], inplace=True)

    # 1. Calculate 5-day Simple Moving Average ('MA5')
    df['MA5'] = df['收盘价'].rolling(window=5).mean()

    # 2. Calculate 15-day Simple Moving Average ('MA15')
    df['MA15'] = df['收盘价'].rolling(window=15).mean()

    # 3. Calculate Relative Strength Index ('RSI') with a period of 14
    delta = df['收盘价'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=14-1, min_periods=14).mean()
    avg_loss = loss.ewm(com=14-1, min_periods=14).mean()

    # Avoid division by zero for RS calculation
    rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))


    # 4. Calculate Moving Average Convergence Divergence ('MACD') with default periods (fast=12, slow=26, signal=9)
    exp1 = df['收盘价'].ewm(span=12, adjust=False).mean()
    exp2 = df['收盘价'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

    # 5. Calculate Bollinger Bands (20-day period, 2 standard deviations)
    window_bb = 20
    df['BBMID'] = df['收盘价'].rolling(window=window_bb).mean()
    df['BBSTD'] = df['收盘价'].rolling(window=window_bb).std()
    df['BBUpper'] = df['BBMID'] + (df['BBSTD'] * 2)
    df['BBLower'] = df['BBMID'] - (df['BBSTD'] * 2)
    # Drop the temporary standard deviation column
    df = df.drop(columns=['BBSTD'])


    # --- Create Features ---
    feature_cols = ['价格/MA5', 'MA5/MA15', 'RSI', 'MACD', '价格/BBMID', '价格/BBUpper', '价格/BBLower']
    if 'Sentiment_Score' in df.columns:
         feature_cols.append('Sentiment_Score') # Add sentiment only if loaded

    # Avoid division by zero for feature creation by adding a small epsilon or handling NaNs later
    df['价格/MA5'] = df['收盘价'] / (df['MA5'] + 1e-9)
    df['MA5/MA15'] = df['MA5'] / (df['MA15'] + 1e-9)
    df['价格/BBMID'] = df['收盘价'] / (df['BBMID'] + 1e-9)
    df['价格/BBUpper'] = df['收盘价'] / (df['BBUpper'] + 1e-9)
    df['价格/BBLower'] = df['收盘价'] / (df['BBLower'] + 1e-9)


    # --- Create Target Variable: Whether the price will increase after 5 days ---
    df['未来价格'] = df['收盘价'].shift(-5)
    df['目标'] = (df['未来价格'] > df['收盘价']).astype(int)

    # --- Clean Data ---
    df = df.dropna() # Drop all rows containing NaN values


    # Define feature columns and target column after dropping NaNs
    feature_cols = [col for col in feature_cols if col in df.columns] # Ensure columns exist

    target_col = '目标'

    # Check if there's enough data after dropping NaNs and for sequence creation
    if len(df) < 30: # Arbitrary minimum length, adjust as needed
         print(f"Warning: Not enough data remaining after cleaning for analysis. Only {len(df)} rows.")
         return pd.DataFrame(), pd.Series(), pd.DataFrame()


    X = df[feature_cols]
    y = df[target_col]

    return X, y, df


# --- Main Program (Modified for Stock Pool) ---
if __name__ == "__main__":
    processed_data = {} # Dictionary to store processed data for each stock

    # Path to the simulated sentiment data file (same file used for all stocks for simplicity)
    sentiment_file_path_ashare = 'ASHare_simulated_sentiment.csv'

    for ticker in STOCK_POOL:
        print(f"\n--- Processing Data for {ticker} ---")

        # 1. Load Historical Stock Data (using yfinance)
        # If you have Tongdaoxin files, you would load them here instead,
        # and pass the DataFrame to load_and_prepare_data_modified
        # For example:
        # tongdaoxin_file = f"{ticker}.csv" # Assuming files are named like '000001.SZ.csv'
        # raw_stock_data_df = pd.read_csv(tongdaoxin_file, encoding='gbk') # Adjust encoding as needed

        raw_stock_data_df = load_stock_data(ticker, START_DATE, END_DATE)


        if not raw_stock_data_df.empty:
            # 2. Prepare Data and Calculate Indicators (using the modified function)
            try:
                X, y, df_full = load_and_prepare_data_modified(
                    stock_data_df=raw_stock_data_df,
                    sentiment_file_path=sentiment_file_path_ashare
                )

                if not X.empty:
                    processed_data[ticker] = {
                        'X': X,
                        'y': y,
                        'df_full': df_full
                    }
                    print(f"Successfully processed data for {ticker}.")
                    print(f"Processed data shape for {ticker}: Features {X.shape}, Target {y.shape}")
                else:
                    print(f"Skipping processing for {ticker} due to insufficient data after preparation.")

            except (FileNotFoundError, ValueError, IOError) as e:
                print(f"Error preparing data for {ticker}: {e}")
            except Exception as e:
                 print(f"An unexpected error occurred during data preparation for {ticker}: {e}")

        else:
            print(f"Skipping processing for {ticker} as raw stock data could not be loaded.")

    # Now, processed_data dictionary contains X, y, df_full for each stock
    # You can access them like:
    # if '000001.SZ' in processed_data:
    #     X_000001 = processed_data['000001.SZ']['X']
    #     y_000001 = processed_data['000001.SZ']['y']
    #     df_full_000001 = processed_data['000001.SZ']['df_full']

    print("\n--- Data Processing Complete ---")
    print(f"Processed data available for {list(processed_data.keys())}")

    # The next steps (scaling, sequence creation, model training, prediction)
    # would typically be done for each stock individually or with a pooled approach,
    # depending on the strategy.
    # This subtask is complete with loading and preparing data for each stock.



--- Processing Data for 000001.SZ ---
Fetching data for 000001.SZ (yfinance ticker: 000001.SZ) from 2020-01-01 to 2025-07-16...


/tmp/ipython-input-75-2361043564.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(yfinance_ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-75-2361043564.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(yfinance_ticker, start=start_date, end=end_date)


Using provided DataFrame for stock data.
Sentiment data loaded and merged.
Successfully processed data for 000001.SZ.
Processed data shape for 000001.SZ: Features (1316, 8), Target (1316,)

--- Processing Data for 600036.SH ---
Fetching data for 600036.SH (yfinance ticker: 600036.SS) from 2020-01-01 to 2025-07-16...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-75-2361043564.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(yfinance_ticker, start=start_date, end=end_date)


Using provided DataFrame for stock data.
Sentiment data loaded and merged.
Successfully processed data for 600036.SH.
Processed data shape for 600036.SH: Features (1316, 8), Target (1316,)

--- Processing Data for 601398.SH ---
Fetching data for 601398.SH (yfinance ticker: 601398.SS) from 2020-01-01 to 2025-07-16...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-75-2361043564.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(yfinance_ticker, start=start_date, end=end_date)


Using provided DataFrame for stock data.
Sentiment data loaded and merged.
Successfully processed data for 601398.SH.
Processed data shape for 601398.SH: Features (1316, 8), Target (1316,)

--- Processing Data for 000300.SH ---
Fetching data for 000300.SH (yfinance ticker: 000300.SS) from 2020-01-01 to 2025-07-16...


[*********************100%***********************]  1 of 1 completed

Using provided DataFrame for stock data.
Sentiment data loaded and merged.
Successfully processed data for 000300.SH.
Processed data shape for 000300.SH: Features (1030, 8), Target (1030,)

--- Data Processing Complete ---
Processed data available for ['000001.SZ', '600036.SH', '601398.SH', '000300.SH']


**Reasoning**:
The previous subtask of loading and preparing data for each stock in the pool is complete. The processed data for each stock is stored in the `processed_data` dictionary. The next logical step according to the overall task description is to scale the data, create sequences, split into training and testing sets, train a model, save the model, evaluate it, and make a prediction for each stock in the pool. I will group these steps together for each stock and store the trained models and evaluation results.



In [76]:
# Reuse functions create_sequences and build_and_train_model from previous cells
# Ensure these functions are defined or available in the environment before running this cell.
# For clarity, I'm including placeholder definitions here, but the actual functions
# from cell 94f65f08 should be used if that cell has been executed.
if 'create_sequences' not in globals():
    def create_sequences(X, y, time_steps=10):
         print("Placeholder create_sequences function called.")
         return np.array([]), np.array([])

if 'build_and_train_model' not in globals():
    def build_and_train_model(X_train, y_train, X_test, y_test):
        print("Placeholder build_and_train_model function called.")
        return None


# --- Configuration for Model Training and Prediction ---
TIME_STEPS = 10 # Number of lookback days for sequences
MODEL_DIR = 'stock_models' # Directory to save trained models
BUY_THRESHOLD = 0.6 # Probability threshold for generating a buy signal
SELL_THRESHOLD = 0.4 # Probability threshold for generating a sell signal

# Ensure the model directory exists
os.makedirs(MODEL_DIR, exist_ok=True)

trained_models = {} # Dictionary to store trained models
evaluation_results = {} # Dictionary to store evaluation results
latest_predictions = {} # Dictionary to store latest predictions and signals


print("\n--- Model Training and Prediction for Each Stock ---")

for ticker, data in processed_data.items():
    print(f"\n--- Processing Model for {ticker} ---")

    X = data['X']
    y = data['y']
    df_full = data['df_full'] # Keep df_full for potential later use if needed

    # Check if there's enough data for sequence creation and splitting
    if len(X) >= TIME_STEPS + 5: # Need enough data for sequences and future target
        # --- 1. Data Scaling ---
        # Create scaler and fit on the features
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index) # Keep as DataFrame for slicing

        print(f"Features scaled for {ticker}. Scaled data shape: {X_scaled_df.shape}")

        # --- 2. Create Sequence Data ---
        X_seq, y_seq = create_sequences(X_scaled_df, y, TIME_STEPS)
        print(f"Created sequences for {ticker}. X_seq shape: {X_seq.shape}, y_seq shape: {y_seq.shape}")

        # --- 3. Split Training and Test Sets (Time Series Method) ---
        # Ensure the split index leaves enough samples for the test set
        if len(X_seq) > int(len(X_seq) * 0.8) + 1: # Check if test set will have at least 2 samples after split
            split_index = int(len(X_seq) * 0.8)
            X_train, X_test = X_seq[:split_index], X_seq[split_index:]
            y_train, y_test = y_seq[:split_index], y_seq[split_index:]

            print(f"Split data for {ticker}: Training samples {len(X_train)}, Test samples {len(X_test)}")

            # --- 4. Build and Train Model ---
            print(f"\nTraining model for {ticker}...")
            model = build_and_train_model(X_train, y_train, X_test, y_test)

            # Only proceed if the model was trained successfully
            if model is not None:
                # --- 5. Save Trained Model ---
                model_filename = os.path.join(MODEL_DIR, f'{ticker}_stock_predictor_v4.h5')
                model.save(model_filename)
                print(f"\nModel saved for {ticker} as {model_filename}")
                trained_models[ticker] = model # Store the trained model

                # --- 6. Evaluate Model on Test Set ---
                print(f"\nEvaluating model for {ticker} on test set...")
                loss, accuracy = model.evaluate(X_test, y_test)
                print(f"Model accuracy on test set for {ticker}: {accuracy*100:.2f}%")
                evaluation_results[ticker] = {'loss': loss, 'accuracy': accuracy} # Store evaluation results

                # --- 7. Make Latest Prediction and Generate Signal ---
                print(f"\nMaking latest prediction for {ticker}...")
                # Need the last TIME_STEPS days of scaled data for the latest prediction
                last_sequence_scaled = X_scaled_df.iloc[-TIME_STEPS:].values
                last_sequence_reshaped = np.reshape(last_sequence_scaled, (1, TIME_STEPS, last_sequence_scaled.shape[1]))

                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print(f"Predicted probability of price increase in the next 5 days for {ticker}: {prediction_proba*100:.2f}%")

                # Generate Trading Signal
                if prediction_proba >= BUY_THRESHOLD:
                    signal = "BUY"
                elif prediction_proba <= SELL_THRESHOLD:
                    signal = "SELL"
                else:
                    signal = "HOLD/NEUTRAL"

                print(f"Trading Signal for {ticker}: {signal} (Probability: {prediction_proba:.2f}, Thresholds: Buy>={BUY_THRESHOLD}, Sell<={SELL_THRESHOLD})")
                latest_predictions[ticker] = {'probability': prediction_proba, 'signal': signal} # Store prediction and signal

            else:
                print(f"Skipping evaluation and prediction for {ticker} as model training failed.")

        else:
             print(f"Skipping training and prediction for {ticker} due to insufficient data for splitting into train/test sets.")

    else:
        print(f"Skipping processing for {ticker} due to insufficient data ({len(X)} rows) for sequence creation (requires at least {TIME_STEPS + 5} rows).")


print("\n--- Model Training and Prediction Complete for all Stocks ---")

# You can now access the results:
# trained_models: dictionary of trained Keras models
# evaluation_results: dictionary of test set loss and accuracy for each stock
# latest_predictions: dictionary of latest prediction probability and trading signal for each stock

print("\nSummary of Latest Predictions and Signals:")
for ticker, result in latest_predictions.items():
    print(f"{ticker}: Probability = {result['probability']:.2f}, Signal = {result['signal']}")



--- Model Training and Prediction for Each Stock ---

--- Processing Model for 000001.SZ ---
Features scaled for 000001.SZ. Scaled data shape: (1316, 8)
Created sequences for 000001.SZ. X_seq shape: (1306, 10, 8), y_seq shape: (1306,)
Split data for 000001.SZ: Training samples 1044, Test samples 262

Training model for 000001.SZ...
模型结构概要：


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 50)             │        11,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,101 (51.18 KB)

...

 Total params: 13,101 (51.18 KB)

 Trainable params: 13,101 (51.18 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.4899 - loss: 0.6931 - val_accuracy: 0.4167 - val_loss: 0.7130
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5519 - loss: 0.6858 - val_accuracy: 0.4167 - val_loss: 0.7262
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5556 - loss: 0.6869 - val_accuracy: 0.4167 - val_loss: 0.7194
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5421 - loss: 0.6889 - val_accuracy: 0.4167 - val_loss: 0.7113
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5565 - loss: 0.6857 - val_accuracy: 0.4167 - val_loss: 0.7187
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5899 - loss: 0.6775 - val_accuracy: 0.4167 - val_loss: 0.7201
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5675 - loss: 0.6813 - val_accuracy: 0.4167 - val_loss: 0.7114
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5404 - loss: 0.6876 - val_accuracy: 0.4167 - v


Model saved for 000300.SH as stock_models/000300.SH_stock_predictor_v4.h5

Evaluating model for 000300.SH on test set...
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5263 - loss: 0.7345 
Model accuracy on test set for 000300.SH: 50.98%

Making latest prediction for 000300.SH...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
Predicted probability of price increase in the next 5 days for 000300.SH: 54.52%
Trading Signal for 000300.SH: HOLD/NEUTRAL (Probability: 0.55, Thresholds: Buy>=0.6, Sell<=0.4)

--- Model Training and Prediction Complete for all Stocks ---

Summary of Latest Predictions and Signals:
000001.SZ: Probability = 0.44, Signal = HOLD/NEUTRAL
600036.SH: Probability = 0.45, Signal = HOLD/NEUTRAL
601398.SH: Probability = 0.29, Signal = SELL
000300.SH: Probability = 0.55, Signal = HOLD/NEUTRAL


In [77]:
# Create a list of A-share stock tickers for the prediction pool
STOCK_POOL = ['000001.SZ', '600036.SH', '601398.SH', '000300.SH'] # Example tickers

print("Stock Pool:")
print(STOCK_POOL)

Stock Pool:
['000001.SZ', '600036.SH', '601398.SH', '000300.SH']


## Generate predictions for each stock in the pool and consolidate results

### Subtask:
Generate predictions for each stock in the pool and consolidate results

**Reasoning**:
The previous subtask of loading and preparing data for each stock in the pool is complete. The processed data for each stock is stored in the `processed_data` dictionary. The next logical step according to the overall task description is to scale the data, create sequences, split into training and testing sets, train a model, save the model, evaluate it, and make a prediction for each stock in the pool. I will group these steps together for each stock and store the trained models and evaluation results.

In [79]:
# Reuse functions create_sequences and build_and_train_model from previous cells
# Ensure these functions are defined or available in the environment before running this cell.
# For clarity, I'm including placeholder definitions here, but the actual functions
# from cell 94f65f08 should be used if that cell has been executed.
if 'create_sequences' not in globals():
    def create_sequences(X, y, time_steps=10):
         print("Placeholder create_sequences function called.")
         return np.array([]), np.array([])

if 'build_and_train_model' not in globals():
    def build_and_train_model(X_train, y_train, X_test, y_test):
        print("Placeholder build_and_train_model function called.")
        return None


# --- Configuration for Model Training and Prediction ---
TIME_STEPS = 10 # Number of lookback days for sequences
MODEL_DIR = 'stock_models' # Directory to save trained models
BUY_THRESHOLD = 0.6 # Probability threshold for generating a buy signal
SELL_THRESHOLD = 0.4 # Probability threshold for generating a sell signal

# Ensure the model directory exists
os.makedirs(MODEL_DIR, exist_ok=True)

trained_models = {} # Dictionary to store trained models
evaluation_results = {} # Dictionary to store evaluation results
latest_predictions = {} # Dictionary to store latest predictions and signals


print("\n--- Model Training and Prediction for Each Stock ---")

for ticker, data in processed_data.items():
    print(f"\n--- Processing Model for {ticker} ---")

    X = data['X']
    y = data['y']
    df_full = data['df_full'] # Keep df_full for potential later use if needed

    # Check if there's enough data for sequence creation and splitting
    if len(X) >= TIME_STEPS + 5: # Need enough data for sequences and future target
        # --- 1. Data Scaling ---
        # Create scaler and fit on the features
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index) # Keep as DataFrame for slicing

        print(f"Features scaled for {ticker}. Scaled data shape: {X_scaled_df.shape}")

        # --- 2. Create Sequence Data ---
        X_seq, y_seq = create_sequences(X_scaled_df, y, TIME_STEPS)
        print(f"Created sequences for {ticker}. X_seq shape: {X_seq.shape}, y_seq shape: {y_seq.shape}")

        # --- 3. Split Training and Test Sets (Time Series Method) ---
        # Ensure the split index leaves enough samples for the test set
        if len(X_seq) > int(len(X_seq) * 0.8) + 1: # Check if test set will have at least 2 samples after split
            split_index = int(len(X_seq) * 0.8)
            X_train, X_test = X_seq[:split_index], X_seq[split_index:]
            y_train, y_test = y_seq[:split_index], y_seq[split_index:]

            print(f"Split data for {ticker}: Training samples {len(X_train)}, Test samples {len(X_test)}")

            # --- 4. Build and Train Model ---
            print(f"\nTraining model for {ticker}...")
            model = build_and_train_model(X_train, y_train, X_test, y_test)

            # Only proceed if the model was trained successfully
            if model is not None:
                # --- 5. Save Trained Model ---
                model_filename = os.path.join(MODEL_DIR, f'{ticker}_stock_predictor_v4.h5')
                model.save(model_filename)
                print(f"\nModel saved for {ticker} as {model_filename}")
                trained_models[ticker] = model # Store the trained model

                # --- 6. Evaluate Model on Test Set ---
                print(f"\nEvaluating model for {ticker} on test set...")
                loss, accuracy = model.evaluate(X_test, y_test)
                print(f"Model accuracy on test set for {ticker}: {accuracy*100:.2f}%")
                evaluation_results[ticker] = {'loss': loss, 'accuracy': accuracy} # Store evaluation results

                # --- 7. Make Latest Prediction and Generate Signal ---
                print(f"\nMaking latest prediction for {ticker}...")
                # Need the last TIME_STEPS days of scaled data for the latest prediction
                last_sequence_scaled = X_scaled_df.iloc[-TIME_STEPS:].values
                last_sequence_reshaped = np.reshape(last_sequence_scaled, (1, TIME_STEPS, last_sequence_scaled.shape[1]))

                prediction = model.predict(last_sequence_reshaped)
                prediction_proba = prediction[0][0]

                print(f"Predicted probability of price increase in the next 5 days for {ticker}: {prediction_proba*100:.2f}%")

                # Generate Trading Signal
                if prediction_proba >= BUY_THRESHOLD:
                    signal = "BUY"
                elif prediction_proba <= SELL_THRESHOLD:
                    signal = "SELL"
                else:
                    signal = "HOLD/NEUTRAL"

                print(f"Trading Signal for {ticker}: {signal} (Probability: {prediction_proba:.2f}, Thresholds: Buy>={BUY_THRESHOLD}, Sell<={SELL_THRESHOLD})")
                latest_predictions[ticker] = {'probability': prediction_proba, 'signal': signal} # Store prediction and signal

            else:
                print(f"Skipping evaluation and prediction for {ticker} as model training failed.")

        else:
             print(f"Skipping training and prediction for {ticker} due to insufficient data for splitting into train/test sets.")

    else:
        print(f"Skipping processing for {ticker} due to insufficient data ({len(X)} rows) for sequence creation (requires at least {TIME_STEPS + 5} rows).")


print("\n--- Model Training and Prediction Complete for all Stocks ---")

# You can now access the results:
# trained_models: dictionary of trained Keras models
# evaluation_results: dictionary of test set loss and accuracy for each stock
# latest_predictions: dictionary of latest prediction probability and trading signal for each stock

print("\nSummary of Latest Predictions and Signals:")
for ticker, result in latest_predictions.items():
    print(f"{ticker}: Probability = {result['probability']:.2f}, Signal = {result['signal']}")


--- Model Training and Prediction for Each Stock ---

--- Processing Model for 000001.SZ ---
Features scaled for 000001.SZ. Scaled data shape: (1316, 8)
Created sequences for 000001.SZ. X_seq shape: (1306, 10, 8), y_seq shape: (1306,)
Split data for 000001.SZ: Training samples 1044, Test samples 262

Training model for 000001.SZ...
模型结构概要：


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_13 (LSTM)                  │ (None, 50)             │        11,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,101 (51.18 KB)

...

 Total params: 13,101 (51.18 KB)

 Trainable params: 13,101 (51.18 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.5057 - loss: 0.6927 - val_accuracy: 0.4167 - val_loss: 0.7217
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5373 - loss: 0.6921 - val_accuracy: 0.4167 - val_loss: 0.7284
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5466 - loss: 0.6880 - val_accuracy: 0.4167 - val_loss: 0.7227
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5656 - loss: 0.6866 - val_accuracy: 0.4167 - val_loss: 0.7236
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5428 - loss: 0.6920 - val_accuracy: 0.4167 - val_loss: 0.7133
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5514 - loss: 0.6869 - val_accuracy: 0.4167 - val_loss: 0.7208
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5747 - loss: 0.6814 - val_accuracy: 0.4167 - val_loss: 0.7176
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5680 - loss: 0.6835 - val_accuracy: 0.4167 - v


Model saved for 000300.SH as stock_models/000300.SH_stock_predictor_v4.h5

Evaluating model for 000300.SH on test set...
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5084 - loss: 0.7077 
Model accuracy on test set for 000300.SH: 48.53%

Making latest prediction for 000300.SH...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Predicted probability of price increase in the next 5 days for 000300.SH: 46.45%
Trading Signal for 000300.SH: HOLD/NEUTRAL (Probability: 0.46, Thresholds: Buy>=0.6, Sell<=0.4)

--- Model Training and Prediction Complete for all Stocks ---

Summary of Latest Predictions and Signals:
000001.SZ: Probability = 0.53, Signal = HOLD/NEUTRAL
600036.SH: Probability = 0.47, Signal = HOLD/NEUTRAL
601398.SH: Probability = 0.54, Signal = HOLD/NEUTRAL
000300.SH: Probability = 0.46, Signal = HOLD/NEUTRAL


# A股情绪增强技术 LSTM 策略

基于结合了技术指标和情绪因子的修改后的 A 股股票预测模型，以下是一个潜在的新的预测策略：

**策略名称：** A股情绪增强技术 LSTM 策略

**目标：** 利用技术分析和市场情绪的结合，预测目标 A 股股票价格的短期（未来 5 个交易日）方向（上涨或下跌）。

**目标市场：** A 股股票（特别是已定义的股票代码，例如 `000001.SZ`）。

**数据来源：**
1.  **历史股票数据：** 目标 A 股股票的每日历史数据，最好手动从通达信或其他可靠来源导入，包括“日期”和“收盘价”，以及可用的其他相关价格和交易量数据。
2.  **情绪数据：** 从新浪财经和雪球等公共在线来源获取并按日期与股票数据对齐的每日汇总情绪得分或指标。（在此实现中，模拟情绪数据用作占位符）。

**使用的技术指标：**
*   5 日移动平均线 (MA5)
*   15 日移动平均线 (MA15)
*   相对强弱指数 (RSI)
*   移动平均收敛扩散指标 (MACD)
*   布林带（中轨、上轨、下轨）

**使用的情绪因子：**
*   每日情绪得分（来自在线新闻和评论分析，或模拟）。

**预测模型：**
*   序列式长短期记忆 (LSTM) 神经网络模型。
*   输入：过去 10 天的缩放技术指标和每日情绪得分序列。
*   输出：一个概率得分（介于 0 到 1 之间），表示股票收盘价在未来 5 天上涨的可能性。

**策略规则（示例）：**

这是一个如何将模型输出用于交易策略的简化示例。实际策略需要更复杂的规则和风险管理。

*   **入场信号（潜在买入）：** 如果模型预测未来 5 天价格上涨的概率高于某个阈值（例如 > 0.6），则生成潜在买入信号。
*   **出场信号（潜在卖出/持有）：**
    *   如果模型预测价格上涨的概率低于某个阈值（例如 < 0.5），或预测下跌，则生成潜在卖出信号。
    *   或者，出场规则可以基于时间（例如，持有 5 天）、止盈目标或止损限价，而不管模型在该 5 天窗口期内的后续预测如何。
*   **风险管理：** 执行止损单，以限制价格与预测相反时的潜在损失。
*   **头寸规模：** 根据风险承受能力和对预测的信心，确定分配给每笔交易的资金量。

**策略流程：**

1.  获取最新的历史股票数据（包括“日期”和“收盘价”）以及相应的每日情绪数据。
2.  使用 `load_and_prepare_data_tongdaoxin` 函数加载和准备数据，该函数计算技术指标并合并情绪数据。
3.  缩放合并后的技术和情绪特征。
4.  为过去 `TIME_STEPS` (10) 天的缩放特征创建时间序列。
5.  将最新的序列输入到训练好的 LSTM 模型中，以获取预测概率。
6.  根据预测概率和定义的策略规则，生成交易信号（例如，买入、卖出、持有）。
7.  根据生成的信号和风险管理规则执行交易。

**评估：**

该策略的有效性需要在历史 A 股数据上进行严格的回测（使用实际日期范围 2020 年 1 月 1 日至今），并可能通过在模拟交易环境中进行前向测试来评估。关键指标将包括盈利能力、胜率、回撤等。

**进一步增强：**

*   优化情绪分析方法（例如，使用更高级的自然语言处理模型，整合不同类型的新闻/评论）。
*   探索其他技术指标或不同的回看周期。
*   优化 LSTM 模型架构和超参数。
*   整合其他类型的数据（例如，基本面数据、宏观经济指标）。
*   开发更复杂的交易规则和风险管理技术。

该策略提供了一个框架，用于在深度学习模型中使用技术和情绪因子进行 A 股股票预测。其实际表现将严重依赖于数据质量、情绪分析的有效性以及交易规则的稳健性。

后续输入股票池的具体股票及代码就可以实现预测？
是的，完全正确！

你只需要修改定义股票池的单元格 (202d7db3)，将 STOCK_POOL 列表中的示例股票代码替换为你想要预测的具体 A 股股票代码和代码。

然后，你需要确保这些股票的历史数据是可用的。如果你有这些股票的通达信导出 CSV 文件，请将它们上传到 Colab 环境中，并确保文件名与代码中预期的格式一致（例如，如果股票代码是 000001.SZ，文件名可以是 000001.SZ_daily.csv，并且 load_and_prepare_data_modified 函数中的逻辑能够找到并读取它）。

一旦你更新了 STOCK_POOL 列表并确保数据文件到位，你就可以运行执行预测的单元格 (b704a9a1)，它就会为你股票池中的每个股票生成预测结果和交易信号。